# import

In [1]:
import pandas as pd
import numpy as np

import time
import os
import sys

from astropy import config as _config 
from astropy import units as u
from astropy.coordinates import SkyCoord

from astroquery.irsa import Irsa  

/var/folders/jp/4mwsn7_x6j9gnv1hl1dztqth0000gn/T/ipykernel_6730/915632295.py:12: DeprecationWarning: the ``irsa`` module has been moved to astroquery.ipac.irsa, please update your imports.
  from astroquery.irsa import Irsa


In [2]:
from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy import constants 
from astropy.coordinates import SkyCoord
from astropy.visualization import hist
from astroML.datasets import fetch_imaging_sample, fetch_sdss_S82standards
from astroML.crossmatch import crossmatch_angular

In [3]:
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr#
from scipy import stats 

import matplotlib.pyplot as plt
from pylab import cm
import matplotlib as mpl

from collections import OrderedDict
from adjustText import adjust_text

%matplotlib inline
%config InlineBackend.figure_format='svg'
# Edit the font, font size, and axes width
mpl.rcParams['font.family'] = 'Times New Roman' #Times New Roman
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

In [4]:
#from adjustText import adjust_text

def set_ax_tick(ax):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
    ax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in', right='on')

def set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2):
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(xma))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(xmi))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(yma))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(ymi)) 

In [5]:
def drop_index(data):
    data=data.reset_index(drop=True)
    return data

## class LAMOST_Spec

In [6]:
class LAMOST_Spec():
    '''
    A simple class to load SDSS spectrum.
    
    '''
    def __init__(self, fname, redshift=None, scale_flux=False):
        """
        Initialize SdssSpec object from a SDSS 1D spectrum file.
        Parameters:
        ----------
            fname : str
                name of the fits file.
            redshift : float
                redshift of the spectrum. Should be provided if not recorded in the fits header.
            scale_flux : bool (`True` or `False`)
                When `True`, the flux and err are multiplied by 1e-17; `False` by default.
        Example:
        ----------
        sp = SdssSpec("spec-plateid-mjd-fiberid.fits", redshift=0.01)
        """
        hdu = fits.open(fname)
        basename = os.path.basename(fname)
        self.basename = basename
        hdr = hdu[0].header
        self.hdr = hdr
        self.ra=hdr['RA']          # RA 
        self.dec=hdr['DEC']        # DEC
        #self.plateid = hdr['plateid']   # SDSS plate ID
        self.mjd = hdr['MJD']           # SDSS MJD
        #self.fiberid = hdr['fiberid']   # SDSS fiber ID  
        self.obsid= hdr['OBSID']
        
        #ra=hdul_i[0].header['RA']
        #dec=hdul_i[0].header['DEC']
        #mjd=hdul_i[0].header['MJD']
    
        
        self.scale_flux = scale_flux
        if redshift is None:
            try: 
                redshift = hdu[2].data['z']
            except: 
                raise ValueError('Redshift not provided. '+ 
                                 'Please check the input parameters carefully.')
        self.redshift = redshift[0]
        
        data = hdu[1].data
        hdu.close()   
        
        #wave = 10**data['loglam']
        wave= data['WAVELENGTH'][0]
        lam=wave
        
        flux = data['FLUX'][0]
        ivar = data['IVAR'][0]
        
        andmask = data['ANDMASK'][0]
        ormask =data['ORMASK'][0]
        
        err = 1./np.sqrt(ivar)
        
        wave_index1 = np.where((3900<lam) &(lam < 4100))
        wave_index2 = np.where((7900<lam) &(lam < 8100))
        
        flux1 = np.median(flux[wave_index1])
        flux2 = np.median(flux[wave_index2])
        
        flux_max     = max([flux1,flux2])
        flux_min     = min([flux1,flux2])
        
        lam_min=int(min(lam))+1
        lam_max=int(max(lam))
        lam_bin=np.arange(0,lam_max-lam_min,1)+lam_min
        
        
        self.wave = wave
        self.flux = flux
        self.err = err    
        
        
    def plot(self):
        fig=plt.figure()
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(111)  
        
        ax.plot(self.wave, self.flux)
        
        plt.xlabel(r'Wavelength [$\mathrm{\AA}$]')
        
        if self.scale_flux == True:
            plt.ylabel(r'Flux [$\mathrm{erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        else:
            plt.ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        plt.title(self.basename[:-5])
        
        return fig,ax
        
    def plot_restframe(self):
        fig=plt.figure()
        #fig = plt.figure(figsize=(12,4))
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(111) 
        
        ax.plot(self.wave/(1+self.redshift), self.flux)
        plt.xlabel(r'Wavelength/(1+z) [$\mathrm{\AA}$]')
        
        if self.scale_flux == False:
            plt.ylabel(r'Flux [$\mathrm{erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        else:
            plt.ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        
        plt.title(self.basename[:-5]+' z=%.3f'%self.redshift)
        
        return fig,ax        

In [7]:
class LAMOST_Spec_Jinscaled():
    '''
    A simple class to load SDSS spectrum.
    
    '''
    def __init__(self, fname, redshift=None, scale_flux=False):
        """
        Initialize SdssSpec object from a SDSS 1D spectrum file.
        Parameters:
        ----------
            fname : str
                name of the fits file.
            redshift : float
                redshift of the spectrum. Should be provided if not recorded in the fits header.
            scale_flux : bool (`True` or `False`)
                When `True`, the flux and err are multiplied by 1e-17; `False` by default.
        Example:
        ----------
        sp = SdssSpec("spec-plateid-mjd-fiberid.fits", redshift=0.01)
        """
        hdu = fits.open(fname)
        basename = os.path.basename(fname)
        self.basename = basename
        hdr = hdu[0].header
        self.hdr = hdr
        self.ra=hdr['RA']          # RA 
        self.dec=hdr['DEC']        # DEC
        #self.plateid = hdr['plateid']   # SDSS plate ID
        self.mjd = hdr['MJD']           # SDSS MJD
        #self.fiberid = hdr['fiberid']   # SDSS fiber ID  
        self.obsid= hdr['OBSID']
        
        #ra=hdul_i[0].header['RA']
        #dec=hdul_i[0].header['DEC']
        #mjd=hdul_i[0].header['MJD']
    
        
        self.scale_flux = scale_flux
        if redshift is None:
            try: 
                #redshift = hdu[2].data['z']
                redshift = hdr['Z']
            except: 
                raise ValueError('Redshift not provided. '+ 
                                 'Please check the input parameters carefully.')
        self.redshift = redshift[0]
        
        data = hdu[0].data
        hdu.close()   
        
        #wave = 10**data['loglam']
        #wave= data['WAVELENGTH'][0]
        #lam=wave
        
        wave= data[2]
        lam=wave
        flux=data[0]
        #flux = data['FLUX'][0]
        #ivar = data['IVAR'][0]
        
        #andmask = data['ANDMASK'][0]
        #ormask =data['ORMASK'][0]
        
        #err = 1./np.sqrt(ivar)
        
            
        self.wave = wave
        self.flux = flux
        #self.err = err    
        
        
    def plot(self):
        fig=plt.figure()
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(111)  
        
        ax.plot(self.wave, self.flux)
        
        plt.xlabel(r'Wavelength [$\mathrm{\AA}$]')
        
        if self.scale_flux == True:
            plt.ylabel(r'Flux [$\mathrm{erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        else:
            plt.ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        plt.title(self.basename[:-5])
        
        return fig,ax
        
    def plot_restframe(self):
        fig=plt.figure()
        #fig = plt.figure(figsize=(12,4))
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(111) 
        
        ax.plot(self.wave/(1+self.redshift), self.flux)
        plt.xlabel(r'Wavelength/(1+z) [$\mathrm{\AA}$]')
        
        if self.scale_flux == False:
            plt.ylabel(r'Flux [$\mathrm{erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        else:
            plt.ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
        
        plt.title(self.basename[:-5]+' z=%.3f'%self.redshift)
        
        return fig,ax        

# PATH

In [8]:
CLAGN_statistic_plot_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/'
CLAGN_wisedata_dir='/Users/lyubing/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'
AGN_wisedata_dir='/Volumes/ExtremeSSD/data_source/WISE_data'


LLAGN_wisedata_dir='%s/wise_data_search/LLAGN'%AGN_wisedata_dir
QSO_wisedata_dir='%s/wise_data_search/QSO'%AGN_wisedata_dir
AGN_MAXIdata_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/MAXIlc/MAXI/'#%s_MAXI_1day_all.dat'%name

wisedata_path='/Users/lyubing/blog/brettlv.github.io/pythoncode/WISEDATA'
wisedata_path_name='/Users/lyubing/blog/brettlv.github.io/pythoncode/WISEDATA'

save_lc_img_dir=os.path.join(CLAGN_statistic_plot_dir,'WISE_lc')
save_lc_img_dir_RGOIII=os.path.join(CLAGN_statistic_plot_dir,'WISE_lc_RGOIII')



Fermi_bll_wisedata_dir='%s/WISE_fermi_blazars/BLL'%AGN_wisedata_dir
Fermi_fsrq_wisedata_dir='%s/WISE_fermi_blazars/FSRQ'%AGN_wisedata_dir
CLB_wisedata_dir='%s/WISE_CLB'%AGN_wisedata_dir

CLBcand_wisedata_dir='%s/WISE_CLB_candidates'%AGN_wisedata_dir



In [9]:
datawise_dir_BFF ='/Users/lyubing/Downloads/TDE_ambiguous nuclear transient(ANT)/Bowen Fluorescence Flare (BFF) class/'
datawise_dir_GALEX='/Users/lyubing/Downloads/GALEX_variable/'

!open '/Users/lyubing/Downloads/TDE_ambiguous nuclear transient(ANT)/Bowen Fluorescence Flare (BFF) class/'

save_lc_img_dir

In [ ]:
AGN_wisedata_dir

In [ ]:
ls /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

## time_convert

In [10]:
from astropy.time import Time
from astropy.io import fits
import time
import datetime
from matplotlib.pyplot import MultipleLocator
import matplotlib.dates as mdates

def datetime2mjd(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    
    x=mdates.date2num(x)
    y = x - mjd_minus_mdates_num   
    return y

def mjd2datetime(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    y= x + mjd_minus_mdates_num
    y= mdates.num2date(y)
    return y



def datenums2mjd(x):
    #x=mdates.date2num(x)
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    y = x - mjd_minus_mdates_num   
    return y

def mjd2numsdate(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    
    y= x + mjd_minus_mdates_num
    #y= mdates.num2date(y)
    return y


def convert_xaxis_mjd(time):
    return Time(time).mjd   

def convert_xaxis_time(mjd):
    return Time(mjd,format='mjd').to_datetime()


def date2yday(x):
    """
    x is in matplotlib datenums, so they are floats.
    """
    y = x - mdates.date2num(datetime(2018, 1, 1))
    return y

def yday2date(x):
    """
    return a matplotlib datenum (x is days since start of year of 2018)
    """
    y = x + mdates.date2num(datetime(2018, 1, 1))
    return y


def convert_partial_year(numbers):
    datetimes=[]
    for number in numbers:
        year = int(number)
        d = timedelta(days=(number - year)*(365 + is_leap(year)))
        day_one = datetime(year,1,1)
        date = d + day_one
        datetimes.append(date)
    return datetimes


def is_leap(year):
    if not year%4 and  year%100 or not year%400:
        return True
    return False


def convert_mjd(times):
    timesmjd=[]
    for i in times:
        timesmjd.append(Time(i).mjd)
    return timesmjd  


def convert_date(times):
    timesdate=[]
    for i in times:
        timesdate.append(Time(i,format='mjd').datetime)
    return timesdate

def convert_date_single(time):
    timedate=Time(time,format='mjd').datetime
    return timedate

In [11]:
convert_date_single(57800)

datetime.datetime(2017, 2, 16, 0, 0)

In [12]:
convert_date_single(57580)

datetime.datetime(2016, 7, 11, 0, 0)

## def plot_secax

In [13]:
from adjustText import adjust_text
import matplotlib as mpl
def set_ax_tick(ax):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
    ax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in', right='on')

def set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2):
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(xma))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(xmi))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(yma))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(ymi))
    
def plot_secax(ax,mi_interval=365,ma_interval=365*2,rotation=30,):
    secax1 = ax.secondary_xaxis('top', functions=(mjd2numsdate,datenums2mjd))
    secax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    secax1.xaxis.set_major_locator(mdates.DayLocator(interval=ma_interval))
    secax1.xaxis.set_minor_locator(mdates.DayLocator(interval=mi_interval))
    secax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='out')
    secax1.xaxis.set_tick_params(which='minor', size=5, width=2, direction='out')
    secax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='out', rotation=rotation,)    
    
def set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99)):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, rotation=0,)
    handles, labels = ax.get_legend_handles_labels()
    # remove the errorbars
    #hdl = [h[0] for h in handles]
    hdl = handles
    labels_dict=dict(zip(labels, hdl)) #key,values
    by_label=OrderedDict(sorted(labels_dict.items(),key=lambda t:t[0]))
    #by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=bbox_to_anchor,
              loc=2, numpoints=1,ncol=1,fontsize=11.)

def set_mag_ylim(ax):
    bottom, top = ax.set_ylim()
    if bottom< top:
        ax.set_ylim(top,bottom)   
        
        
def set_mag_xlim(ax):
    bottom, top = ax.set_xlim()
    if bottom< top:
        ax.set_xlim(top,bottom)         

In [14]:
def get_new_range_datas(mjd1, flux1, err1,time_min,time_max):
    mjd_s=[]
    flux_s=[]
    err_s=[]
    for i,j,k in zip(mjd1, flux1, err1):
        if i>=time_min and i<time_max:
            mjd_s.append(i)
            flux_s.append(j)
            err_s.append(k)
    return np.array(mjd_s),np.array(flux_s),np.array(err_s)

## cosmo

In [15]:
from astropy.cosmology import FlatLambdaCDM,Planck13,Planck15,z_at_value
from astropy import units as u
from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.27, Ode0=0.73)
#cosmo=FlatLambdaCDM(H0=73,Om0=0.3)

## get_mag_weighted_mean_mag

In [16]:
import numpy as np
def get_mag_weighted_mean_mag(mags,e_mags):
    N=len(mags)
    mag_weighted_mean_mag =0  
    
    w_n=np.sum(1.0/e_mags**2)   
    
    for mag,e_mag in zip(mags,e_mags):
        w_i=1.0/e_mag**2
        w=w_i/w_n
        mag_weighted_mean_mag=mag_weighted_mean_mag+w*mag
                          
    return mag_weighted_mean_mag 



def get_mag_weighted_mean_e_mag(mags,e_mags):
    N=len(mags)  

    w_n=np.sum(1.0/e_mags**2)
    
    mag_weighted_mean_mag = get_mag_weighted_mean_mag(mags,e_mags)
    
    mag_weighted_mean_e_mag =0 
    
    for mag,e_mag in zip(mags,e_mags):        
        w_i=1.0/e_mag**2
        w=w_i/w_n
        
        mag_weighted_mean_e_mag=mag_weighted_mean_e_mag+w*(mag-mag_weighted_mean_mag)**2
        
    mag_weighted_mean_e_mag_=(mag_weighted_mean_e_mag/(N-1))**0.5                                 
    return mag_weighted_mean_e_mag_

## def_median_lc

In [17]:
def median_lc(data,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=100):
    #data=data.sort_values(timelabel)
    data=drop_index(data)
    mean_mjds=[]
    mean_mags=[]
    mean_mags_e=[]
    
    if len(data)>2:
        mjd_t0=data[timelabel][0]-10
        mjd_tend=data[timelabel][len(data)-1]+10

        N=int((mjd_tend-mjd_t0)//interval+1)
        #print(N)

        time_interval=np.linspace(mjd_t0,interval*N+mjd_t0,N+1)
        #print(time_interval)

        for index,i in enumerate(time_interval):
            if index < N:
                data_select=data[data[timelabel]>=time_interval[index]]
                data_select=data_select[data_select[timelabel]<time_interval[index+1]]
                data_select=drop_index(data_select)

                if len(data_select)>2:
                    #mean_mjd=np.mean(data_select[timelabel])
                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]               
                    mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
                    mag_weighted_mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
                    #mean_mag_e=np.mean(data_select[e_maglabel])

                    data_select=data_select[abs(data_select[maglabel]-mean_mag)<sigma_outlier*mag_weighted_mean_e_mag]
                    data_select=drop_index(data_select)


                    #print(len(data_select))

                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]  
                    if len(data_select)>2:
                        mean_mag=get_mag_weighted_mean_mag(mags,e_mags)               
                        mean_mag_e=get_mag_weighted_mean_e_mag(mags,e_mags)

                        mean_mjd=np.mean(data_select[timelabel])               
                        mean_mjds.append(mean_mjd)
                        mean_mags.append(mean_mag)
                        mean_mags_e.append(mean_mag_e)
                        
        mean_mjds,mean_mags,mean_mags_e=(np.array(mean_mjds),np.array(mean_mags),np.array(mean_mags_e))
        
    return mean_mjds,mean_mags,mean_mags_e


## def median_color

## get_outlier_remove

In [18]:
def get_outlier_remove_data(data,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10):
    data_remove=pd.DataFrame([])
    #data_outlier=pd.DataFrame([])
    
    #data=data.sort_values(timelabel)
    data=drop_index(data)
    mean_mjds=[]
    mean_mags=[]
    mean_mags_e=[]
    
    if len(data)>2:
        mjd_t0=data[timelabel][0]-20
        mjd_tend=data[timelabel][len(data)-1]+20

        N=int((mjd_tend-mjd_t0)//interval+1)
        #print(N)

        time_interval=np.linspace(mjd_t0,interval*N+mjd_t0,N+1)
        #print(time_interval)

        for index,i in enumerate(time_interval):
            if index < N:
                data_select=data[data[timelabel]>=time_interval[index]]
                data_select=data_select[data_select[timelabel]<time_interval[index+1]]
                data_select=drop_index(data_select)
                
                if len(data_select)>2:
                    #mean_mjd=np.mean(data_select[timelabel])
                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]               
                    mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
                    mag_weighted_mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
                    #mean_mag_e=np.mean(data_select[e_maglabel])

                    data_remove_outlier=data_select[abs(data_select[maglabel]-mean_mag)<=sigma_outlier*mag_weighted_mean_e_mag]
                    
                    data_remove_outlier=drop_index(data_remove_outlier) 
                    data_remove=data_remove.append(data_remove_outlier,ignore_index=True)                   
         
        
                    '''                 
                    data_out=data_select[abs(data_select[maglabel]-mean_mag)>sigma_outlier*mag_weighted_mean_e_mag]
                    data_out=drop_index(data_out)
                    data_outlier.append(data_out,ignore_index=True)
              
                    '''
    #data_remove=data_remove.sort_values(timelabel)
    data_remove=drop_index(data_remove)    
    return data_remove#,data_outlier                   
                    
    

## def get_outlier_data

In [19]:
def get_outlier_data(data,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10):
    data_remove=pd.DataFrame([])
    data_outlier=pd.DataFrame([])
    
    #data=data.sort_values(timelabel)
    data=drop_index(data)
    mean_mjds=[]
    mean_mags=[]
    mean_mags_e=[]
    
    if len(data)>2:
        mjd_t0=data[timelabel][0]-20
        mjd_tend=data[timelabel][len(data)-1]+20

        N=int((mjd_tend-mjd_t0)//interval+1)
        #print(N)

        time_interval=np.linspace(mjd_t0,interval*N+mjd_t0,N+1)
        #print(time_interval)

        for index,i in enumerate(time_interval):
            if index < N:
                data_select=data[data[timelabel]>=time_interval[index]]
                data_select=data_select[data_select[timelabel]<time_interval[index+1]]
                data_select=drop_index(data_select)
                
                if len(data_select)>2:
                    #mean_mjd=np.mean(data_select[timelabel])
                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]               
                    mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
                    mag_weighted_mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
                    #mean_mag_e=np.mean(data_select[e_maglabel])

                    '''data_remove_outlier=data_select[abs(data_select[maglabel]-mean_mag)<=sigma_outlier*mag_weighted_mean_e_mag]                   
                    data_remove_outlier=drop_index(data_remove_outlier) 
                    data_remove=data_remove.append(data_remove_outlier,ignore_index=True)  '''                 
         
        
                                    
                    data_out=data_select[abs(data_select[maglabel]-mean_mag)>sigma_outlier*mag_weighted_mean_e_mag]
                    data_out=drop_index(data_out)
                    data_outlier=data_outlier.append(data_out,ignore_index=True)
                                  
    #data_outlier=data_outlier.sort_values(timelabel)
    #data_outlier=drop_index(data_outlier)   
    return data_outlier#,data_outlier                   
                    
    

data_test_neo,data_test_mep

## def get_remove_neo_mep(data_test_neo,data_test_mep):


data_test_neo.columns,len(data_test_neo)

timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
data_test_neo_remove=get_outlier_remove_data(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=5)
timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
data_test_neo_remove=get_outlier_remove_data(data_test_neo_remove,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=5)


In [20]:
def get_remove_neo_mep(data_test_neo,data_test_mep):
    #neo
    if len(data_test_neo)>0:
        timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
        data_test_neo_remove=get_outlier_remove_data(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
        timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
        data_test_neo_remove=get_outlier_remove_data(data_test_neo_remove,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
    else:
        data_test_neo_remove=[]
        
    #mep
    if len(data_test_mep)>0:
        timelabel,maglabel,e_maglabel=('mjd','w1mpro_ep','w1sigmpro_ep')
        data_test_mep_remove=get_outlier_remove_data(data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
        timelabel,maglabel,e_maglabel=('mjd','w2mpro_ep','w2sigmpro_ep')
        data_test_mep_remove=get_outlier_remove_data(data_test_mep_remove,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
    else:
        data_test_mep_remove=[]
    return data_test_neo_remove,data_test_mep_remove

In [21]:
def get_outlier_neo_mep(data_test_neo,data_test_mep):
    #neo
    if len(data_test_neo)>0:    
        timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
        data_test_neo_outiler=get_outlier_data(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
        timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
        data_test_neo_outiler=get_outlier_data(data_test_neo_outiler,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
    else:
        data_test_neo_outiler=[]
    #mep
    if len(data_test_mep)>0:    
        timelabel,maglabel,e_maglabel=('mjd','w1mpro_ep','w1sigmpro_ep')
        data_test_mep_outiler=get_outlier_data(data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
        timelabel,maglabel,e_maglabel=('mjd','w2mpro_ep','w2sigmpro_ep')
        data_test_mep_outiler=get_outlier_data(data_test_mep_outiler,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10)
    else:
        data_test_mep_outiler=[]
    return data_test_neo_outiler,data_test_mep_outiler

## plot_outlier_median_lc

In [22]:
def plot_outlier_median_lc(ax,data,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=10):
    #data=data.sort_values(timelabel)
    data=drop_index(data)
    mean_mjds=[]
    mean_mags=[]
    mean_mags_e=[]
    if len(data)>2:
        mjd_t0=data[timelabel][0]-20
        mjd_tend=data[timelabel][len(data)-1]+20

        N=int((mjd_tend-mjd_t0)//interval+1)
        #print(N)

        time_interval=np.linspace(mjd_t0,interval*N+mjd_t0,N+1)
        #print(time_interval)

        for index,i in enumerate(time_interval):
            if index < N:
                data_select=data[data[timelabel]>=time_interval[index]]
                data_select=data_select[data_select[timelabel]<time_interval[index+1]]
                data_select=drop_index(data_select)

                if len(data_select)>2:
                    #mean_mjd=np.mean(data_select[timelabel])
                    mags=data_select[maglabel]
                    e_mags=data_select[e_maglabel]               
                    mean_mag=get_mag_weighted_mean_mag(mags,e_mags)
                    mag_weighted_mean_e_mag=get_mag_weighted_mean_e_mag(mags,e_mags)
                    #mean_mag_e=np.mean(data_select[e_maglabel])

                    data_select_outlier=data_select[abs(data_select[maglabel]-mean_mag)>=sigma_outlier*mag_weighted_mean_e_mag]
                    data_select_outlier=drop_index(data_select_outlier)  
                    if len(data_select_outlier)>0:
                        ax.scatter(data_select_outlier[timelabel],data_select_outlier[maglabel],
                               color='red',zorder=3,s=6)


## def plot_outliers

def plot_outliers_w1(ax,data_test_neo,data_test_mep,sigma_outlier=5):
    timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
    plot_outlier_median_lc(ax,data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    timelabel,maglabel,e_maglabel=('mjd','w1mpro_ep','w1sigmpro_ep')
    plot_outlier_median_lc(ax,data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    #return data_test_neo_w1_outlier,data_test_neo_w2_outlier,data_test_mep_w1_outlier,data_test_mep_w2_outlier

def plot_outliers_w2(ax,data_test_neo,data_test_mep,sigma_outlier=5):
    timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
    plot_outlier_median_lc(ax,data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    timelabel,maglabel,e_maglabel=('mjd','w2mpro_ep','w2sigmpro_ep')
    plot_outlier_median_lc(ax,data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

In [ ]:
print()

## def get_wise_data

a = None
if a:
    print(1)
else:
    print(2)

In [23]:
def get_wise_data(name_test,datawise_dir=CLAGN_wisedata_dir):
    #print(name_test)
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    if os.path.exists(os.path.join(datawise_dir,'wise_%s_neo.csv'%save_name)):
        data_test_neo=pd.read_csv(os.path.join(datawise_dir,'wise_%s_neo.csv'%save_name))
        data_test_neo=data_test_neo[data_test_neo['w1mpro'].notnull()]
        data_test_neo=data_test_neo[data_test_neo['w2mpro'].notnull()]
        data_test_neo=data_test_neo[data_test_neo['w1sigmpro'].notnull()]
        data_test_neo=data_test_neo[data_test_neo['w2sigmpro'].notnull()]
        data_test_neo['W1-W2']=data_test_neo['w1mpro']-data_test_neo['w2mpro']
        #data_test_neo=data_test_neo[data_test_neo['w1mpro']<15]
        #data_test_neo=data_test_neo[data_test_neo['w2mpro']<13]  
        #data_test_neo=data_test_neo[data_test_neo['w1rchi2']<2]
        #data_test_neo=data_test_neo[data_test_neo['w2rchi2']<2]
        #data_test_neo=data_test_neo[data_test_neo['cc_flags']=="b'0000'"]
        #data_test_neo=data_test_neo[data_test_neo['nb']<3]
        #data_test_neo=data_test_neo[data_test_neo['na']==0] 
        #data_test_neo=data_test_neo[data_test_neo['w1snr']>10] 
        #data_test_neo=data_test_neo[data_test_neo['w2snr']>10] 

        data_test_neo=data_test_neo[data_test_neo['qual_frame']>0]
        #print(len(data_test_neo))
        data_test_neo=(data_test_neo.loc[:, ['mjd', 'w1mpro','w2mpro',
                                     'w1sigmpro','w2sigmpro','W1-W2',
                                     ]].sort_values('mjd'))

        data_test_neo=data_test_neo.reset_index(drop=True)
    #print(data_test_neo.columns)
     
    else:
        data_test_neo=[]
    if os.path.exists(os.path.join(datawise_dir,'wise_%s_mep.csv'%save_name)):
        
        data_test_mep=pd.read_csv(os.path.join(datawise_dir,'wise_%s_mep.csv'%save_name))
        data_test_mep=data_test_mep[data_test_mep['w1mpro_ep'].notnull()]
        data_test_mep=data_test_mep[data_test_mep['w2mpro_ep'].notnull()]
        data_test_mep=data_test_mep[data_test_mep['w1sigmpro_ep'].notnull()]
        data_test_mep=data_test_mep[data_test_mep['w2sigmpro_ep'].notnull()]
        data_test_mep['W1-W2']=data_test_mep['w1mpro_ep']-data_test_mep['w2mpro_ep']
        
        #data_test_mep=data_test_mep[data_test_mep['qual_frame']>0]
        data_test_mep=data_test_mep[data_test_mep['qi_fact']>0]
        data_test_mep=data_test_mep[data_test_mep['saa_sep']>0]
        #data_test_mep=data_test_mep[data_test_mep['cc_flags']=="b'0000'"]
        #data_test_mep=data_test_mep[data_test_mep['nb']<3]
        #data_test_mep=data_test_mep[data_test_mep['na']==0] 

        #print(len(data_test_mep))
        data_test_mep=(data_test_mep.loc[:, ['mjd', 'w1mpro_ep','w2mpro_ep',
                                             'w1sigmpro_ep','w2sigmpro_ep',
                                             'W1-W2',
                                             ]].sort_values('mjd'))

        data_test_mep=data_test_mep.reset_index(drop=True)
    #print(data_test_mep.columns)
    #if not os.path.exists(save_lc_img_path):
    else:
        data_test_mep=[]
    return data_test_neo,data_test_mep
    

In [24]:
def get_M1minM2_data(data_test_neo,data_test_mep,neo_maglabel_1='w1mpro',neo_maglabel_2='w2mpro',\
                     mep_maglabel_1='w1mpro_ep',mep_maglabel_2='w2mpro_ep'):
    
    data_test_neo['delta_M']=data_test_neo[neo_maglabel_1]-data_test_neo[neo_maglabel_2]
    data_test_mep['delta_M']=data_test_mep[neo_maglabel_1]-data_test_mep[neo_maglabel_2]
    
    
    return data_test_neo['mjd','delta_M'],data_test_mep['mjd','delta_M']
    

## def get_wise_<font color=red>RADEC</font>_data 

In [25]:
def get_wise_radec_data(radeg,decdeg,datawise_dir=CLAGN_wisedata_dir):
    #print(name_test)
    save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #name=save_name

    data_test_neo=pd.read_csv(os.path.join(datawise_dir,'wise_%s_neo.csv'%save_name))
    data_test_neo=data_test_neo[data_test_neo['w1mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2mpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w1sigmpro'].notnull()]
    data_test_neo=data_test_neo[data_test_neo['w2sigmpro'].notnull()]
    data_test_neo['W1-W2']=data_test_neo['w1mpro']-data_test_neo['w2mpro']
    #data_test_neo=data_test_neo[data_test_neo['w1mpro']<15]
    #data_test_neo=data_test_neo[data_test_neo['w2mpro']<13]  
    #data_test_neo=data_test_neo[data_test_neo['w1rchi2']<2]
    #data_test_neo=data_test_neo[data_test_neo['w2rchi2']<2]
    #data_test_neo=data_test_neo[data_test_neo['cc_flags']=="b'0000'"]
    #data_test_neo=data_test_neo[data_test_neo['cc_flags']=='0000']
    #data_test_neo=data_test_neo[data_test_neo['nb']<3]
    #data_test_neo=data_test_neo[data_test_neo['na']==0] 
    data_test_neo=data_test_neo[data_test_neo['w1snr']>3] 
    data_test_neo=data_test_neo[data_test_neo['w2snr']>3]     
    #data_test_neo=data_test_neo[data_test_neo['qual_frame']>0]
    
    #print(len(data_test_neo))
    data_test_neo=(data_test_neo.loc[:, ['mjd', 'w1mpro','w2mpro',
                                 'w1sigmpro','w2sigmpro','W1-W2',
                                 ]].sort_values('mjd'))

    data_test_neo=data_test_neo.reset_index(drop=True)
    #print(data_test_neo.columns)

    data_test_mep=pd.read_csv(os.path.join(datawise_dir,'wise_%s_mep.csv'%save_name))
    data_test_mep=data_test_mep[data_test_mep['w1mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2mpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w1sigmpro_ep'].notnull()]
    data_test_mep=data_test_mep[data_test_mep['w2sigmpro_ep'].notnull()]
    data_test_mep['W1-W2']=data_test_mep['w1mpro_ep']-data_test_mep['w2mpro_ep']
       
    #data_test_mep=data_test_mep[data_test_mep['qual_frame']>5]
    data_test_mep=data_test_mep[data_test_mep['qi_fact']>0]
    data_test_mep=data_test_mep[data_test_mep['saa_sep']>0]
    #data_test_mep=data_test_mep[data_test_mep['cc_flags']=="b'0000'"]
    #data_test_mep=data_test_mep[data_test_mep['cc_flags']==0]
    #data_test_mep=data_test_mep[data_test_mep['nb']<3]
    #data_test_mep=data_test_mep[data_test_mep['na']==0] 

    #print(len(data_test_mep))
    data_test_mep=(data_test_mep.loc[:, ['mjd', 'w1mpro_ep','w2mpro_ep',
                                         'w1sigmpro_ep','w2sigmpro_ep',
                                         'W1-W2',
                                         ]].sort_values('mjd'))

    data_test_mep=data_test_mep.reset_index(drop=True)
    #print(data_test_mep.columns)
    #if not os.path.exists(save_lc_img_path):
      
    return data_test_neo,data_test_mep
    

## def get_intrinsic_var

In [26]:
epsilon_s_w1=0.024
epsilon_s_w2=0.028

def get_intrinsic_var(data,mag_label,magerr_label,epsilon_s):
    
    mags=data[mag_label]
    e_mags=data[magerr_label]
    
    mag_weighted_mean_mag = get_mag_weighted_mean_mag(mags,e_mags)    
    mag_weighted_mean_e_mag_= get_mag_weighted_mean_e_mag(mags,e_mags)
    mag_mean=mag_weighted_mean_mag
    
    length_N=len(data[mag_label])
    epsilon_square=np.average(data[magerr_label]**2)+epsilon_s**2
    Sigma_square=np.sum((data[mag_label]-mag_mean)**2)/(length_N-1)
    
    if Sigma_square>epsilon_square:
        sigma_m=np.sqrt(Sigma_square-epsilon_square)
    else:
        sigma_m=0
    
    if length_N<2:
        sigma_m=-1#print(length_N)
    return sigma_m

## def get_intrinsic_var_rebin

In [27]:
def get_intrinsic_var_rebin(data_rebin_neo,data_rebin_mep,epsilon_s):
    
    mags=data[mag_label]
    e_mags=data[magerr_label]
    
    mag_weighted_mean_mag = get_mag_weighted_mean_mag(mags,e_mags)    
    mag_weighted_mean_e_mag_= get_mag_weighted_mean_e_mag(mags,e_mags)
    mag_mean=mag_weighted_mean_mag
    
    length_N=len(mags)
    epsilon_square=np.average(data[magerr_label]**2)+epsilon_s**2
    Sigma_square=np.sum((data[mag_label]-mag_mean)**2)/(length_N-1)
    
    if Sigma_square>epsilon_square:
        sigma_m=np.sqrt(Sigma_square-epsilon_square)
    else:
        sigma_m=0
    
    if length_N<2:
        sigma_m=-1#print(length_N)
        
    return sigma_m

## def get_minmax_mag_name_one

## def get_<font color=red>$\Delta M$_radec</font>

In [28]:
def get_minmax_mag_name_one(data_test_neo,data_test_mep):    
    maxmag=0
    minmag=20
    maxmag_var=0 
    
    if len(data_test_neo)>20:    
        mean_mjds_neo,mean_mags_neo,mean_mags_e_neo=median_lc(data_test_neo,'mjd','w1mpro','w1sigmpro',interval=180,)
        maxmag=max(max(mean_mags_neo),maxmag)  
        minmag=min(min(mean_mags_neo),minmag)  
    
    if len(data_test_mep)>20: 
        mean_mjds_mep,mean_mags_mep,mean_mags_e_mep=median_lc(data_test_mep,'mjd','w1mpro_ep','w1sigmpro_ep',interval=180,)
        maxmag=max(max(mean_mags_mep),maxmag)  
        minmag=min(min(mean_mags_mep),minmag) 
        
    maxmag_var=maxmag-minmag
    return maxmag_var   

## def Wmag_to_Lum

In [29]:
def W1mag_to_Lum(w1,d_cm):
    Lum=(-48.6-w1-2.699)/2.5+np.log10(4*np.pi*d_cm**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
    return Lum #logLum

def W2mag_to_Lum(w2,d_cm):
    Lum=(-48.6-w2-3.339)/2.5+np.log10(4*np.pi*d_cm**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)
    return Lum #logLum
 

def W3mag_to_Lum(w3,d_cm):
    Lum=(-48.6-w3-5.174)/2.5+np.log10(4*np.pi*d_cm**2)+np.log10((constants.c/(12*u.um)).to(u.Hz).value)
    return Lum #logLum

def W4mag_to_Lum(w4,d_cm):
    Lum=(-48.6-w4-6.620)/2.5+np.log10(4*np.pi*d_cm**2)+np.log10((constants.c/(22*u.um)).to(u.Hz).value)
    return Lum #logLum

## get_rebinwise_data

In [30]:
def get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100):
    #print(name_test)
    #search_name=name_test.replace(' ','') 
    #save_name=search_name.strip()
    
    
    data_rebin_neo=pd.DataFrame([])
    data_rebin_mep=pd.DataFrame([])
    
    timelabel,maglabel,e_maglabel=('mjd','w1mpro','w1sigmpro')
    band='W1'
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    data_rebin_neo['t_W1']=mean_mjds
    data_rebin_neo['W1']=mean_mags
    data_rebin_neo['e_W1']=mean_mags_e
    
    timelabel,maglabel,e_maglabel=('mjd','w2mpro','w2sigmpro')
    band='W2' 
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_neo,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)
    
    data_rebin_neo['t_W2']=mean_mjds
    data_rebin_neo['W2']=mean_mags
    data_rebin_neo['e_W2']=mean_mags_e
    
#############################################################
    timelabel,maglabel,e_maglabel=('mjd','w1mpro_ep','w1sigmpro_ep')
    band='W1'
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)

    data_rebin_mep['t_W1']=mean_mjds
    data_rebin_mep['W1']=mean_mags
    data_rebin_mep['e_W1']=mean_mags_e
    
    timelabel,maglabel,e_maglabel=('mjd','w2mpro_ep','w2sigmpro_ep')
    band='W2' 
    mean_mjds,mean_mags,mean_mags_e=median_lc(data_test_mep,timelabel,maglabel,e_maglabel,interval=180,sigma_outlier=sigma_outlier)
    
    data_rebin_mep['t_W2']=mean_mjds
    data_rebin_mep['W2']=mean_mags
    data_rebin_mep['e_W2']=mean_mags_e
        
        
    return data_rebin_neo,data_rebin_mep
    

data_rebin_neo,data_rebin_mep

In [ ]:
save_lc_img_dir

## def plot_mag 

In [31]:
def plot_mag_w1(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w1mpro'],data_test_neo['w1sigmpro'],
                    fillstyle="none",
                    ls='',marker='o',color='red',label='W1',zorder=0)
    if len(data_test_mep)>0:    
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],data_test_mep['w1sigmpro_ep'],
                    fillstyle="none",
                    ls='',marker='o',color='red',label='W1',zorder=0)
    #ax.scatter(data_test_neo_w1_outlier['mjd'],data_test_neo_w1_outlier['w1mpro'],marker='.',color='red',zorder=1)
    #ax.scatter(data_test_mep_w1_outlier['mjd'],data_test_mep_w1_outlier['w1mpro_ep'],marker='.',color='red',zorder=1)

def plot_mag_w2(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w2mpro'],data_test_neo['w2sigmpro'],
                    fillstyle="none",
                    ls='',marker='o',color='blue',label='W2',zorder=0)
    if len(data_test_mep)>0:
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],data_test_mep['w2sigmpro_ep'],
                    fillstyle="none",
                    ls='',marker='o',color='blue',label='W2',zorder=0)
    #ax.scatter(data_test_neo_w2_outlier['mjd'],data_test_neo_w2_outlier['w2mpro'],marker='.',color='red',zorder=1)
    #ax.scatter(data_test_mep_w2_outlier['mjd'],data_test_mep_w2_outlier['w2mpro_ep'],marker='.',color='red',zorder=1)
    
def plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep):
    if len(data_rebin_neo)>0:
        ax.errorbar(data_rebin_neo['t_W1'],data_rebin_neo['W1'],data_rebin_neo['e_W1'],
                    ls='',marker='^',color='red',label='W1',zorder=2)
    if len(data_rebin_mep)>0:
        ax.errorbar(data_rebin_mep['t_W1'],data_rebin_mep['W1'],data_rebin_mep['e_W1'],
                    ls='',marker='^',color='red',label='W1',zorder=2)

def plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep):  
    if len(data_rebin_neo)>0:
        ax.errorbar(data_rebin_neo['t_W2'],data_rebin_neo['W2'],data_rebin_neo['e_W2'],
                    ls='',marker='+',color='blue',label='W2',zorder=2)
    if len(data_rebin_mep)>0:
        ax.errorbar(data_rebin_mep['t_W2'],data_rebin_mep['W2'],data_rebin_mep['e_W2'],
                    ls='',marker='+',color='blue',label='W2',zorder=2)


## def plot_mag_w1_filled

In [32]:
def plot_mag_w1_filled(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w1mpro'],data_test_neo['w1sigmpro'],
                    markersize=3,
                    ls='',marker='.',color='red',label='W1',zorder=0)
        
    if len(data_test_mep)>0:    
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],data_test_mep['w1sigmpro_ep'],
                    markersize=3,
                    ls='',marker='.',color='red',label='W1',zorder=0)
        
    #ax.scatter(data_test_neo_w1_outlier['mjd'],data_test_neo_w1_outlier['w1mpro'],marker='.',color='red',zorder=1)
    #ax.scatter(data_test_mep_w1_outlier['mjd'],data_test_mep_w1_outlier['w1mpro_ep'],marker='.',color='red',zorder=1)

def plot_mag_w2_filled(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w2mpro'],data_test_neo['w2sigmpro'],
                    markersize=3,
                    ls='',marker='.',color='blue',label='W2',zorder=0)
        
    if len(data_test_mep)>0:
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],data_test_mep['w2sigmpro_ep'],
                    markersize=3,
                    ls='',marker='.',color='blue',label='W2',zorder=0)

In [33]:
def plot_mag_w1_outlier(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w1mpro'],data_test_neo['w1sigmpro'],
                    markersize=4,
                    ls='',marker='*',color='red',label='W1',zorder=5)
        
    if len(data_test_mep)>0:    
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],data_test_mep['w1sigmpro_ep'],
                    markersize=4,
                    ls='',marker='*',color='red',label='W1',zorder=5)
        
    #ax.scatter(data_test_neo_w1_outlier['mjd'],data_test_neo_w1_outlier['w1mpro'],marker='.',color='red',zorder=1)
    #ax.scatter(data_test_mep_w1_outlier['mjd'],data_test_mep_w1_outlier['w1mpro_ep'],marker='.',color='red',zorder=1)

def plot_mag_w2_outlier(ax,data_test_neo,data_test_mep):
    if len(data_test_neo)>0:
        ax.errorbar(data_test_neo['mjd'],data_test_neo['w2mpro'],data_test_neo['w2sigmpro'],
                    markersize=4,
                    ls='',marker='*',color='blue',label='W2',zorder=5)
        
    if len(data_test_mep)>0:
        ax.errorbar(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],data_test_mep['w2sigmpro_ep'],
                    markersize=4,
                    ls='',marker='*',color='blue',label='W2',zorder=5)

## def wise_plot_lc

!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

In [34]:
def wise_plot_lc(name_test,save_lc_img_dir=save_lc_img_dir,datawise_dir='/Users/lyubing/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir)
    #data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)    
    
    search_name=name_test.replace(' ','') 
    save_name=search_name.strip()
    save_lc_img_path=os.path.join(save_lc_img_dir,'%s_wise.png'%save_name)
    
    fig = plt.figure(figsize=(8,4))
    fig.subplots_adjust(hspace=0.0, wspace = 0.0)
    ax = fig.add_subplot(111)
    
    #plot_mag_w1(ax,data_test_neo,data_test_mep)
    #plot_mag_w2(ax,data_test_neo,data_test_mep)
    
    plot_mag_w1_filled(ax,data_test_neo,data_test_mep)
    plot_mag_w2_filled(ax,data_test_neo,data_test_mep) 
    #plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
    #plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep) 
    
    ''' 
    data_test_neo_remove,data_test_mep_remove=get_remove_neo_mep(data_test_neo,data_test_mep)
    plot_mag_w1_filled(ax,data_test_neo_remove,data_test_mep_remove)
    plot_mag_w2_filled(ax,data_test_neo_remove,data_test_mep_remove)   
    data_test_neo_outlier,data_test_mep_outlier=get_outlier_neo_mep(data_test_neo,data_test_mep)
    plot_mag_w1_outlier(ax,data_test_neo_outlier,data_test_mep_outlier)
    plot_mag_w2_outlier(ax,data_test_neo_outlier,data_test_mep_outlier) 
    '''    
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')
    
    
    plot_secax(ax,mi_interval=365,ma_interval=365*2)   
    handles, labels = ax.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.8, 0.9),ncol=1,
          loc=2,fontsize=10)

    bottom, top = ax.set_ylim() 
    if bottom< top:
        ax.set_ylim(top,bottom)

    plt.xlabel('mjd')
    plt.ylabel(r'$mag$')

    ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
    
    if not os.path.exists(save_lc_img_path):
        plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()

In [ ]:
print(save_lc_img_dir)

## def wise_plotradec_lc

In [35]:
def wise_plotradec_lc(radeg,decdeg,datawise_dir,plotwise_dir):
    data_test_neo,data_test_mep=get_wise_radec_data(radeg,decdeg,datawise_dir)
    #data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)    
    save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    name_test=save_name
    
    save_lc_img_path=os.path.join(plotwise_dir,'%s_wise.png'%save_name)
    
    fig = plt.figure(figsize=(8,4))
    fig.subplots_adjust(hspace=0.0, wspace = 0.0)
    ax = fig.add_subplot(111)
    
    #plot_mag_w1(ax,data_test_neo,data_test_mep)
    #plot_mag_w2(ax,data_test_neo,data_test_mep)

    data_test_neo_remove,data_test_mep_remove=get_remove_neo_mep(data_test_neo,data_test_mep)
    plot_mag_w1_filled(ax,data_test_neo_remove,data_test_mep_remove)
    plot_mag_w2_filled(ax,data_test_neo_remove,data_test_mep_remove)
    #plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
    #plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)
    
    data_test_neo_outlier,data_test_mep_outlier=get_outlier_neo_mep(data_test_neo,data_test_mep)
    plot_mag_w1_outlier(ax,data_test_neo_outlier,data_test_mep_outlier)
    plot_mag_w2_outlier(ax,data_test_neo_outlier,data_test_mep_outlier)   
    
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')


    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')
    plot_secax(ax,mi_interval=365,ma_interval=365*2)
    
    handles, labels = ax.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.8, 0.9),ncol=1,
          loc=2,fontsize=10)

    bottom, top = ax.set_ylim() 
    if bottom< top:
        ax.set_ylim(top,bottom)

    plt.xlabel('mjd')
    plt.ylabel(r'$mag$')

    ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
    plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()

In [ ]:
data_test_neo=[]
len(data_test_neo)

## def wise_rebin_lc 

In [36]:
def wise_rebin_lc(name_test,save_lc_img_dir=save_lc_img_dir,datawise_dir='/Users/lyubing/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir)
    
    if len(data_test_neo)>20:
        data_test_neo,data_test_mep=get_remove_neo_mep(data_test_neo,data_test_mep)
        data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100)    

        #data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)

        search_name=name_test.replace(' ','') 
        save_name=search_name.strip()
        save_lc_img_path=os.path.join(save_lc_img_dir,'%s_rebin.png'%save_name)

        fig = plt.figure(figsize=(8,4))
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(111)

        #plot_mag_w1(ax,data_test_neo,data_test_mep)
        #plot_mag_w2(ax,data_test_neo,data_test_mep)

        plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
        plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)

        #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
        #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

        plot_secax(ax,mi_interval=365,ma_interval=365*2)
        #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
        #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

        handles, labels = ax.get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
              loc=2,fontsize=10)

        bottom, top = ax.set_ylim() 
        if bottom< top:
            ax.set_ylim(top,bottom)

        plt.xlabel('mjd')
        plt.ylabel(r'$mag$')

        ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
        if not os.path.exists(save_lc_img_path):
            plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
        plt.close()

## def wise_rebin_lcandcolor

In [37]:
def wise_rebin_lcandcolor(name_test,save_lc_img_dir=save_lc_img_dir,datawise_dir='/Users/lyubing/blog/brettlv.github.io/pythoncode/changinglookAGN/CLAGN_wise_data/'):
    data_test_neo,data_test_mep=get_wise_data(name_test,datawise_dir)
    
    if len(data_test_neo)>20:
        data_test_neo,data_test_mep=get_remove_neo_mep(data_test_neo,data_test_mep)
        data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100)    

        #data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)

        search_name=name_test.replace(' ','') 
        save_name=search_name.strip()
        save_lc_img_path=os.path.join(save_lc_img_dir,'%s_rebin_lc_color.png'%save_name)

        fig = plt.figure(figsize=(8,8))
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(211)

        #plot_mag_w1(ax,data_test_neo,data_test_mep)
        #plot_mag_w2(ax,data_test_neo,data_test_mep)

        plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
        plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)

        #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
        #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

        plot_secax(ax,mi_interval=365,ma_interval=365*2)
        #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
        #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

        
        
        
        
        
        handles, labels = ax.get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
              loc=2,fontsize=10)

        bottom, top = ax.set_ylim() 
        if bottom< top:
            ax.set_ylim(top,bottom)
            
        
        
        ax.set_ylabel(r'$mag$')
        ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
        ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',labelsize=0)

        
        ax1 = fig.add_subplot(212,sharex=ax)
        ax1.set_xlabel('mjd')
        
        
        ax1.errorbar(data_rebin_neo['t_W1'],data_rebin_neo['W1']-data_rebin_neo['W2'],ls='',marker='.',color='black')        
        ax1.errorbar(data_rebin_mep['t_W1'],data_rebin_mep['W1']-data_rebin_mep['W2'],ls='',marker='.',color='black')
        
        ax1.set_ylabel(r'$W1-W2$')
        set_ax_tick(ax1)
        
        if not os.path.exists(save_lc_img_path):
            plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
        plt.close()

['t_W1'],['W1'],['e_W1'],['t_W2'].['W2'],['e_W2'],

In [ ]:
pwd

## def wise_radec_rebin_lc

In [38]:
def wise_radec_rebin_lc(radeg,decdeg,datawise_dir,plotwise_dir):
    
    data_test_neo,data_test_mep=get_wise_radec_data(radeg,decdeg,datawise_dir)
    if len(data_test_neo)>20:
        
        data_test_neo,data_test_mep=get_remove_neo_mep(data_test_neo,data_test_mep)
        data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100)    

        #data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)

        save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
        name_test=save_name
        save_lc_img_path=os.path.join(plotwise_dir,'%s_rebin.png'%save_name)

        fig = plt.figure(figsize=(8,4))
        fig.subplots_adjust(hspace=0.0, wspace = 0.0)
        ax = fig.add_subplot(111)

        #plot_mag_w1(ax,data_test_neo,data_test_mep)
        #plot_mag_w2(ax,data_test_neo,data_test_mep)

        plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
        plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)

        #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
        #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

        plot_secax(ax,mi_interval=365,ma_interval=365*2)
        #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
        #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

        handles, labels = ax.get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
              loc=2,fontsize=10)

        bottom, top = ax.set_ylim() 
        if bottom< top:
            ax.set_ylim(top,bottom)

        plt.xlabel('mjd')
        plt.ylabel(r'$mag$')

        ax.text(0.5, 0.9,name_test , horizontalalignment = 'center', verticalalignment = 'center',transform = ax.transAxes, fontsize = 10)
        if not os.path.exists(save_lc_img_path):
            plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
        plt.close()

# Surveyfits_path

In [39]:
# wisedata_path

Surveyfits_root='/Volumes/ExtremeSSD/Survey_fits/'
Surveyfits_path='/Volumes/ExtremeSSD/Survey_fits/Survey_fits/Survey_fits'


wisedata_path_LoTSS='%s/FadingGalaxy_wisedata/FadingGalaxy_LoTSS_wisedata'%Surveyfits_root
if not os.path.exists(wisedata_path_LoTSS):
    os.mkdir(wisedata_path_LoTSS)

wiseplot_path_LoTSS='%s/FadingGalaxy_wisedata/FadingGalaxy_LoTSS_wiseplot'%Surveyfits_root
if not os.path.exists(wiseplot_path_LoTSS):
    os.mkdir(wiseplot_path_LoTSS)
    
wise_rebin_plot_path_LoTSS='%s/FadingGalaxy_wisedata/FadingGalaxy_LoTSS_wiseplot_rebin'%Surveyfits_root
if not os.path.exists(wise_rebin_plot_path_LoTSS):
    os.mkdir(wise_rebin_plot_path_LoTSS)


wisedata_path_GGSM='%s/FadingGalaxy_wisedata/FadingGalaxy_GGSM_wisedata'%Surveyfits_root
if not os.path.exists(wisedata_path_GGSM):
    os.mkdir(wisedata_path_GGSM)

wiseplot_path_GGSM='%s/FadingGalaxy_wisedata/FadingGalaxy_GGSM_wiseplot'%Surveyfits_root
if not os.path.exists(wiseplot_path_GGSM):
    os.mkdir(wiseplot_path_GGSM)
    
wise_rebin_plot_path_GGSM='%s/FadingGalaxy_wisedata/FadingGalaxy_GGSM_wiseplot_rebin'%Surveyfits_root
if not os.path.exists(wise_rebin_plot_path_GGSM):
    os.mkdir(wise_rebin_plot_path_GGSM)

wisedata_path_TGSS='%s/FadingGalaxy_wisedata/FadingGalaxy_TGSS_wisedata'%Surveyfits_root
if not os.path.exists(wisedata_path_TGSS):
    os.mkdir(wisedata_path_TGSS)

wiseplot_path_TGSS='%s/FadingGalaxy_wisedata/FadingGalaxy_TGSS_wiseplot'%Surveyfits_root
if not os.path.exists(wiseplot_path_TGSS):
    os.mkdir(wiseplot_path_TGSS)
    
wise_rebin_plot_path_TGSS='%s/FadingGalaxy_wisedata/FadingGalaxy_TGSS_wiseplot_rebin'%Surveyfits_root
if not os.path.exists(wise_rebin_plot_path_TGSS):
    os.mkdir(wise_rebin_plot_path_TGSS)


os.mkdir('/Users/brettlv/blog/brettlv.github.io/pythoncode/WISEDATA')

# mission_table

In [ ]:
from astroquery.heasarc import Heasarc, Conf
heasarc = Heasarc()
mission_table = heasarc.query_mission_list()
mission_table_pd=mission_table.to_pandas()

mission_table_pd[mission_table_pd['Mission'].str.contains('SWIFT')]

#  download_wise

## def download_wise_name

In [40]:
#Irsa.ROW_LIMIT = 1000 # value of new row limit here.
#Irsa.TIMEOUT = 120
#radius=2*u.arcsec default 10*u.arcsec


def download_wise(name,wisedata_path):

    #print(i)
    search_name=name.replace(' ','') 
    save_name=name.strip().replace(' ','') 
   
    if not os.path.exists('%s/wise_%s_mep.csv'%(wisedata_path,save_name)):
        print(search_name)
        try:
            Irsa.ROW_LIMIT = 1000
            Irsa.TIMEOUT = 120
            table_wise_mep=Irsa.query_region(search_name,catalog='allwise_p3as_mep',spatial='Cone',radius=10*u.arcsec,)
            table_wise_mep.write('%s/wise_%s_mep.csv'%(wisedata_path,save_name), format='csv')
            time.sleep(10+np.random.randint(5,10))
        except Exception:
            print(name,'mep')
            #continue
            #raise             
    
    if not os.path.exists('%s/wise_%s_neo.csv'%(wisedata_path,save_name)):
        try:
            Irsa.ROW_LIMIT = 1000
            Irsa.TIMEOUT = 120

            table_wise_neo=Irsa.query_region(search_name,catalog='neowiser_p1bs_psd',spatial='Cone',radius=10*u.arcsec,)  
            table_wise_neo.write('%s/wise_%s_neo.csv'%(wisedata_path,save_name), format='csv')       
            time.sleep(10+np.random.randint(5,10))   
        except Exception:
            #continue
            print(name,'neo')
            #raise                

# def download_wise_withradec

from astropy import config as _config
from astroquery.irsa import Irsa   
from astropy import units as u
from astropy.coordinates import SkyCoord

In [41]:
#Irsa.ROW_LIMIT = 1000 # value of new row limit here.
#Irsa.TIMEOUT = 120
#radius=2*u.arcsec default 10*u.arcsec

def download_wise_withradec(radeg,decdeg,wisedata_path):

    #print(i)
    #search_name=name.replace(' ','') 
    #save_name=name.strip().replace(' ','') 
    save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    name=save_name
    
    if not os.path.exists('%s/wise_%s_mep.csv'%(wisedata_path,save_name)):
        #print(search_name)
        try:
            Irsa.ROW_LIMIT = 1000
            Irsa.TIMEOUT = 120
            table_wise_mep=Irsa.query_region(SkyCoord(radeg,
                             decdeg, unit=(u.deg,u.deg),frame='icrs'),
                            catalog='allwise_p3as_mep',
                            spatial='Cone',radius=2*u.arcsec,)
            
            
                           
            table_wise_mep.write('%s/wise_%s_mep.csv'%(wisedata_path,save_name), format='csv')
            time.sleep(10+np.random.randint(5,10))
        except Exception as e:
            print(name,'mep')
            print(e)
            #continue
            #raise             
    
    if not os.path.exists('%s/wise_%s_neo.csv'%(wisedata_path,save_name)):
        try:
            Irsa.ROW_LIMIT = 1000
            Irsa.TIMEOUT = 120

            table_wise_neo=Irsa.query_region(SkyCoord(radeg,
                             decdeg, unit=(u.deg,u.deg),frame='icrs'),
                             catalog='neowiser_p1bs_psd',spatial='Cone',radius=2*u.arcsec,)  
            table_wise_neo.write('%s/wise_%s_neo.csv'%(wisedata_path,save_name), format='csv')       
            time.sleep(10+np.random.randint(5,10))   
        except Exception:
            #continue
            print(name,'neo')
            #raise                

In [ ]:
31.5666250  -0.2914444 02h06m15.990s -00d17m29.20s

In [ ]:
c1=SkyCoord(31.5666250, -0.2914444, unit=(u.deg,u.deg),frame='fk5')

In [ ]:
c1.ra.hms  

In [ ]:
c1.dec.dms  

In [ ]:
c1.dec.degree,c1.dec.radian  

In [ ]:
c1.to_string('decimal')

In [ ]:
c1.to_string('dms')

In [ ]:
c1.to_string('hmsdms')

In [ ]:
c1.galactic.l, c1.galactic.b 

## DESI_CLAGN

## DESI_CLAGN0

In [ ]:
DESI_CLAGN0=pd.read_excel('/Users/lyubing/Downloads/DESI_CLAGN.xlsx',sheet_name='table4',header=None)

In [ ]:
DESI_CLAGN0.columns=['Name','ra','dec','redshift','g','r','z','mjd1','mjd2','N','R','F','Transition','Line']

In [ ]:
DESI_CLAGN0

In [ ]:
#input empty with and without spaces string
s = ""
str = "    "

if not s:
    print("string is empty")
else:
    print("string is not empty")
    
if not str:
    print("string is empty")
else:
    print("string is not empty")

In [ ]:
len(DESI_CLAGN0.loc[0,'Name'])

In [ ]:
DESI_CLAGN_multi=[]

for index,Name in enumerate(DESI_CLAGN0['Name']):   
    if len(DESI_CLAGN0.loc[index,'Name'])<2:
        if len(DESI_CLAGN0.loc[index-1,'Name'])>2:        
            if DESI_CLAGN0.loc[index-1,'Name'] not in DESI_CLAGN_multi:
                DESI_CLAGN_multi.append(DESI_CLAGN0.loc[index-1,'Name'])
        
        

In [ ]:
len(DESI_CLAGN_multi)

In [ ]:
DESI_CLAGN_multi

In [ ]:
os.listdir(wisedata_path)

In [ ]:
name='SDSSJ115103.77+530140.6'
name=name.strip()
D_cm_115103= cosmo.luminosity_distance(z=0.5475).to(u.cm)
data_test_neo,data_test_mep=get_wise_data(name,wisedata_path)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_115103)
datalag_w1_115103,datalag_w2_115103=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

fig = plt.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)


#BH_mass_115103=1e6*u.solMass
#Ledd_115103=1.26*10**38*BH_mass_115103/u.solMass

ax.errorbar(datalag_w1_115103['t'],datalag_w1_115103['flux'],datalag_w1_115103['err'],label='w1',ls=':',marker='*')
ax.errorbar(datalag_w2_115103['t'],datalag_w2_115103['flux'],datalag_w2_115103['err'],label='w2',ls=':',marker='*')

#ax.axhline(np.log10(Ledd_115103),ls='-.',label='Ledd') #+np.log10(9)
ax.set_xlabel('MJD')
ax.set_ylabel('log(L_MIR)')
ax.set_title('%s z=0.5475'%name)
set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99))


In [ ]:
name='SDSSJ155426.13+200527.7'
name=name.strip()
D_cm_115103= cosmo.luminosity_distance(z=0.5215).to(u.cm)
data_test_neo,data_test_mep=get_wise_data(name,wisedata_path)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_115103)
datalag_w1_115103,datalag_w2_115103=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

fig = plt.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)


#BH_mass_115103=1e6*u.solMass
#Ledd_115103=1.26*10**38*BH_mass_115103/u.solMass

ax.errorbar(datalag_w1_115103['t'],datalag_w1_115103['flux'],datalag_w1_115103['err'],label='w1',ls=':',marker='*')
ax.errorbar(datalag_w2_115103['t'],datalag_w2_115103['flux'],datalag_w2_115103['err'],label='w2',ls=':',marker='*')

#ax.axhline(np.log10(Ledd_115103),ls='-.',label='Ledd') #+np.log10(9)
ax.set_xlabel('MJD')
ax.set_ylabel('log(L_MIR)')
ax.set_title('%s z=0.5215'%name)
set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99))


In [ ]:
name='SDSSJ141535.46+022338.7'
name=name.strip()
D_cm_115103= cosmo.luminosity_distance(z=0.3519).to(u.cm)
data_test_neo,data_test_mep=get_wise_data(name,wisedata_path)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_115103)
datalag_w1_115103,datalag_w2_115103=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

fig = plt.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)


#BH_mass_115103=1e6*u.solMass
#Ledd_115103=1.26*10**38*BH_mass_115103/u.solMass

ax.errorbar(datalag_w1_115103['t'],datalag_w1_115103['flux'],datalag_w1_115103['err'],label='w1',ls=':',marker='*')
ax.errorbar(datalag_w2_115103['t'],datalag_w2_115103['flux'],datalag_w2_115103['err'],label='w2',ls=':',marker='*')

#ax.axhline(np.log10(Ledd_115103),ls='-.',label='Ledd') #+np.log10(9)
ax.set_xlabel('MJD')
ax.set_ylabel('log(L_MIR)')
ax.set_title('%s z=0.3519'%name)
set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99))


In [ ]:
wisedata_path

In [ ]:
save_lc_img_dir='/Volumes/ExtremeSSD/data_source/WISE_data/DESI_CLAGN_plot/mul_line'
for i in DESI_CLAGN_multi:
    name_i='SDSS J'+i
    wise_rebin_lc(name_i,save_lc_img_dir=save_lc_img_dir, datawise_dir=wisedata_path)

In [ ]:
!open /Volumes/ExtremeSSD/data_source/WISE_data/DESI_CLAGN_plot/mul_line

## DESI_CLAGN

In [ ]:
DESI_CLAGN=pd.read_excel('/Users/lyubing/Downloads/DESI_CLAGN.xlsx',sheet_name='Sheet2')
DESI_CLAGN=DESI_CLAGN[DESI_CLAGN['Name'].notnull()]

In [ ]:
DESI_CLAGN

In [ ]:
wisedata_path

wisedata_path='/Volumes/ExtremeSSD/data_source/WISE_data/DESI_CLAGN'
save_lc_img_dir='/Volumes/ExtremeSSD/data_source/WISE_data/DESI_CLAGN_plot'

for index,(ra,dec) in enumerate(zip(DESI_CLAGN['ra'],DESI_CLAGN['dec'])):
    name_i='SDSS J'+DESI_CLAGN.loc[index,'Name']
    print(name_i)
    
    #download_wise(name_i,wisedata_path)
    wise_rebin_lc(name_i,save_lc_img_dir=save_lc_img_dir, datawise_dir=wisedata_path)

wisedata_path='/Volumes/ExtremeSSD/data_source/WISE_data/DESI_CLAGN'
save_lc_img_dir='/Volumes/ExtremeSSD/data_source/WISE_data/DESI_CLAGN_plot'


for index,(ra,dec) in enumerate(zip(DESI_CLAGN['ra'],DESI_CLAGN['dec'])):
    name_i='SDSS J'+DESI_CLAGN.loc[index,'Name']
    print(name_i)    
    wise_rebin_lc(name_i,save_lc_img_dir=save_lc_img_dir, datawise_dir=wisedata_path)

## MWA J221304-414358

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

#22 13 04.40 -41 43 56.9
name='MWAJ221304'
coord_MWAJ221304=SkyCoord('22h13m04.40s','-41d43m56.9s', unit=(u.hourangle, u.deg),frame="icrs",) 
ra_MWAJ221304=coord_MWAJ221304.ra.deg
dec_MWAJ221304=coord_MWAJ221304.dec.deg
z_MWAJ221304=0.080456

In [ ]:
data_test_neo,data_test_mep=get_wise_radec_data(ra_MWAJ221304,dec_MWAJ221304,datawise_dir=wisedata_path_name)

In [ ]:
ra_MWAJ221304,dec_MWAJ221304

In [ ]:
wisedata_path_name

In [ ]:
save_lc_img_dir

In [ ]:
!open /Users/lyubing/blog/brettlv.github.io/pythoncode/WISEDATA

In [ ]:
!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

In [ ]:
#download_wise_withradec(ra_MWAJ221304,dec_MWAJ221304,wisedata_path_name)
#wise_radec_rebin_lc(ra_MWAJ221304,dec_MWAJ221304,datawise_dir=wisedata_path_name,plotwise_dir=save_lc_img_dir)
wise_plotradec_lc(ra_MWAJ221304,dec_MWAJ221304,datawise_dir=wisedata_path_name,plotwise_dir=save_lc_img_dir)

## ZTF20aawpldl

In [ ]:
name='ZTF20aawpldl'
download_wise(name,wisedata_path_name)

In [ ]:
radeg_ZTF20aawpldl=356.869696888455 
decdeg_ZTF20aawpldl=24.462699346859083
download_wise_withradec(radeg_ZTF20aawpldl,decdeg_ZTF20aawpldl,wisedata_path_name)

In [ ]:
wisedata_path_name

In [ ]:
!open /Users/lyubing/blog/brettlv.github.io/pythoncode/WISEDATA

In [ ]:
wise_radec_rebin_lc(radeg_ZTF20aawpldl,decdeg_ZTF20aawpldl,datawise_dir=wisedata_path_name,plotwise_dir=save_lc_img_dir)



In [ ]:
wise_plotradec_lc(radeg_ZTF20aawpldl,decdeg_ZTF20aawpldl,datawise_dir=wisedata_path_name,plotwise_dir=save_lc_img_dir)


In [ ]:
save_lc_img_dir

In [ ]:
!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

## PS16dtm

In [ ]:
name='PS16dtm'
download_wise(name,wisedata_path_name)

In [ ]:
!open /Users/lyubing/blog/brettlv.github.io/pythoncode/WISEDATA

In [ ]:
name='PS16dtm'
D_cm_PS16dtm= cosmo.luminosity_distance(z=0.0804).to(u.cm) 
data_test_neo,data_test_mep=get_wise_data(name,wisedata_path_name)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)

data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_PS16dtm)
datalag_w1_PS16dtm,datalag_w2_PS16dtm=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

### plot_WISE_Lum

In [ ]:
fig = plt.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

name='PS16dtm'
BH_mass_PS16dtm=1e6*u.solMass
Ledd_PS16dtm=1.26*10**38*BH_mass_PS16dtm/u.solMass

ax.errorbar(datalag_w1_PS16dtm['t'],datalag_w1_PS16dtm['flux']+np.log10(9),datalag_w1_PS16dtm['err'],label='w1',ls=':',marker='*')
ax.errorbar(datalag_w2_PS16dtm['t'],datalag_w2_PS16dtm['flux']+np.log10(9),datalag_w2_PS16dtm['err'],label='w2',ls=':',marker='*')

ax.axhline(np.log10(Ledd_PS16dtm),ls='-.',label='Ledd')
ax.set_xlabel('MJD')
ax.set_ylabel('log(9*L_MIR)')
ax.set_title('%s'%name)
set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99))

In [ ]:
datalag_w1_PS16dtm,datalag_w2_PS16dtm

In [ ]:
name='PS16dtm'
wise_plot_lc(name,datawise_dir=wisedata_path_name)

In [ ]:
name='PS16dtm'
wise_rebin_lc(name,datawise_dir=wisedata_path_name)

In [ ]:
save_lc_img_dir

In [ ]:
!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

## ASASSN-20hx

In [ ]:
wisedata_path_name

In [ ]:
name='ASASSN-20hx'
download_wise(name,wisedata_path_name)

In [ ]:
D_cm_ASASSN_20hx= cosmo.luminosity_distance(z=0.0167).to(u.cm) 

name='ASASSN-20hx'
data_test_neo,data_test_mep=get_wise_data(name,wisedata_path_name)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep)
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_ASASSN_20hx)
datalag_w1_ASASSN_20hx,datalag_w2_ASASSN_20hx=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

In [ ]:
name='ASASSN-20hx'
wise_plot_lc(name,datawise_dir=wisedata_path_name)
wise_rebin_lc(name,datawise_dir=wisedata_path_name)

In [ ]:
fig = plt.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)

name='ASASSN-20hx'

BH_mass_ASASSN_20hx=10**7.9*u.solMass
Ledd_ASASSN_20hx=1.26*10**38*BH_mass_ASASSN_20hx/u.solMass

ax.errorbar(datalag_w1_ASASSN_20hx['t'],datalag_w1_ASASSN_20hx['flux']+np.log10(3),datalag_w1_ASASSN_20hx['err'],label='w1',ls=':',marker='*')
ax.errorbar(datalag_w2_ASASSN_20hx['t'],datalag_w2_ASASSN_20hx['flux']+np.log10(3),datalag_w2_ASASSN_20hx['err'],label='w2',ls=':',marker='*')

#ax.axhline(np.log10(Ledd_ASASSN_20hx),ls='-.',label='Ledd')
ax.set_xlabel('MJD')
ax.set_ylabel('log(3*L_MIR)')
ax.set_title('%s'%name)
set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99))


## ULIRG F01004−2237

In [ ]:
The nearby ULIRG F01004-2237 (RA:01h 02m 50.007s and Dec: -22d 21m 57.22s (J2000); z=0.117835) w

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

name='ULIRG_F01004_2237'
coord_ULIRG_F01004_2237=SkyCoord('01h02m50.007s','-22d21m57.22s', unit=(u.hourangle, u.deg),frame="icrs",) 
ra_ULIRG_F01004_2237=coord_ULIRG_F01004_2237.ra.deg
dec_ULIRG_F01004_2237=coord_ULIRG_F01004_2237.dec.deg
z_ULIRG_F01004_2237=0.117835

In [ ]:
ra_ULIRG_F01004_2237,dec_ULIRG_F01004_2237

In [ ]:
download_wise_withradec(ra_ULIRG_F01004_2237,dec_ULIRG_F01004_2237,wisedata_path_name)

In [ ]:
wise_radec_rebin_lc(ra_ULIRG_F01004_2237,dec_ULIRG_F01004_2237,datawise_dir=wisedata_path_name,plotwise_dir=save_lc_img_dir)


In [ ]:
wise_plotradec_lc(ra_ULIRG_F01004_2237,dec_ULIRG_F01004_2237,datawise_dir=wisedata_path_name,plotwise_dir=save_lc_img_dir)


In [ ]:
save_lc_img_dir

In [ ]:
!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

## WISEA J195739.10+503320.4

In [ ]:
name='WISEA J195739.10+503320.4'
wisedata_path_name='/Users/lyubing/blog/brettlv.github.io/pythoncode/WISEDATA'

In [ ]:
print(name,wisedata_path_name)

download_wise(name,wisedata_path)

##  'WISEA J160450.22+520159.0'

In [ ]:
name='WISEA J160450.22+520159.0'
print(name,wisedata_path_name)

In [ ]:
data_test_neo,data_test_mep=get_wise_data(name,wisedata_path_name)

In [ ]:
data_test_neo,

In [ ]:
data_test_mep

## AT 2021loi

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

name='AT2021loi'
coord_AT2021loi=SkyCoord('22h11m21.93s','+39d42m30.31s', unit=(u.hourangle, u.deg),frame="fk5",) 
ra_AT2021loi_1=coord_AT2021loi.ra.deg
dec_AT2021loi_1=coord_AT2021loi.dec.deg
z_AT2021loi=0.083

print(ra_AT2021loi_1,dec_AT2021loi_1)


In [ ]:
datawise_dir='/Users/lyubing/Downloads/TDE_ambiguous nuclear transient(ANT)/Bowen Fluorescence Flare (BFF) class/'

In [ ]:
ra_AT2021loi,dec_AT2021loi=15.16517,39.70832
z_AT2021loi=0.083

download_wise_withradec(ra_AT2021loi,dec_AT2021loi,datawise_dir_BFF)

In [ ]:
from astroquery.ipac.ned import Ned
#from astroquery.ipac.ned import Ned
import astropy.units as u
from astropy import coordinates

result_table_AT2021loi = Ned.query_object('WISEA J010039.62+394230.3')

In [ ]:
result_table_AT2021loi

In [ ]:
name='WISEA J010039.62+394230.3'
download_wise(name,datawise_dir_BFF)

wise_rebin_lc(name,datawise_dir_BFF,datawise_dir_BFF)

In [ ]:
data_ztf_AT2021loi = lightcurve.LCQuery.download_data(circle=[ra_AT2021loi,dec_AT2021loi,2/3600], 
                                                      #bandname="g",
                                                      BAD_CATFLAGS_MASK=32768)

In [ ]:
data_ztf_AT2021loi 

In [ ]:
fig,ax=ztflc_show_in_jupyter(data_ztf_AT2021loi,'AT2021loi',figsize=[20,4])
fig.savefig('%s/ZTF_%s.png'%(datawise_dir_BFF,'AT2021loi'),dpi=400, transparent=False, bbox_inches='tight')


In [ ]:
ztflc_show_in_jupyter(data_ztf_AT2021loi,'AT2021loi')

## AT 2021seu wise/ztf 

In [ ]:
name='AT2021seu'
ra=215.39427 
dec=37.90969 
download_wise_withradec(ra,dec,datawise_dir)

print(ra,dec)
wise_radec_rebin_lc(ra,dec,datawise_dir_BFF,datawise_dir_BFF)#AT 2021seu

In [ ]:
name='AT2021seu'
ra=215.39427 
dec=37.90969 
data_AT2021seu = lightcurve.LCQuery.download_data(circle=[ra,dec,2/3600], 
                                                  #bandname="g",
                                                  BAD_CATFLAGS_MASK=32768)


In [ ]:
datawise_dir

In [ ]:
fig,ax=ztflc_show_in_jupyter(data_AT2021seu,'AT2021seu',figsize=[20,4])
fig.savefig('%s/ZTF_%s.png'%(datawise_dir,'AT2021seu'),dpi=400, transparent=False, bbox_inches='tight')


In [ ]:
2/3600

In [ ]:
save_lc_img_dir

In [ ]:
wise_radec_rebin_lc(ra_AT2021loi,dec_AT2021loi,datawise_dir,save_lc_img_dir)

In [ ]:
!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc

## GALEX 2673143681802830594

In [ ]:
result_table_GALEX2673143681802830594 = Ned.query_object('GALEX 2673143681802830594')

In [ ]:
result_table_GALEX2673143681802830594['RA'].value[0],result_table_GALEX2673143681802830594['DEC'].value[0]

In [ ]:
data_GALEX2673143681802830594 = lightcurve.LCQuery.download_data(circle=[8.81274, -2.44037,2/3600], 
                                                  #bandname="g",
                                                  BAD_CATFLAGS_MASK=32768)

In [ ]:
data_GALEX2673143681802830594

In [ ]:
#ztflc_show_in_jupyter(data_GALEX2673143681802830594,'GALEX2673143681802830594')

fig,ax=ztflc_show_in_jupyter(data_GALEX2673143681802830594,'GALEX2673143681802830594',figsize=[10,4])
fig.savefig('%s/ZTF_%s.png'%(datawise_dir_GALEX,'GALEX2673143681802830594'),dpi=400, transparent=False, bbox_inches='tight')


In [ ]:
download_wise('GALEX 2673143681802830594',datawise_dir_GALEX)

In [ ]:
wise_rebin_lc('GALEX2673143681802830594',datawise_dir_GALEX,datawise_dir_GALEX)

In [ ]:
name

In [ ]:
from astroquery.ipac.ned import Ned
#from astroquery.ipac.ned import Ned
import astropy.units as u
from astropy import coordinates

co = coordinates.SkyCoord(ra=356.86970,dec=24.46270,
                          unit=(u.deg, u.deg), frame='fk5')
result_table = Ned.query_region(co, radius=0.1 * u.deg, equinox='J2000.0')

#result_table = Ned.query_object('name')

In [ ]:
result_table_QSO=result_table[result_table['Type']=='QSO']

In [ ]:
result_table_QSO

In [ ]:
result_table = Ned.query_object('WISEA J010039.62+394230.3')

In [ ]:
result_table

In [ ]:
result_table['RA'],result_table['DEC']

In [ ]:
15.16517,39.70832

## BFF_candidate_WISE

In [ ]:
BFF_candidates=pd.read_csv('/Users/lyubing/Downloads/BFF_candidatesnomatch.py',header=None,delimiter='\s')

BFF_candidatesmatchSDSS=pd.read_csv('/Users/lyubing/Downloads/BFF_candidatesmatchSDSSpy.py',header=None,delimiter='\s')

In [ ]:
Fits_DR10_dir='/Users/lyubing/Downloads/LAMOST/LAMOST_DR10/'
LAMOST_dr10_qsobySDSS_dir=os.path.join(Fits_DR10_dir,'LAMOST_dr10_qsobySDSS/')  

In [ ]:
pd_LAMOST_dr10_match_z_SDSS16=pd.read_csv('/Users/lyubing/Downloads/LAMOST/pd_LAMOST_dr10_match_z_SDSS16.csv')
for index, filename in enumerate(pd_LAMOST_dr10_match_z_SDSS16['#cols: spUrl']):
#filename=pd_LAMOST_dr10_match_z_SDSS16.loc[index,'#cols: spUrl']
    files_fits_path_i=os.path.join(LAMOST_dr10_qsobySDSS_dir,filename)
    if os.path.exists(files_fits_path_i): 
        #print(files_fits_path_i)

        hdul_i=fits.open(files_fits_path_i)

        obsid_i= hdul_i[0].header['OBSID']
        pd_LAMOST_dr10_match_z_SDSS16.loc[index,'OBSID']=obsid_i
    
pd_LAMOST_dr10_match_z_SDSS16_zlt0dot37=pd_LAMOST_dr10_match_z_SDSS16[pd_LAMOST_dr10_match_z_SDSS16['zSDSS']<0.2]
pd_LAMOST_dr10_match_z_SDSS16_zlt0dot37=drop_index(pd_LAMOST_dr10_match_z_SDSS16_zlt0dot37)
    
    

In [ ]:
pd_asera_QSOandCandidates=pd.read_csv('/Users/lyubing/Downloads/pd_asera_dr11and12_QSOandCandidates.csv')
for index, filename in enumerate(pd_asera_QSOandCandidates['#cols: spUrl']):
#filename=pd_LAMOST_dr10_match_z_SDSS16.loc[index,'#cols: spUrl']
    files_fits_path_i= filename[8:]
    if os.path.exists(files_fits_path_i):
        #print(files_fits_path_i)
        hdul_i=fits.open(files_fits_path_i)
        obsid_i= hdul_i[0].header['OBSID']
        pd_asera_QSOandCandidates.loc[index,'OBSID']=obsid_i

In [ ]:
pd_asera_QSOandCandidates_zlt0dot37=pd_asera_QSOandCandidates[pd_asera_QSOandCandidates['z']<0.2]
pd_asera_QSOandCandidates_zlt0dot37=drop_index(pd_asera_QSOandCandidates_zlt0dot37)


In [ ]:
BFF_candidates_dir1='/Users/lyubing/Downloads/TDE_ambiguous nuclear transient(ANT)/Bowen Fluorescence Flare (BFF) class/qsofitmore_test/output/BFF_candidate/Fe_verner09_True'

BFF_candidates_dir2='/Users/lyubing/Downloads/TDE_ambiguous nuclear transient(ANT)/Bowen Fluorescence Flare (BFF) class/qsofitmore_test/output/BFF_candidate/Fe_verner09_True_matchSDSS'

### matchsdss

In [ ]:
BFF_candidatesmatchSDSS[0]

In [ ]:
for obsid_i in BFF_candidatesmatchSDSS[0]:
    #print(i)
    select_i=  pd_LAMOST_dr10_match_z_SDSS16_zlt0dot37[pd_LAMOST_dr10_match_z_SDSS16_zlt0dot37['OBSID']==obsid_i]   
    select_i=drop_index(select_i)
    
    files_fits_path_i=os.path.join(LAMOST_dr10_qsobySDSS_dir,select_i.loc[0,'#cols: spUrl'])

    #print(files_fits_path_i)
    
    hdul_i=fits.open(files_fits_path_i)
    obsid_i= hdul_i[0].header['OBSID']
    radeg=hdul_i[0].header['RA']
    decdeg=hdul_i[0].header['DEC'] 
    print(obsid_i,radeg,decdeg)
    download_wise_withradec(radeg,decdeg,wisedata_path=datawise_dir_BFF)
    wise_plotradec_lc(radeg,decdeg,datawise_dir_BFF,datawise_dir_BFF)     
    #save_name= 'id%dra%.5f_dec%.5f'%(obsid_i,ra,dec)
    #ztf_data_path_= os.path.join(lamostdr10_12_matchSDSS_ztfdata,'%s.csv'%save_name)
    #if os.path.exists(ztf_data_path_):
        #print(ztf_data_path_)

In [ ]:
#RA223.83144_DEC16.95677
wise_plotradec_lc(223.83144,16.95677,datawise_dir_BFF,datawise_dir_BFF) 

### nomatchsdss

In [ ]:
for obsid_i in BFF_candidates[0]:
    #print(obsid_i)
    select_i=  pd_asera_QSOandCandidates_zlt0dot37[pd_asera_QSOandCandidates_zlt0dot37['OBSID']==obsid_i]   
    select_i=drop_index(select_i)
    
    files_fits_path_i=select_i.loc[0,'#cols: spUrl'][8:]
    
    hdul_i=fits.open(files_fits_path_i)
    #obsid_i= hdul_i[0].header['OBSID']
    radeg=hdul_i[0].header['RA']
    decdeg=hdul_i[0].header['DEC']    
    mjd=hdul_i[0].header['MJD']
    download_wise_withradec(radeg,decdeg,wisedata_path=datawise_dir_BFF)
    wise_plotradec_lc(radeg,decdeg,datawise_dir_BFF,datawise_dir_BFF)     

# plot

In [ ]:
data_test_neo,data_test_mep

## wise_plot_lc

In [ ]:
name='WISEA J160450.22+520159.0'
wise_plot_lc(name,datawise_dir=wisedata_path_name)

## wise_rebin_lc

In [ ]:
wise_rebin_lc(name,datawise_dir=wisedata_path)

data_rebin_neo,data_rebin_mep

## download_wise_radecdata

In [ ]:
RG_OIII_match_LoTSS_selectsnrgt5_fadeing=pd.read_csv('%s/RG_OIII_match_LoTSS_selectsnrgt5_fadeing.csv'%Surveyfits_root)
print(len(RG_OIII_match_LoTSS_selectsnrgt5_fadeing))
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_LoTSS_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_LoTSS_selectsnrgt5_fadeing['DEdeg'])):
    download_wise_withradec(radeg,decdeg,wisedata_path_LoTSS) 
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)

In [ ]:
RG_OIII_match_TGSS_selectsnrgt5_fadeing=pd.read_csv('%s/RG_OIII_match_TGSS_selectsnrgt5_fadeing.csv'%Surveyfits_root)
print(len(RG_OIII_match_TGSS_selectsnrgt5_fadeing))
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_TGSS_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_TGSS_selectsnrgt5_fadeing['DEdeg'])):
    download_wise_withradec(radeg,decdeg,wisedata_path_TGSS)
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)

In [ ]:
RG_OIII_match_GGSM_selectsnrgt5_fadeing=pd.read_csv('%s/RG_OIII_match_GGSM_selectsnrgt5_fadeing.csv'%Surveyfits_root)
print(len(RG_OIII_match_GGSM_selectsnrgt5_fadeing))
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_GGSM_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_GGSM_selectsnrgt5_fadeing['DEdeg'])):
    download_wise_withradec(radeg,decdeg,wisedata_path_GGSM)
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)


In [ ]:
RG_OIII_match_LoTSS_selectsnrgt5_fadeing.loc[0].values

## wise_plotradec_lc

In [ ]:
save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
print(save_name)  
#name=save_name
data_test_neo=pd.read_csv(os.path.join(wisedata_path_GGSM,'wise_%s_neo.csv'%save_name))
data_test_mep=pd.read_csv(os.path.join(wisedata_path_GGSM,'wise_%s_mep.csv'%save_name))

In [ ]:
data_test_neo.columns

In [ ]:
data_test_neo[['w1snr','w2snr']]

In [ ]:
data_test_neo['cc_flags'][0]

In [ ]:
data_test_mep.columns

In [ ]:
data_test_mep['cc_flags']#[0]

In [ ]:
radeg,decdeg=RG_OIII_match_GGSM_selectsnrgt5_fadeing.loc[0].values
save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
print(save_name)       
data_test_neo,data_test_mep=get_wise_radec_data(radeg,decdeg,wisedata_path_GGSM)

In [ ]:
data_test_neo,data_test_mep

wisedata_path

In [ ]:
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_LoTSS_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_LoTSS_selectsnrgt5_fadeing['DEdeg'])):
    wise_plotradec_lc(radeg,decdeg,wisedata_path_LoTSS,wiseplot_path_LoTSS)                                        
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)                                       


In [ ]:
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_TGSS_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_TGSS_selectsnrgt5_fadeing['DEdeg'])):
    wise_plotradec_lc(radeg,decdeg,wisedata_path_TGSS,wiseplot_path_TGSS)                                        
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)                                       


In [ ]:
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_GGSM_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_GGSM_selectsnrgt5_fadeing['DEdeg'])):
    wise_plotradec_lc(radeg,decdeg,wisedata_path_GGSM,wiseplot_path_GGSM)                                        
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)                                       


## wise_radec_rebin_lc

In [ ]:
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_LoTSS_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_LoTSS_selectsnrgt5_fadeing['DEdeg'])):
    wise_radec_rebin_lc(radeg,decdeg,wisedata_path_LoTSS,wise_rebin_plot_path_LoTSS)    

In [ ]:
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_TGSS_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_TGSS_selectsnrgt5_fadeing['DEdeg'])):
    wise_radec_rebin_lc(radeg,decdeg,wisedata_path_TGSS,wise_rebin_plot_path_TGSS)                                        
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)                                       


In [ ]:
for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_GGSM_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_GGSM_selectsnrgt5_fadeing['DEdeg'])):
    wise_radec_rebin_lc(radeg,decdeg,wisedata_path_GGSM,wise_rebin_plot_path_GGSM)                                        
    #save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)
    #print(save_name)                                       


In [ ]:
RG_OIII_match_GGSM_selectsnrgt5_fadeing

# RG_OIII_match_ALL_selectsnrgt5_fadeing

In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing_radec=pd.read_csv('%s/RG_OIII_match_ALL_selectsnrgt5_fadeing_radec.csv'%Surveyfits_root)
RG_OIII_match_ALL_selectsnrgt5_fadeing_radec_Table=Table.read('%s/RG_OIII_match_ALL_selectsnrgt5_fadeing_radec.csv'%Surveyfits_root)

In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing=RG_OIII_match_ALL_selectsnrgt5_fadeing_radec

RG_OIII_match_ALL_selectsnrgt5_fadeing

In [ ]:
RG_OIII_match_Radio_selectsnrgt5_fadeing=pd.read_csv('%s/RG_OIII_match_Radio_selectsnrgt5_fadeing_all'%Surveyfits_root)


In [ ]:
RG_OIII_match_Radio_selectsnrgt5_fadeing.columns

RG_OIII_match_Radio_selectsnrgt5_fadeing['logLw1']=W1mag_to_Lum(RG_OIII_match_Radio_selectsnrgt5_fadeing['w1mpro_wise'],RG_OIII_match_Radio_selectsnrgt5_fadeing['Dcm'])
RG_OIII_match_Radio_selectsnrgt5_fadeing['logLw2']=W2mag_to_Lum(RG_OIII_match_Radio_selectsnrgt5_fadeing['w2mpro_wise'],RG_OIII_match_Radio_selectsnrgt5_fadeing['Dcm'])
RG_OIII_match_Radio_selectsnrgt5_fadeing['logLw3']=W3mag_to_Lum(RG_OIII_match_Radio_selectsnrgt5_fadeing['w3mpro_wise'],RG_OIII_match_Radio_selectsnrgt5_fadeing['Dcm'])
RG_OIII_match_Radio_selectsnrgt5_fadeing['logLw4']=W4mag_to_Lum(RG_OIII_match_Radio_selectsnrgt5_fadeing['w4mpro_wise'],RG_OIII_match_Radio_selectsnrgt5_fadeing['Dcm'])

In [ ]:
RG_OIII_match_Radio_selectsnrgt5_fadeing.columns

In [ ]:
wisedata_path_GGSM

In [ ]:
ALLdatawise_dir='/Volumes/ExtremeSSD/Survey_fits//FadingGalaxy_wisedata/ALL/'

RG_OIII_match_ALL_selectsnrgt5_fadeing_Table.loc[0,'RAdeg']

In [ ]:
epsilon_s_w1=0.024
epsilon_s_w2=0.028

for index, (radeg,decdeg) in enumerate(zip(RG_OIII_match_ALL_selectsnrgt5_fadeing['RAdeg'],RG_OIII_match_ALL_selectsnrgt5_fadeing['DEdeg'])):
    #print(i)
    #search_name='SDSS'+name.replace(' ','')
    #save_name='SDSS '+name.replace(' ','')
    
    data_test_neo,data_test_mep=get_wise_radec_data(radeg,decdeg,ALLdatawise_dir)
    #data_test_neo,data_test_mep=get_remove_neo_mep(data_test_neo,data_test_mep)
    
    data_test_neo_copy=data_test_neo[['mjd','w1mpro','w1sigmpro','w2mpro','w2sigmpro']]
    data_test_mep_copy=data_test_mep[['mjd','w1mpro_ep','w1sigmpro_ep','w2mpro_ep','w2sigmpro_ep']]
    data_test_mep_copy.columns=['mjd','w1mpro','w1sigmpro','w2mpro','w2sigmpro']
    
    data_test_neo_plus_mep=pd.concat([data_test_neo_copy,data_test_mep_copy],ignore_index=True)
    data_test_neo_plus_mep=drop_index(data_test_neo_plus_mep)
    
    #data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100)
   
    if len(data_test_neo_plus_mep)>2:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1

        pearson_coeff,p_value=pearsonr(data_test_neo_plus_mep['w1mpro'],data_test_neo_plus_mep['w2mpro'])
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'pearson_coeff_neo']=pearson_coeff

        sigma_m_w1=get_intrinsic_var(data_test_neo_plus_mep,mag_label,magerr_label,epsilon_s)
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        
        mags=data_test_neo_plus_mep[mag_label]
        e_mags=data_test_neo_plus_mep[magerr_label]
        
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'mean_m_w1_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'mean_e_m_w1_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)


        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data_test_neo_plus_mep,mag_label,magerr_label,epsilon_s)
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        
        mags=data_test_neo_plus_mep[mag_label]
        e_mags=data_test_neo_plus_mep[magerr_label]
        
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'mean_m_w2_neo']=get_mag_weighted_mean_mag(mags,e_mags)
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'mean_e_m_w2_neo']=get_mag_weighted_mean_e_mag(mags,e_mags)
        
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'len_neo_mep']=len(data_test_neo_plus_mep)
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'W1-W2']=np.average(data_test_neo_plus_mep['w1mpro']-data_test_neo_plus_mep['w2mpro'])


    else:
        RG_OIII_match_ALL_selectsnrgt5_fadeing.loc[index,'len_neo_mep']=0



In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing.columns

In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing['mean_logLw1']=W1mag_to_Lum(RG_OIII_match_ALL_selectsnrgt5_fadeing['mean_m_w1_neo'],
                                                        RG_OIII_match_Radio_selectsnrgt5_fadeing['Dcm']          
                                                                  )


RG_OIII_match_ALL_selectsnrgt5_fadeing['mean_logLw2']=W1mag_to_Lum(RG_OIII_match_ALL_selectsnrgt5_fadeing['mean_m_w2_neo'],
                                                        RG_OIII_match_Radio_selectsnrgt5_fadeing['Dcm']          
                                                                  )


In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing[RG_OIII_match_ALL_selectsnrgt5_fadeing['W1-W2']>0.5]

In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing[RG_OIII_match_ALL_selectsnrgt5_fadeing['sigma_m_w1_neo']>0.2].sort_values('RAdeg')

wise_radec_rebin_lc(radeg,decdeg,datawise_dir,plotwise_dir)
    
data_test_neo,data_test_mep=get_wise_radec_data(radeg,decdeg,datawise_dir)

data_test_neo,data_test_mep=get_remove_neo_mep(data_test_neo,data_test_mep)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100)    

#data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=10)

save_name='RA%.5f_DEC%.5f'%(radeg,decdeg)

# QSO

In [ ]:
CLAGN_statistic_plot_dir

In [ ]:
#CLAGN_statistic_plot_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/'
#QSO=pd.read_csv('/Users/brettlv/Downloads/CLAGN_statistic_plot/QSO_wise_var.csv')
QSO=pd.read_csv('%s/QSO_wise_var.csv'%CLAGN_statistic_plot_dir)
print(QSO.columns)

In [ ]:
QSO=QSO[QSO['len_neo']>20]
QSO=drop_index(QSO)

In [ ]:
QSO

In [ ]:
len(QSO)

In [ ]:
QSO['log(L5100)']=(QSO['lg(LHalpha)']-np.log10(5.25)-42)/1.157+44
QSO['log(Lbol)']=np.log10(9)+QSO['log(L5100)']
QSO['log(Ledd)']=np.log10(1.263*10**38)+QSO['lg(L_Mbh)']
QSO['log(Lbol/Ledd)']=QSO['log(Lbol)']-QSO['log(Ledd)']
#print(QSO.columns)
#D_cm_QSO=cosmo.luminosity_distance(z=QSO['redshift']).to(u.cm)

In [ ]:
QSO

In [ ]:
QSO['D_cm']=cosmo.luminosity_distance(z=QSO['redshift'].values).to(u.cm).value

QSO['logL_W1']= (-48.6-QSO['mean_m_w1_neo']-2.699)/2.5+np.log10(4*np.pi*QSO['D_cm']**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
QSO['logL_W2']= (-48.6-QSO['mean_m_w2_neo']-3.339)/2.5+np.log10(4*np.pi*QSO['D_cm']**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)

In [ ]:
QSO.columns

# LLAGN

In [ ]:
LLAGN_gu2009=pd.read_csv('%s/LLAGN_gu2009.csv'%CLAGN_statistic_plot_dir)
LLAGN_gu2009=LLAGN_gu2009[LLAGN_gu2009['len_neo']>10]
LLAGN_gu2009=drop_index(LLAGN_gu2009)

In [ ]:
len(LLAGN_gu2009)

In [ ]:
LLAGN_gu2009.columns

In [ ]:
plt.scatter(RG_OIII_match_ALL_selectsnrgt5_fadeing['sigma_m_w1_neo'],RG_OIII_match_ALL_selectsnrgt5_fadeing['W1-W2'])
plt.xlabel(r'$\sigma W1$')
plt.ylabel(r'$W1-W2$')

In [ ]:
RG_OIII_match_ALL_selectsnrgt5_fadeing.columns

In [ ]:
fig = plt.figure(figsize=(9,9))
fig.subplots_adjust(hspace=0.4, wspace = 0.4)



ax = fig.add_subplot(221)
plotdata=RG_OIII_match_ALL_selectsnrgt5_fadeing
#column_x=3
#column_y=4
colum_xlabel='mean_logLw1'
colum_ylabel='sigma_m_w1_neo'
print(colum_xlabel,colum_ylabel)
for i in range(len(plotdata)):
    if isinstance(plotdata[colum_xlabel][i],float) and isinstance(plotdata[colum_xlabel][i],float):
        ax.scatter(plotdata[colum_xlabel][i],
                   plotdata[colum_ylabel][i],
                   color='red',marker='*',label='')
                
                
plotdata=QSO
#column_x=3
#column_y=4
colum_xlabel='logL_W1'
colum_ylabel='sigma_m_w1_neo'
print(colum_xlabel,colum_ylabel)
for i in range(len(plotdata)):
    if isinstance(plotdata[colum_xlabel][i],float) and isinstance(plotdata[colum_xlabel][i],float):
        ax.scatter(plotdata[colum_xlabel][i],
                   plotdata[colum_ylabel][i],
                   color='black',marker='.',label='QSO')
                
plotdata=LLAGN_gu2009
#column_x=3
#column_y=4
colum_xlabel='logL_W1'
colum_ylabel='sigma_m_w1_neo'
print(colum_xlabel,colum_ylabel)
for i in range(len(plotdata)):
    if isinstance(plotdata[colum_xlabel][i],float) and isinstance(plotdata[colum_xlabel][i],float):
        ax.scatter(plotdata[colum_xlabel][i],
                   plotdata[colum_ylabel][i],
                   color='grey',marker='+',label='LLAGN')
                
                
#ax.set_xlabel(r'$L_{178MHz}$')
#ax.set_ylabel(r'$L_{OIII}$')
#linx_plot=np.linspace(41,46,10)
#ax.plot(linx_plot,linx_plot,ls=':')
ax.set_xlabel(colum_xlabel)
ax.set_ylabel(colum_ylabel)
set_ax_tick(ax)
handles, labels = ax.get_legend_handles_labels()
# remove the errorbars
hdl=handles
#hdl = [h[0] for h in handles]
# use them in the legend
#
labels_dict=dict(zip(labels, hdl)) #key,values
#by_label=OrderedDict(sorted(labels_dict.items(),key=lambda t:t[0]))
by_label = OrderedDict(zip(labels, hdl)) 
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 0.99),
          loc=2, numpoints=1,ncol=1,fontsize=11.)



###########################################
ax = fig.add_subplot(222)
plotdata=RG_OIII_match_ALL_selectsnrgt5_fadeing
#column_x=3
#column_y=4
colum_xlabel='mean_logLw1'
colum_ylabel='W1-W2'
print(colum_xlabel,colum_ylabel)
for i in range(len(plotdata)):
    if isinstance(plotdata[colum_xlabel][i],float) and isinstance(plotdata[colum_xlabel][i],float):
        ax.scatter(plotdata[colum_xlabel][i],
                   plotdata[colum_ylabel][i],
                   color='red',marker='*',label='')
                
                
plotdata=QSO
#column_x=3
#column_y=4
colum_xlabel='logL_W1'
colum_ylabel='W1-W2'
print(colum_xlabel,colum_ylabel)
for i in range(len(plotdata)):
    if isinstance(plotdata[colum_xlabel][i],float) and isinstance(plotdata[colum_xlabel][i],float):
        ax.scatter(plotdata[colum_xlabel][i],
                   plotdata[colum_ylabel][i],
                   color='black',marker='.',label='QSO')
                
plotdata=LLAGN_gu2009
#column_x=3
#column_y=4
colum_xlabel='logL_W1'
colum_ylabel='W1-W2'
print(colum_xlabel,colum_ylabel)
for i in range(len(plotdata)):
    if isinstance(plotdata[colum_xlabel][i],float) and isinstance(plotdata[colum_xlabel][i],float):
        ax.scatter(plotdata[colum_xlabel][i],
                   plotdata[colum_ylabel][i],
                   color='grey',marker='+',label='LLAGN')
                
                
#ax.set_xlabel(r'$L_{178MHz}$')
#ax.set_ylabel(r'$L_{OIII}$')
#linx_plot=np.linspace(41,46,10)
#ax.plot(linx_plot,linx_plot,ls=':')
ax.set_xlabel(colum_xlabel)
ax.set_ylabel(colum_ylabel)
set_ax_tick(ax)
handles, labels = ax.get_legend_handles_labels()
# remove the errorbars
hdl=handles
#hdl = [h[0] for h in handles]
# use them in the legend
#
labels_dict=dict(zip(labels, hdl)) #key,values
#by_label=OrderedDict(sorted(labels_dict.items(),key=lambda t:t[0]))
by_label = OrderedDict(zip(labels, hdl)) 
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 0.99),
          loc=2, numpoints=1,ncol=1,fontsize=11.)
###########################################





###########################################



In [ ]:
colum_ylabel='sigma_m_w1_neo'
np.mean(LLAGN_gu2009[colum_ylabel]),np.mean(QSO[colum_ylabel]),np.mean(RG_OIII_match_ALL_selectsnrgt5_fadeing['sigma_m_w1_neo'])

# rebin data

data_rebin_neo,data_rebin_mep

# def_wise_mag_to_flux

In [ ]:
def wise_mag_to_flux(data,maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2,D_cm):
    data['logL_W1']= (-48.6-data[maglabel_w1]-2.699)/2.5+np.log10(4*np.pi*D_cm.value**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
    data['logL_W2']= (-48.6-data[maglabel_w2]-3.339)/2.5+np.log10(4*np.pi*D_cm.value**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)
    data['e_logL_W1'] =   data[e_maglabel_w1]/2.5
    data['e_logL_W2'] =   data[e_maglabel_w2]/2.5
    return data

def get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm):
    maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2=('W1','W2','e_W1','e_W2')  
    data_rebinflux_neo=wise_mag_to_flux(data_rebin_neo,maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2,D_cm)
    data_rebinflux_mep=wise_mag_to_flux(data_rebin_mep,maglabel_w1,maglabel_w2,e_maglabel_w1,e_maglabel_w2,D_cm)
                                                                                                              
    return data_rebinflux_neo,data_rebinflux_mep

## def get_wise_lc

In [ ]:
def get_wise_lc(data_rebinflux_neo,data_rebinflux_mep):
    lc_w1= pd.concat([data_rebinflux_neo[['t_W1','logL_W1','e_logL_W1']],data_rebinflux_mep[['t_W1','logL_W1','e_logL_W1']]])
    lc_w2= pd.concat([data_rebinflux_neo[['t_W2','logL_W2','e_logL_W2']],data_rebinflux_mep[['t_W2','logL_W2','e_logL_W2']]])
    
    lc_w1=lc_w1.sort_values('t_W1')
    lc_w2=lc_w2.sort_values('t_W2')
    lc_w1=drop_index(lc_w1)
    lc_w2=drop_index(lc_w2)
    
    #return lc_w1,lc_w2
    datalag_w1=get_lag_data(lc_w1,'t_W1','logL_W1','e_logL_W1')
    datalag_w2=get_lag_data(lc_w2,'t_W2','logL_W2','e_logL_W2')
    return datalag_w1,datalag_w2

## def get_lag_data

In [ ]:
def get_lag_data(data,timelabel,fluxlabel_w1,e_fluxlabel_w1):
    
    datalag_w1=pd.DataFrame({'t':data[timelabel],'flux':data[fluxlabel_w1],\
                                  'err':data[e_fluxlabel_w1]},\
                                  index=None,columns=['t','flux','err'])
    return datalag_w1

## def write_lagfile

In [ ]:
def write_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax):
    if not os.path.exists(os.path.join(rootdir,sourcename)):
        os.mkdir(os.path.join(rootdir,sourcename))
        
    data1= dataname1[dataname1['t']>rangemin]
    data1= data1[data1['t']<rangemax]   
    path_data1=os.path.join(rootdir,sourcename,labelname1+'.csv')
    data1.to_csv(path_data1,index=False,header=False,sep=',')
    
    data2= dataname2[dataname2['t']>rangemin]
    data2= data2[data2['t']<rangemax]    
    path_data2=os.path.join(rootdir,sourcename,labelname2+'.csv')
    data2.to_csv(path_data2,index=False,header=False,sep=',')  
    return path_data1,path_data2    

## def get_lagfile_path

In [ ]:
def get_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax):
    
    #data1= dataname1[dataname1['t']>rangemin]
    #data1= data1[data1['t']<rangemax]   
    path_data1=os.path.join(rootdir,sourcename,labelname1+'.csv')
    #data1.to_csv(path_data1,index=False,header=False,sep=',')
    
    #data2= dataname2[dataname2['t']>rangemin]
    #data2= data2[data2['t']<rangemax]    
    path_data2=os.path.join(rootdir,sourcename,labelname2+'.csv')
    #data2.to_csv(path_data2,index=False,header=False,sep=',')  
    return path_data1,path_data2 

In [ ]:
name

In [ ]:
#name_test='NGC 1566'
name_test='WISEA J195739.10+503320.4'
z_WISEAJ195739=0.1
D_cm_WISEAJ195739=cosmo.luminosity_distance(z=z_WISEAJ195739).to(u.cm).value

print(name_test,D_cm_WISEAJ195739)
data_test_neo,data_test_mep=get_wise_data(name,datawise_dir=wisedata_path)
data_test_neo,data_test_mep=get_remove_neo_mep(data_test_neo,data_test_mep)
data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100)



In [ ]:
data_rebin_neo,data_rebin_mep

In [ ]:
data_rebinflux_neo,data_rebinflux_mep=get_wiseflux_rebin(data_rebin_neo,data_rebin_mep,D_cm_WISEAJ195739)
datalag_w1_WISEAJ195739,datalag_w2_WISEAJ195739=get_wise_lc(data_rebinflux_neo,data_rebinflux_mep)

In [ ]:
rootdir_WWZ='/Users/brettlv/Downloads/WWZ/'
if not os.path.exists(rootdir_WWZ):
    os.mkdir(rootdir_WWZ)
    print(rootdir_WWZ)

In [ ]:
min(datalag_w1_WISEAJ195739['t']),max(datalag_w1_WISEAJ195739['t'])

In [ ]:
labelname1="W1"
labelname2="W2"
#dataname1=lag_NGC1566_V
dataname1=datalag_w1_WISEAJ195739
dataname2=datalag_w2_WISEAJ195739
sourcename=name

rangemin,rangemax=55000,59900
lagrangel=0
lagrangeh=180

In [ ]:
rootdir=rootdir_WWZ
os.chdir(rootdir)
source_path=os.path.join(rootdir,sourcename)
path_data1,path_data2=write_lagfile(rootdir,sourcename,labelname1,labelname2,dataname1,dataname2,rangemin,rangemax)


# Lomb-scargle

In [ ]:
datalag_w1_WISEAJ195739.columns

In [ ]:
from astropy.stats import LombScargle

In [ ]:
fig = plt.figure(figsize=(12, 6))
gs = plt.GridSpec(2, 2)

f = lambda x: 0.8 * np.sin(x)
rng = np.random.RandomState(15566)

#data=datalag_w1_WISEAJ195739
t = datalag_w1_WISEAJ195739['t']
y= datalag_w1_WISEAJ195739['flux']
dy =datalag_w1_WISEAJ195739['err']

ax = fig.add_subplot(gs[:, 0])
ax.errorbar(t, y, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.plot(tfit, f(tfit), '-k', color='gray')
ax.set(xlabel='MJD',
       ylabel='signal',
       title='W1')
ax.set_xlim(55000, 59900)
set_ax_tick(ax)
plot_secax(ax)



ls = LombScargle(t, y,dy)
freq, power = ls.autopower(normalization='standard',
                           minimum_frequency=0,
                           maximum_frequency=0.1,
                           samples_per_peak=20)
freq, p = ls.autopower(normalization='psd',
                       minimum_frequency=0,
                       maximum_frequency=0.1,
                       samples_per_peak=20)
#p = np.exp(p)

ax = [fig.add_subplot(gs[0, 1]),
      fig.add_subplot(gs[1, 1])]
ax[0].set_yscale('log')
ax[1].set_yscale('log')
ax[0].set_xscale('log')
ax[1].set_xscale('log')

ax[0].plot(freq, power, '-k')
ax[0].set(xlim=(0, 0.1),
          ylabel='$P_{LS}$',
          title='Lomb-Scargle Periodogram')
ax[0].xaxis.set_major_formatter(plt.NullFormatter())

ax[1].plot(freq, p, '-k')
ax[1].set(xlim=(0, 0.1),
          xlabel='frequency',
          ylabel='$p(f|D)$',
          title='Bayesian Periodogram')

fig.savefig('/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/WISEAJ195739_ls.png',dpi=300)

In [ ]:
fig = plt.figure(figsize=(12, 6))
gs = plt.GridSpec(2, 2)

f = lambda x: 0.8 * np.sin(x)
rng = np.random.RandomState(15566)

#data=datalag_w1_WISEAJ195739
t = datalag_w2_WISEAJ195739['t']
y= datalag_w2_WISEAJ195739['flux']
dy =datalag_w2_WISEAJ195739['err']

ax = fig.add_subplot(gs[:, 0])
ax.errorbar(t, y, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.plot(tfit, f(tfit), '-k', color='gray')
ax.set(xlabel='MJD',
       ylabel='signal',
       title='W2')
ax.set_xlim(55000, 59900)
set_ax_tick(ax)
plot_secax(ax)



ls = LombScargle(t, y,dy)
freq, power = ls.autopower(normalization='standard',
                           minimum_frequency=0,
                           maximum_frequency=0.1,
                           samples_per_peak=20)
freq, p = ls.autopower(normalization='psd',
                       minimum_frequency=0,
                       maximum_frequency=0.1,
                       samples_per_peak=20)
#p = np.exp(p)

ax = [fig.add_subplot(gs[0, 1]),
      fig.add_subplot(gs[1, 1])]
ax[0].set_yscale('log')
ax[1].set_yscale('log')
ax[0].set_xscale('log')
ax[1].set_xscale('log')

ax[0].plot(freq, power, '-k')
ax[0].set(xlim=(0, 0.1),
          ylabel='$P_{LS}$',
          title='Lomb-Scargle Periodogram')
ax[0].xaxis.set_major_formatter(plt.NullFormatter())

ax[1].plot(freq, p, '-k')
ax[1].set(xlim=(0, 0.1),
          xlabel='frequency',
          ylabel='$p(f|D)$',
          title='Bayesian Periodogram')

fig.savefig('/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/WISEAJ195739_ls_W2.png',dpi=300)

## LS and BIC

In [ ]:
 np.linspace(1, 3, 100)

In [ ]:
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
import numpy as np
from matplotlib import pyplot as plt
from astroML.time_series import lomb_scargle, lomb_scargle_BIC, lomb_scargle_bootstrap

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=True)

#------------------------------------------------------------
# Generate Data
np.random.seed(0)


'''
N = 30
P = 0.3
t = np.random.randint(100, size=N) + 0.3 + 0.4 * np.random.random(N)
y = 10 + np.sin(2 * np.pi * t / P)
dy = 0.5 + 0.5 * np.random.random(N)
y_obs = np.random.normal(y, dy)

'''
t = datalag_w1_WISEAJ195739['t']
y_obs= datalag_w1_WISEAJ195739['flux']
dy =datalag_w1_WISEAJ195739['err']


#------------------------------------------------------------
# Compute periodogram
period = 10 ** np.linspace(2, 3.5, 1000)
omega = 2 * np.pi / period
PS = lomb_scargle(t, y_obs, dy, omega, generalized=True)

#------------------------------------------------------------
# Get significance via bootstrap
D = lomb_scargle_bootstrap(t, y_obs, dy, omega, generalized=True,
                           N_bootstraps=1000, random_state=0)

sig1, sig3, sig2 = np.percentile(D, [68, 99.74, 95.44])

#------------------------------------------------------------



In [ ]:
t

In [ ]:
# Plot the results
fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(left=0.1, right=0.9, hspace=0.25)

# First panel: the data
ax = fig.add_subplot(211)
ax.errorbar(t, y_obs, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.set_xlabel('MJD')
#ax.set_ylabel('flux')

ax.set(xlabel='MJD',
       ylabel='signal',
       title='W1')

ax.set_xlim(55000, 59900)
set_ax_tick(ax)
plot_secax(ax)
# Second panel: the periodogram & significance levels

ax1 = fig.add_subplot(212, xscale='log')
ax1.plot(period, PS, '-', c='black', lw=1, zorder=1)
#ax1.plot([period[0], period[-1]], [sig1, sig1], ':', c='black')
#ax1.plot([period[0], period[-1]], [sig3, sig3], ':', c='blue')
#ax1.plot([period[0], period[-1]], [sig2, sig2], ':', c='red')
ax1.axhline(sig1, linestyle='dotted', c='black',label=r'1$\sigma$')
ax1.axhline(sig2, linestyle='-', c='red',label=r'2$\sigma$')
ax1.axhline(sig3, linestyle='-.', c='blue',label=r'3$\sigma$')

'''ax1.annotate("", (0.3, 0.65), (0.3, 0.85), ha='center',
             arrowprops=dict(arrowstyle='->'))
'''
#ax1.set_xlim(period[0], period[-1])
ax1.set_ylim(-0.05, 0.95)

ax1.set_xlabel(r'period (days)')
ax1.set_ylabel('power')

# Twin axis: label BIC on the right side
ax2 = ax1.twinx()
ax2.set_ylim(tuple(lomb_scargle_BIC(ax1.get_ylim(), y_obs, dy)))
ax2.set_ylabel(r'$\Delta BIC$')

ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_minor_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_major_locator(plt.LogLocator(10))
ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3g'))

ax1.legend()
#plt.show()
fig.savefig('/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/WISEAJ195739_ls_sig.png',dpi=300)

In [ ]:
t = datalag_w2_WISEAJ195739['t']
y_obs= datalag_w2_WISEAJ195739['flux']
dy =datalag_w2_WISEAJ195739['err']


#------------------------------------------------------------
# Compute periodogram
period = 10 ** np.linspace(2, 3.5, 1000)
omega = 2 * np.pi / period
PS = lomb_scargle(t, y_obs, dy, omega, generalized=True)

#------------------------------------------------------------
# Get significance via bootstrap
D = lomb_scargle_bootstrap(t, y_obs, dy, omega, generalized=True,
                           N_bootstraps=1000, random_state=0)

sig1, sig3, sig2 = np.percentile(D, [68, 99.74, 95.44])

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(left=0.1, right=0.9, hspace=0.25)

# First panel: the data
ax = fig.add_subplot(211)
ax.errorbar(t, y_obs, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.set_xlabel('MJD')
#ax.set_ylabel('flux')

ax.set(xlabel='MJD',
       ylabel='signal',
       title='W2')

ax.set_xlim(55000, 59900)
set_ax_tick(ax)
plot_secax(ax)
# Second panel: the periodogram & significance levels

ax1 = fig.add_subplot(212, xscale='log')
ax1.plot(period, PS, '-', c='black', lw=1, zorder=1)
#ax1.plot([period[0], period[-1]], [sig1, sig1], ':', c='black')
#ax1.plot([period[0], period[-1]], [sig3, sig3], ':', c='blue')
#ax1.plot([period[0], period[-1]], [sig2, sig2], ':', c='red')
ax1.axhline(sig1, linestyle='dotted', c='black',label=r'1$\sigma$')
ax1.axhline(sig2, linestyle='-', c='red',label=r'2$\sigma$')
ax1.axhline(sig3, linestyle='-.', c='blue',label=r'3$\sigma$')

'''ax1.annotate("", (0.3, 0.65), (0.3, 0.85), ha='center',
             arrowprops=dict(arrowstyle='->'))
'''
#ax1.set_xlim(period[0], period[-1])
ax1.set_ylim(-0.05, 0.95)

ax1.set_xlabel(r'period (days)')
ax1.set_ylabel('power')

# Twin axis: label BIC on the right side
ax2 = ax1.twinx()
ax2.set_ylim(tuple(lomb_scargle_BIC(ax1.get_ylim(), y_obs, dy)))
ax2.set_ylabel(r'$\Delta BIC$')

ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_minor_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_major_locator(plt.LogLocator(10))
ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3g'))

ax1.legend()
#plt.show()
fig.savefig('/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/WISEAJ195739_ls_sig_W2.png',dpi=300)

# ZTF

## def ztflc_show_in_jupyter

In [ ]:
def ztflc_show_in_jupyter(data,name,figsize=[7,4],showtoday=False, show_upperlimits=False, formattime=False,
             marker=".", mec="0.7", ms=6, ecolor="0.7", ls="", zorder=4, **kwargs):
        """ kwargs goes to matplotlib's errorbar() """
        FILTER_COLORS = ["C2","C3","C1"]
        FILTER_CODE   = ["zg","zr","zi"]
        marker_CODE   = [".","^","*"]
        
        save_name=name.strip().replace(' ','_')
        
        from astropy import time
        import matplotlib.pyplot as mpl
        # ----------- #
        # Global      #
        # ----------- #
        lc_dataframe = data
        prop = dict(marker=marker, mec=mec, ms=ms, ecolor=ecolor, ls=ls, zorder=zorder)
        # ----------- #
        #
        # ----------- #
        fig = mpl.figure(figsize=figsize)
        ax = fig.add_axes([0.1,0.15,0.8,0.75])
        for filter_ in np.unique(lc_dataframe["filtercode"]):
            d = lc_dataframe[lc_dataframe["filtercode"]==filter_]
            if filter_ in FILTER_CODE:
                prop["color"] = np.asarray(FILTER_COLORS)[np.where(np.asarray(FILTER_CODE)==filter_)][0]
            else:
                prop["color"] = "0.7"
            if formattime:
                dates = time.Time(np.asarray(d["mjd"].values, dtype="float"), format="mjd").datetime
            else:
                dates = d["mjd"]
            ax.errorbar(dates, d["mag"], 
                                yerr=d["magerr"],label=filter_,
                                **{**prop,**kwargs})
        ax.invert_yaxis()
        
        if show_upperlimits:
            for filter_ in np.unique(lc_dataframe["filtercode"]):
                d = lc_dataframe[lc_dataframe["filtercode"]==filter_]
                if filter_ in FILTER_CODE:
                    color = np.asarray(FILTER_COLORS)[np.where(np.asarray(FILTER_CODE)==filter_)][0]
                else:
                    color = "0.7"
                if formattime:
                    dates = time.Time(np.asarray(d["mjd"].values, dtype="float"), format="mjd").datetime
                else:
                    dates = d["mjd"]
                    
                ax.errorbar(dates, d["limitmag"], 
                            yerr=0.1, lolims=True,marker="None", ls="None", 
                            color=color, alpha=0.1,label=filter_
                            )
        if showtoday:
            today_color = "0.7"
            import datetime
            today = time.Time(datetime.date.today().isoformat(),format="iso").mjd
            ax.axvline(today, ls="--", color=today_color, zorder=1, lw=1)
            ax.text(today, ax.get_ylim()[0]-0.05, "Today", va="bottom", ha="right", rotation=90, color=today_color)
        
        
        maxmjd= max(lc_dataframe['mjd'])
        minmjd= min(lc_dataframe['mjd'])
        delta_mjd=maxmjd-minmjd
        
        if delta_mjd>750: 
            
            ma_intervals= 365
            min_intervals= 90
                       
        if delta_mjd<750:     
            ma_intervals= delta_mjd//3
            mi_intervals= delta_mjd//6
            
        ax.set_xlabel("MJD")
        plot_secax(ax,rotation=30,) 
        # - Labels           
        ax.set_ylabel("mag")
        #ax.set_title('%s_%s'%(name,band))
        ax.set_title('%s'%(name))
        set_ax_legend(ax,bbox_to_anchor=(1.0, 0.99))
        
        return fig,ax

def ztflc_show_in_jupyter(data,name,band, showtoday=False, show_upperlimits=False, formattime=False,
             marker="o", mec="0.7", ms=7, ecolor="0.7", ls="None", zorder=4, **kwargs):
        """ kwargs goes to matplotlib's errorbar() """
        FILTER_COLORS = ["C2","C3","C1"]
        FILTER_CODE   = ["zg","zr","zi"]
        save_name=name.strip().replace(' ','_')
        

        from astropy import time
        import matplotlib.pyplot as mpl

        # ----------- #
        # Global      #
        # ----------- #
        lc_dataframe = data
        prop = dict(marker=marker, mec=mec, ms=ms, ecolor=ecolor, ls=ls, zorder=zorder)

        # ----------- #
        #
        # ----------- #
        fig = mpl.figure(figsize=[7,4])
        ax = fig.add_axes([0.1,0.15,0.8,0.75])
        for filter_ in np.unique(lc_dataframe["filtercode"]):
            d = lc_dataframe[lc_dataframe["filtercode"]==filter_]
            if filter_ in FILTER_CODE:
                prop["color"] = np.asarray(FILTER_COLORS)[np.where(np.asarray(FILTER_CODE)==filter_)][0]
            else:
                prop["color"] = "0.7"

            if formattime:
                dates = time.Time(np.asarray(d["mjd"].values, dtype="float"), format="mjd").datetime
            else:
                dates = d["mjd"]
            ax.errorbar(dates, d["mag"], 
                                yerr=d["magerr"],
                                **{**prop,**kwargs})

        ax.invert_yaxis()
        
        
        if show_upperlimits:
            for filter_ in np.unique(lc_dataframe["filtercode"]):
                d = lc_dataframe[lc_dataframe["filtercode"]==filter_]
                if filter_ in FILTER_CODE:
                    color = np.asarray(FILTER_COLORS)[np.where(np.asarray(FILTER_CODE)==filter_)][0]
                else:
                    color = "0.7"

                if formattime:
                    dates = time.Time(np.asarray(d["mjd"].values, dtype="float"), format="mjd").datetime
                else:
                    dates = d["mjd"]
                    
                ax.errorbar(dates, d["limitmag"], 
                            yerr=0.1, lolims=True,marker="None", ls="None", 
                            color=color, alpha=0.1,
                            )

        if showtoday:
            today_color = "0.7"
            import datetime
            today = time.Time(datetime.date.today().isoformat(),format="iso").mjd
            ax.axvline(today, ls="--", color=today_color, zorder=1, lw=1)
            ax.text(today, ax.get_ylim()[0]-0.05, "Today", va="bottom", ha="right", rotation=90, color=today_color)

        if formattime:
            from matplotlib import dates as mdates            
            locator = mdates.AutoDateLocator()
            formatter = mdates.ConciseDateFormatter(locator)
            ax.xaxis.set_major_locator(locator)
            ax.xaxis.set_major_formatter(formatter)
            ax.set_xlabel("Date")
        else:
            ax.set_xlabel("MJD")
        # - Labels

        ax.set_ylabel("mag")
        ax.set_title('%s_%s'%(name,band))
        plot_secax(ax,mi_interval=30,ma_interval=180,rotation=30)  
        return fig
        

## ztflc_Lum_show_in_jupyter

In [ ]:
def ztflc_Lum_show_in_jupyter(data,name,band,D_cm, showtoday=False, show_upperlimits=False, formattime=True,
             marker="o", mec="0.7", ms=7, ecolor="0.7", ls="None", zorder=4, **kwargs):
        """ kwargs goes to matplotlib's errorbar() """
        FILTER_COLORS = ["C2","C3","C1"]
        FILTER_CODE   = ["zg","zr","zi"]
        save_name=name.strip().replace(' ','_')
        
        if band=='g':
            wavelength=(410+550)/2*10
    
        if band=='r':
            wavelength=(700+550)/2*10 
            
        if band=='i':
            wavelength=(700+900)/2*10 
            
            
        from astropy import time
        import matplotlib.pyplot as mpl

        # ----------- #
        # Global      #
        # ----------- #
        lc_dataframe = data
        prop = dict(marker=marker, mec=mec, ms=ms, ecolor=ecolor, ls=ls, zorder=zorder)

        # ----------- #
        #
        # ----------- #
        fig = mpl.figure(figsize=[7,4])
        ax = fig.add_axes([0.1,0.15,0.8,0.75])
        for filter_ in np.unique(lc_dataframe["filtercode"]):
            d = lc_dataframe[lc_dataframe["filtercode"]==filter_]
            if filter_ in FILTER_CODE:
                prop["color"] = np.asarray(FILTER_COLORS)[np.where(np.asarray(FILTER_CODE)==filter_)][0]
            else:
                prop["color"] = "0.7"

            if formattime:
                dates = time.Time(np.asarray(d["mjd"].values, dtype="float"), format="mjd").datetime
            else:
                dates = d["mjd"]
            ax.errorbar(dates, d["flux"]*4*np.pi*D_cm**2*1e-23*wavelength, 
                                yerr=d["flux_err"]*4*np.pi*D_cm**2*1e-23*wavelength,
                                **{**prop,**kwargs})

    

        if showtoday:
            today_color = "0.7"
            import datetime
            today = time.Time(datetime.date.today().isoformat(),format="iso").mjd
            ax.axvline(today, ls="--", color=today_color, zorder=1, lw=1)
            ax.text(today, ax.get_ylim()[0]-0.05, "Today", va="bottom", ha="right", rotation=90, color=today_color)

        if formattime:
            from matplotlib import dates as mdates            
            locator = mdates.AutoDateLocator()
            formatter = mdates.ConciseDateFormatter(locator)
            ax.xaxis.set_major_locator(locator)
            ax.xaxis.set_major_formatter(formatter)
            ax.set_xlabel("Date")
        else:
            ax.set_xlabel("MJD")
        # - Labels

        ax.set_ylabel("$\lambda*L_{%s}$"%band)
        ax.set_title('%s_%s'%(name,band))
        plot_secax(ax)       
        

In [ ]:
table_IRSA=Irsa.list_catalogs()   

In [ ]:
table_IRSA

In [ ]:
name

## ptf

In [ ]:
#Irsa.ROW_LIMIT = 1000 # value of new row limit here.
#Irsa.TIMEOUT = 120
#name='Mrk 6'
search_name=name.strip().replace(' ','')
save_name=search_name
ztf_datadir='/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/'
catalog='ptf_lightcurves'
ztf_datapath='%s/%s_%s.csv'%(ztf_datadir,catalog,save_name)
# objects_dr7

if not os.path.exists(ztf_datapath):
    print(search_name)
    Irsa.ROW_LIMIT = 10000
    Irsa.TIMEOUT = 120
    table_ztf_objects_dr7 =Irsa.query_region(search_name,catalog,spatial='Cone',radius=1*u.arcsec,)
    table_ztf_objects_dr7.write(ztf_datapath, format='csv')
    time.sleep(15+np.random.randint(5,10))    

In [ ]:

search_name=name.strip().replace(' ','')
save_name=search_name
ztf_datadir='/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/'
catalog='ptf_lightcurves'
ztf_datapath='%s/%s_%s.csv'%(ztf_datadir,catalog,save_name)
print(save_name)
ztf_lc=pd.read_csv(ztf_datapath)

In [ ]:
ztf_lc.columns

In [ ]:
min(ztf_lc['obsmjd']),max(ztf_lc['obsmjd']),max(ztf_lc['obsmjd'])-min(ztf_lc['obsmjd'])

In [ ]:
fig = plt.figure(figsize=(6, 6))
#gs = plt.GridSpec(2, 2)



#data=datalag_w1_WISEAJ195739
t = ztf_lc['obsmjd']
y= ztf_lc['mag_autocorr']
dy =ztf_lc['magerr_auto']

ax = fig.add_subplot(111)
ax.errorbar(t, y, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.plot(tfit, f(tfit), '-k', color='gray')
ax.set(xlabel='MJD',
       ylabel='signal',
       title='ptf')
ax.set_xlim(56775, 56850)
set_ax_tick(ax)
plot_secax(ax,30,10)

## ztf

In [ ]:
ztf_datadir

In [ ]:
#Irsa.ROW_LIMIT = 1000 # value of new row limit here.
#Irsa.TIMEOUT = 120

#name='Mrk 6'
name='PS16dtm'
search_name=name.strip().replace(' ','')
save_name=search_name
ztf_datadir='/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/'
catalog='ztf_objects_dr11'
ztf_datapath='%s/%s_%s.csv'%(ztf_datadir,catalog,save_name)
# objects_dr7

if not os.path.exists(ztf_datapath):
    print(search_name)
    Irsa.ROW_LIMIT = 10000
    Irsa.TIMEOUT = 120
    table_ztf_objects_dr7 =Irsa.query_region(search_name,catalog=catalog,spatial='Cone',radius=1*u.arcsec,)
    table_ztf_objects_dr7.write(ztf_datapath, format='csv')
    time.sleep(15+np.random.randint(5,10))    

In [ ]:
search_name=name.strip().replace(' ','')
save_name=search_name
ztf_datadir='/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/'
catalog='ztf_objects_dr11'
ztf_datapath='%s/%s_%s.csv'%(ztf_datadir,catalog,save_name)
print(save_name)
ztf_lc=pd.read_csv(ztf_datapath)

In [ ]:
ztf_lc.columns

In [ ]:
ztf_lc

## ztfquery 

In [ ]:
$ZTFDATA

In [ ]:
ZTFDATA='/Volumes/ExtremeSSD/ZTFdata'
if not os.path.exists(ZTFDATA):
    os.mkdir(ZTFDATA)

In [ ]:
import ztfquery

In [ ]:
from ztfquery import query
zquery = query.ZTFQuery()

## def ztfdata_downloadradec

In [76]:
def ztfdata_downloadradec(ra,dec,pathztf):
    data = lightcurve.LCQuery.download_data(circle=[ra,dec,2.0/3600],BAD_CATFLAGS_MASK=32768)
    
    save_name= 'ra%.5f_dec%.5f'%(ra,dec)
    
    data.to_csv('%s/%s.csv'%(pathztf,save_name),index=False) 
    
    #return data

## def ztfdata_download_name

In [87]:
def ztfdata_download_name(name,ra,dec,pathztf):
    data = lightcurve.LCQuery.download_data(circle=[ra,dec,2.0/3600],BAD_CATFLAGS_MASK=32768)
    
    #save_name= 'ra%.5f_dec%.5f'%(ra,dec)
    save_name= name.strip().replace(' ','')
    data.to_csv('%s/%s.csv'%(pathztf,save_name),index=False) 
    
    #return data

ztfquery.io.load_id({service_name})

In [ ]:
fritz.download_lightcurve?

#LOCALSOURCE=ZTFDATA

export $ZTFDATA=/Volumes/ExtremeSSD/ZTFdata

%env ZTFDATA=/Volumes/ExtremeSSD/ZTFdata
%env LOCALSOURCE=/Volumes/ExtremeSSD/ZTFdata

In [ ]:
%set_env ZTFDATA=/Volumes/ExtremeSSD/ZTFdata
%set_env LOCALSOURCE=/Volumes/ExtremeSSD/ZTFdata/Data

In [ ]:
pwd

In [ ]:
os.chdir(ZTFDATA)

In [ ]:
#%matplotlib notebook
from ztfquery import fritz

In [ ]:
pwd

lc = fritz.download_lightcurve("ZTF20acrzwvx")

In [ ]:
os.getenv('ZTFDATA'),os.getenv('LOCALSOURCE')

In [ ]:
{service_name}

In [ ]:
ztfquery.io._load_id_('irsa') #{"username":"lyubing@hust.edu.cn",'password':'qwe123irsa'}

cat /Users/brettlv/anaconda3/lib/python3.8/site-packages/ztfquery/io.py

In [ ]:
coord_WISEAJ195739=SkyCoord('19h57m38.9s','+50d33m18s')

In [ ]:
coord_WISEAJ195739.ra.deg,coord_WISEAJ195739.dec.deg

In [ ]:
299.41200    50.55500

## from ztfquery import lightcurve

In [80]:
from ztfquery import lightcurve
#lcq_PS16dtm = lightcurve.LCQuery.from_position(29.519793, -0.872746, 3)
#lcq = lightcurve.LCQuery.from_position(299.412,50.55500, 3)

## lcq_PS16dtm

lcq_PS16dtm = lightcurve.LCQuery.from_position(29.519793, -0.872746, 3)

In [ ]:
data_g_PS16dtm = lightcurve.LCQuery.download_data(circle=[29.519793, -0.872746,0.0014], bandname="g",BAD_CATFLAGS_MASK=32768)

In [ ]:
data_i_PS16dtm = lightcurve.LCQuery.download_data(circle=[29.519793, -0.872746,0.0014], bandname="i",BAD_CATFLAGS_MASK=15)

In [ ]:
data_r_PS16dtm= lightcurve.LCQuery.download_data(circle=[29.519793, -0.872746,0.0014], bandname="r",BAD_CATFLAGS_MASK=32768)



In [ ]:
data_g_PS16dtm['flux']=  10**((data_g_PS16dtm['mag']-8.9)*(-0.4))#Jy
data_g_PS16dtm['flux_err'] =0.4 *np.log(10)*data_g_PS16dtm['flux']*data_g_PS16dtm['magerr']


data_i_PS16dtm['flux']=  10**((data_i_PS16dtm['mag']-8.9)*(-0.4))#Jy
data_i_PS16dtm['flux_err'] =0.4 *np.log(10)*data_i_PS16dtm['flux']*data_i_PS16dtm['magerr']


data_r_PS16dtm['flux']=  10**((data_r_PS16dtm['mag']-8.9)*(-0.4))#Jy
data_r_PS16dtm['flux_err'] =0.4 *np.log(10)*data_r_PS16dtm['flux']*data_r_PS16dtm['magerr']

In [ ]:
data_r_PS16dtm.columns

In [ ]:
ztf_data_PS16dtm=[data_g_PS16dtm,data_r_PS16dtm,data_i_PS16dtm]
ztf_bands=['g','r','i']
ztf_bands_color=['green','red','cyan']


## data_TDE_PS16dtm

In [ ]:
data_TDE_PS16dtm=pd.read_excel('/Users/lyubing/Downloads/PS16dtm/PS16dtmTDE.xlsx')

In [ ]:
data_TDE_PS16dtm

In [ ]:
data_TDE_PS16dtm.columns

In [ ]:
len(data_TDE_PS16dtm.columns)

In [ ]:
colors_cm_10 = cm.get_cmap('tab10', 10)

markers_bands_9=['o','+','^','v','d','.','.','.','.']
colors_bands_9=['purple',colors_cm_10(4),'yellow',colors_cm_10(3),'blue',colors_cm_10(0),colors_cm_10(2),'red','cyan']

marker_dict_uv = dict(zip(data_TDE_PS16dtm.columns[1:], markers_bands_9))
color_dict_uv = dict(zip(data_TDE_PS16dtm.columns[1:], colors_bands_9))
#label_dict_uv= dict(zip(data_TDE_PS16dtm.columns[1:], labels_bands))

for index,j in enumerate(data_TDE_PS16dtm['uvw2']):
    if j!='cdots':
        mag= j.split('(')[0]
        magerr=j.split('(')[1].split(')')[0]
        print(data_TDE_PS16dtm.loc[index,'MJD'],mag,magerr)

In [ ]:
fig = plt.figure(figsize=(8,4))
fig.subplots_adjust(hspace=0.0, wspace = 0.0)
ax = fig.add_subplot(111)


BH_mass_PS16dtm=1e6*u.solMass
Ledd_PS16dtm=1.26*10**38*BH_mass_PS16dtm/u.solMass

for i in data_TDE_PS16dtm.columns[1:]:
    for index,j in enumerate(data_TDE_PS16dtm[i]):
        if j!='cdots':
            mag= j.split('(')[0]
            magerr=j.split('(')[1].split(')')[0]
            mag=float(mag)
            magerr=float(magerr)
            
            marker=marker_dict_uv[i]
            color=color_dict_uv[i]
            
            #print(mag,magerr)
            ax.errorbar(data_TDE_PS16dtm.loc[index,'MJD'],mag,magerr,
                        marker=marker,
                        color=color,
                        label=i)
#ax.errorbar(datalag_w1_PS16dtm['t'],datalag_w1_PS16dtm['flux']+np.log10(9),datalag_w1_PS16dtm['err'],label='w1',ls=':',marker='*')
#ax.errorbar(datalag_w2_PS16dtm['t'],datalag_w2_PS16dtm['flux']+np.log10(9),datalag_w2_PS16dtm['err'],label='w2',ls=':',marker='*')

#ax.axhline(np.log10(Ledd_PS16dtm),ls='-.',label='Ledd')

for index,i in enumerate(ztf_data_PS16dtm):
    ax.errorbar(i['mjd'],i['mag'],yerr=i['magerr'],
                ls='',marker='*',
                color='%s'%ztf_bands_color[index],label='ztf+%s'%ztf_bands[index])

plot_rebinmag_w1(ax,data_rebin_neo,data_rebin_mep)
plot_rebinmag_w2(ax,data_rebin_neo,data_rebin_mep)    
    
set_ax_tick(ax)
ax.set_xlabel('MJD')
ax.set_ylabel('mag')
set_mag_ylim(ax)
#set_ax_legend(ax,bbox_to_anchor=(0.99, 0.99))
#ax.legend(bbox_to_anchor=(0.99, 0.99),ncol=3)
ax.xaxis.set_tick_params(which='major', size=10, width=2, rotation=0,)
handles, labels = ax.get_legend_handles_labels()
# remove the errorbars
#hdl = [h[0] for h in handles]
hdl = handles
labels_dict=dict(zip(labels, hdl)) #key,values
#by_label=OrderedDict(sorted(labels_dict.items(),key=lambda t:t[0]))
by_label = OrderedDict(zip(labels, handles))
bbox_to_anchor=(0.99, 0.99)
plot_secax(ax)
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=bbox_to_anchor,
          loc=2, numpoints=1,ncol=3,fontsize=11.)

In [ ]:
lcq_g_PS16dtm = lightcurve.LCQuery(data_g_PS16dtm)
lcq_i_PS16dtm = lightcurve.LCQuery(data_i_PS16dtm)
lcq_r_PS16dtm = lightcurve.LCQuery(data_r_PS16dtm)

In [ ]:
ztflc_show_in_jupyter(data_g_PS16dtm,name,'g',formattime=False)

In [ ]:
ztflc_Lum_show_in_jupyter(data_g_PS16dtm,name,'g',D_cm_PS16dtm,formattime=False)

In [ ]:
ztflc_Lum_show_in_jupyter(data_r_PS16dtm,name,'r',D_cm_PS16dtm,formattime=False)

In [ ]:
ztflc_Lum_show_in_jupyter(data_i_PS16dtm,name,'i',D_cm_PS16dtm,formattime=False)

In [ ]:
ztflc_show_in_jupyter(data_i_PS16dtm,name,'i',formattime=False)

In [ ]:
ztflc_show_in_jupyter(data_r_PS16dtm,name,'r',formattime=False)

In [ ]:
lcq_g_PS16dtm.show()

In [ ]:
lcq_i_PS16dtm.show()

### 51 3 arcsec

In [ ]:
0.0014*3600,1/3600,3/3600

## Changing-Look blazar OQ 334 (B2 1420+32)

In [ ]:
ra_OQ_334=215.62657165527344
dec_OQ_334=32.386234283447266


data_OQ_334 = lightcurve.LCQuery.download_data(circle=[ra_OQ_334, dec_OQ_334,0.00027],BAD_CATFLAGS_MASK=0)

data_OQ_334.to_csv('/Users/lyubing/Downloads/OQ334_ztfdata.csv')

In [ ]:
data_OQ_334.columns

data_OQ_334

In [ ]:
data_OQ_334_g=data_OQ_334[data_OQ_334['filtercode']=='zg']
data_OQ_334_r=data_OQ_334[data_OQ_334['filtercode']=='zr']
data_OQ_334_i=data_OQ_334[data_OQ_334['filtercode']=='zi']

In [ ]:
fig=ztflc_show_in_jupyter(data_OQ_334_g,'OQ_334','g', showtoday=False,formattime=False)
fig.savefig('/Users/lyubing/Downloads/data_OQ_334_g.png',dpi=400, transparent=False, bbox_inches='tight')

In [ ]:
fig=ztflc_show_in_jupyter(data_OQ_334_r,'OQ_334','r', showtoday=False,formattime=False)
fig.savefig('/Users/lyubing/Downloads/data_OQ_334_r.png',dpi=400, transparent=False, bbox_inches='tight')

In [ ]:
fig=ztflc_show_in_jupyter(data_OQ_334_i,'OQ_334','i', showtoday=False,formattime=False)
fig.savefig('/Users/lyubing/Downloads/data_OQ_334_i.png',dpi=400, transparent=False, bbox_inches='tight')

data_OQ_334_g

## ZTF light curve parameters

The complete list of parameters supported by IRSA's ZTF-LC-API implementation is
ID
CIRCLE
POS
BAND
BANDNAME
MAG
NUM_OBS
TIME
BAD_CATFLAGS_MASK
COLLECTION
FORMAT


The BANDNAME parameter identifies by filter-id the wavelength interval(s) to be searched for data. Possible values are "g", "r", and "i", respectively equivalent to "1", "2" and "3", and comma-separated combinations thereof.

## gband 

In [ ]:
data_g = lightcurve.LCQuery.download_data(circle=[299.412,50.55500,0.0014], bandname="g",BAD_CATFLAGS_MASK=32768)

In [ ]:
len(data_g)

data_g.columns

In [ ]:
data_g.head()

In [ ]:
lcq = lightcurve.LCQuery(data_g)

In [ ]:
lcq.show()

## LSBIC gbandZTF

In [ ]:
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
import numpy as np
from matplotlib import pyplot as plt
from astroML.time_series import lomb_scargle, lomb_scargle_BIC, lomb_scargle_bootstrap

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=True)

#------------------------------------------------------------
# Generate Data
np.random.seed(0)


'''
N = 30
P = 0.3
t = np.random.randint(100, size=N) + 0.3 + 0.4 * np.random.random(N)
y = 10 + np.sin(2 * np.pi * t / P)
dy = 0.5 + 0.5 * np.random.random(N)
y_obs = np.random.normal(y, dy)

'''
t = data_g['mjd']
y_obs= -data_g['mag']
dy =data_g['magerr']


#------------------------------------------------------------
# Compute periodogram
period = 10 ** np.linspace(-1, 1, 1000)
omega = 2 * np.pi / period
PS = lomb_scargle(t, y_obs, dy, omega, generalized=True)

#------------------------------------------------------------
# Get significance via bootstrap
D = lomb_scargle_bootstrap(t, y_obs, dy, omega, generalized=True,
                           N_bootstraps=1000, random_state=0)

sig1, sig3, sig2 = np.percentile(D, [68, 99.74, 95.44])

#------------------------------------------------------------



In [ ]:
min(t),max(t),max(t)-min(t),(max(t)-min(t))/len(t)

In [ ]:
# Plot the results
fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(left=0.1, right=0.9, hspace=0.25)

# First panel: the data
ax = fig.add_subplot(211)
ax.errorbar(t, y_obs, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.set_xlabel('MJD')
#ax.set_ylabel('flux')

ax.set(xlabel='MJD',
       ylabel='signal',
       title='g band')

ax.set_xlim(58000, 59900)
set_ax_tick(ax)
plot_secax(ax)
# Second panel: the periodogram & significance levels

ax1 = fig.add_subplot(212, xscale='log')
ax1.plot(period, PS, '-', c='black', lw=1, zorder=1)
#ax1.plot([period[0], period[-1]], [sig1, sig1], ':', c='black')
#ax1.plot([period[0], period[-1]], [sig3, sig3], ':', c='blue')
#ax1.plot([period[0], period[-1]], [sig2, sig2], ':', c='red')
ax1.axhline(sig1, linestyle='dotted', c='black',label=r'1$\sigma$')
ax1.axhline(sig2, linestyle='-', c='red',label=r'2$\sigma$')
ax1.axhline(sig3, linestyle='-.', c='blue',label=r'3$\sigma$')

'''ax1.annotate("", (0.3, 0.65), (0.3, 0.85), ha='center',
             arrowprops=dict(arrowstyle='->'))
'''
#ax1.set_xlim(period[0], period[-1])
ax1.set_ylim(-0.05, 0.1)

ax1.set_xlabel(r'period (days)')
ax1.set_ylabel('power')

# Twin axis: label BIC on the right side
ax2 = ax1.twinx()
ax2.set_ylim(tuple(lomb_scargle_BIC(ax1.get_ylim(), y_obs, dy)))
ax2.set_ylabel(r'$\Delta BIC$')

ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_minor_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_major_locator(plt.LogLocator(10))
ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3g'))

ax1.legend()
#plt.show()
fig.savefig('/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/WISEAJ195739_ls_sig_gband.png',dpi=300)

## iband

In [ ]:
BAD_CATFLAGS_MASK=15

In [ ]:
data_i = lightcurve.LCQuery.download_data(circle=[299.412,50.55500,0.0014], bandname="i",BAD_CATFLAGS_MASK=15)

In [ ]:
len(data_i )

## rband

In [ ]:
data_r_3arcsec= lightcurve.LCQuery.download_data(circle=[299.412,50.55500,3.0/3600], bandname="r",BAD_CATFLAGS_MASK=15)
print(len(data_r_3arcsec))

In [ ]:
data_r= lightcurve.LCQuery.download_data(circle=[299.412,50.55500,0.0014], bandname="r",BAD_CATFLAGS_MASK=32768)

In [ ]:
len(data_r)

In [ ]:
'''
N = 30
P = 0.3
t = np.random.randint(100, size=N) + 0.3 + 0.4 * np.random.random(N)
y = 10 + np.sin(2 * np.pi * t / P)
dy = 0.5 + 0.5 * np.random.random(N)
y_obs = np.random.normal(y, dy)

'''
t = data_r['mjd']
y_obs= -data_r['mag']
dy =data_r['magerr']


#------------------------------------------------------------
# Compute periodogram
period = 10 ** np.linspace(-1, 1, 1000)
omega = 2 * np.pi / period
PS = lomb_scargle(t, y_obs, dy, omega, generalized=True)

#------------------------------------------------------------
# Get significance via bootstrap
D = lomb_scargle_bootstrap(t, y_obs, dy, omega, generalized=True,
                           N_bootstraps=1000, random_state=0)

sig1, sig3, sig2 = np.percentile(D, [68, 99.74, 95.44])

#------------------------------------------------------------



In [ ]:
min(t),max(t),max(t)-min(t),(max(t)-min(t))/len(t),len(t)

In [ ]:
# Plot the results
fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(left=0.1, right=0.9, hspace=0.25)

# First panel: the data
ax = fig.add_subplot(211)
ax.errorbar(t, y_obs, dy, fmt='ok', ecolor='gray',
            markersize=3, capsize=0)
#ax.set_xlabel('MJD')
#ax.set_ylabel('flux')

ax.set(xlabel='MJD',
       ylabel='signal',
       title='r band')

ax.set_xlim(58000, 59900)
set_ax_tick(ax)
plot_secax(ax)
# Second panel: the periodogram & significance levels

ax1 = fig.add_subplot(212, xscale='log')
ax1.plot(period, PS, '-', c='black', lw=1, zorder=1)
#ax1.plot([period[0], period[-1]], [sig1, sig1], ':', c='black')
#ax1.plot([period[0], period[-1]], [sig3, sig3], ':', c='blue')
#ax1.plot([period[0], period[-1]], [sig2, sig2], ':', c='red')
ax1.axhline(sig1, linestyle='dotted', c='black',label=r'1$\sigma$')
ax1.axhline(sig2, linestyle='-', c='red',label=r'2$\sigma$')
ax1.axhline(sig3, linestyle='-.', c='blue',label=r'3$\sigma$')

'''ax1.annotate("", (0.3, 0.65), (0.3, 0.85), ha='center',
             arrowprops=dict(arrowstyle='->'))
'''
#ax1.set_xlim(period[0], period[-1])
ax1.set_ylim(-0.05, 0.1)

ax1.set_xlabel(r'period (days)')
ax1.set_ylabel('power')

# Twin axis: label BIC on the right side
ax2 = ax1.twinx()
ax2.set_ylim(tuple(lomb_scargle_BIC(ax1.get_ylim(), y_obs, dy)))
ax2.set_ylabel(r'$\Delta BIC$')

ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_minor_formatter(plt.FormatStrFormatter('%.1f'))
ax1.xaxis.set_major_locator(plt.LogLocator(10))
ax1.xaxis.set_major_formatter(plt.FormatStrFormatter('%.3g'))

ax1.legend()
#plt.show()
fig.savefig('/Users/brettlv/Downloads/WWZ/WISEA J195739.10+503320.4/WISEAJ195739_ls_sig_rband.png',dpi=300)

## Mrk590

data_g_Mrk590 = lightcurve.LCQuery.download_data(circle=circle_Mrk590, bandname="g",BAD_CATFLAGS_MASK=32768)

data_i_Mrk590 = lightcurve.LCQuery.download_data(circle=circle_Mrk590, bandname="i",BAD_CATFLAGS_MASK=15)

data_r_Mrk590= lightcurve.LCQuery.download_data(circle=circle_Mrk590, bandname="r",BAD_CATFLAGS_MASK=32768)

In [ ]:
circle_Mrk590=[33.6398442,-0.7666930,2/3600]
data_ztf_Mrk590 = lightcurve.LCQuery.download_data(circle=circle_Mrk590,BAD_CATFLAGS_MASK=32768)

In [ ]:
fig,ax=ztflc_show_in_jupyter(data_ztf_Mrk590,'Mrk590')

In [ ]:
0.0014*3600

In [ ]:
download_wise('Mrk 590',wisedata_path='/Users/lyubing/Downloads/WISEDATA/')

In [ ]:
wise_rebin_lc('Mrk 590','/Users/lyubing/Downloads/WISEDATA/','/Users/lyubing/Downloads/WISEDATA/')

## Mrk1018

In [ ]:
download_wise('Mrk 1018',wisedata_path='/Users/lyubing/Downloads/WISEDATA/')

In [ ]:
wise_rebin_lc('Mrk 1018','/Users/lyubing/Downloads/WISEDATA/','/Users/lyubing/Downloads/WISEDATA/')

In [ ]:
circle_Mrk1018=[31.5666250,  -0.2914444,2/3600]
data_ztf_Mrk1018 = lightcurve.LCQuery.download_data(circle=circle_Mrk1018,BAD_CATFLAGS_MASK=32768)

In [ ]:
fig,ax=ztflc_show_in_jupyter(data_ztf_Mrk1018,'Mrk1018')

## NGC4287

In [75]:
download_wise('NGC 4287',wisedata_path='/Users/lyubing/Downloads/WISEDATA/')

NGC4287


In [81]:
from astroquery.ipac.ned import Ned
from ztfquery import lightcurve

In [111]:
result_table_NGC4287 = Ned.query_object("NGC 4287")

In [112]:
ra_NGC4287=result_table_NGC4287['RA'][0]
dec_NGC4287=result_table_NGC4287['DEC'][0]
ztfdata_downloadradec(ra_NGC4287,dec_NGC4287,pathztf='/Users/lyubing/Downloads/ZTFdata/')

In [101]:
z_NGC4287=result_table_NGC4287['Redshift'][0]

In [102]:
print(z_NGC4287)

0.007188


In [90]:
ztfdata_download_name('NGC 4287',ra_NGC4287,dec_NGC4287,'/Users/lyubing/Downloads/ZTFdata/')

In [118]:
print(ra_NGC4287,dec_NGC4287,z_NGC4287)

185.20208 5.6399 0.007188


## NGC 4278

In [114]:
download_wise('NGC 4278',wisedata_path='/Users/lyubing/Downloads/WISEDATA/')

NGC4278


In [115]:
result_table_NGC4278 = Ned.query_object("NGC 4278")
ra_NGC4278=result_table_NGC4278['RA'][0]
dec_NGC4278=result_table_NGC4278['DEC'][0]
z_NGC4278=result_table_NGC4278['Redshift'][0]
#ztfdata_downloadradec(ra_NGC4278,dec_NGC4278,pathztf='/Users/lyubing/Downloads/ZTFdata/')
ztfdata_download_name('NGC 4278',ra_NGC4278,dec_NGC4278,'/Users/lyubing/Downloads/ZTFdata/')


In [117]:
print(ra_NGC4278,dec_NGC4278,z_NGC4278)

185.02843 29.28076 0.002165


## def plot_wise_ztf_lc

In [109]:
def plot_wise_ztf_lc(name,ra,dec,ztfdir,wisedir,plotdir,marker=".", mec="0.7", ms=6, ecolor="0.7", ls="", zorder=4,**kwargs):
    #filename_lamost='%d.fits'%obsid
    
    prop = dict(marker=marker, mec=mec, ms=ms, ecolor=ecolor, ls=ls, zorder=zorder)
    prop1 = dict(marker='*', mec=mec, ms=ms, ecolor='red', ls=ls, zorder=zorder)
    
    
    
    
    
    ztfsave_name= name.strip().replace(' ','')
    ztf_file_path='%s/%s.csv'%(ztfdir,ztfsave_name)
    data=pd.read_csv(ztf_file_path) 
    
    data_test_neo,data_test_mep=get_wise_data(name,datawise_dir=wisedir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100) 

    
    
    fig=plt.figure()
    fig = plt.figure(figsize=(20,8))
    fig.subplots_adjust(hspace=0.0, wspace = 0.2)
    
    ax1 = fig.add_subplot(211)
    
    
    lc_dataframe = data[data['filtercode']=='zg']
    lc_dataframe=drop_index(lc_dataframe)
    
    lc_dataframe_zr = data[data['filtercode']=='zr']
    lc_dataframe_zr=drop_index(lc_dataframe_zr)
    
    
    
    maxmjd= max(lc_dataframe['mjd'])
    minmjd= min(lc_dataframe['mjd'])
    delta_mjd=maxmjd-minmjd

    if delta_mjd>750: 
        ma_intervals= 365
        min_intervals= 90

    if delta_mjd<750:     
        ma_intervals= delta_mjd//3
        mi_intervals= delta_mjd//6
    
    
    ax1.errorbar(lc_dataframe['mjd'], lc_dataframe["mag"], yerr=lc_dataframe["magerr"],label='zg',
                                **{**prop,**kwargs})
    
    ax1.errorbar(lc_dataframe_zr['mjd'], lc_dataframe_zr["mag"], yerr=lc_dataframe_zr["magerr"],label='zr',
                               **{**prop1,**kwargs})

        
        
    ax1.set_xlabel("MJD")
    #plot_secax(ax1,mi_interval=ma_intervals,ma_interval=min_intervals,rotation=30,)  
    plot_secax(ax1,rotation=30,)  
    #ax1.axvline(mjd,color='black',ls=':',linewidth=1)

    handles, labels = ax1.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax1.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    #ax1.set_xlim(58100,59999)
    
    
    ax2 = fig.add_subplot(212,sharex=ax1)
    
    plot_rebinmag_w1(ax2,data_rebin_neo,data_rebin_mep)
    plot_rebinmag_w2(ax2,data_rebin_neo,data_rebin_mep)

    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

    #plot_secax(ax2,mi_interval=365,ma_interval=365*2)
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

    handles, labels = ax2.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    ax1.invert_yaxis()
    ax2.invert_yaxis()

    ax2.set_xlabel('mjd')
    ax1.set_ylabel(r'$mag$')    
    ax2.set_ylabel(r'$mag$')  
    ax1.set_title('%s'%(ztfsave_name))
    
    
    save_lc_img_path=os.path.join(plotdir,'%s.png'%(ztfsave_name))
    if not os.path.exists(save_lc_img_path):
        plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()    
        
    #return fig,ax,ax1,ax2


In [113]:
plot_wise_ztf_lc('NGC 4287',ra_NGC4287,dec_NGC4287,'/Users/lyubing/Downloads/ZTFdata/','/Users/lyubing/Downloads/WISEDATA/',plotdir='/Users/lyubing/Downloads/PLOTDIR/')

<Figure size 432x288 with 0 Axes>

In [116]:
plot_wise_ztf_lc('NGC 4278',ra_NGC4278,dec_NGC4278,'/Users/lyubing/Downloads/ZTFdata/','/Users/lyubing/Downloads/WISEDATA/',plotdir='/Users/lyubing/Downloads/PLOTDIR/')



<Figure size 432x288 with 0 Axes>

# WWZ

ls.py

In [ ]:
WWZ_path='/Volumes/ExtremeSSD/Code_backup/timing_analysis/WWZ/'

In [ ]:
pwd_path='/Users/brettlv/blog/brettlv.github.io/pythoncode/changinglookAGN'

In [ ]:
cmd ='python2 ls.py -f W1_timemag.csv -o ls_freq_power.txt -l 0.001 -hi 0.1 -n 10000'
print(cmd)

In [ ]:
data=pd.read_csv('%s/W1_timemag.csv'%WWZ_path,header=None,delimiter=',')
data.columns=['mjd','arbitary_flux']

In [ ]:
np.array(data['mjd']),np.array(data['arbitary_flux'])

In [ ]:
cmd ='python2 ls_origin.py -f W1_t_mag.py -o ls_freq_power.txt -l 0.001 -hi 0.1 -n 10000'
print(cmd)

In [ ]:
#data_time = []
#data_flux= []
data_time=np.loadtxt('%s/W1_timemag.csv'%WWZ_path,dtype=np.str,unpack=True, usecols=[0],delimiter=",")
data_flux=np.loadtxt('%s/W1_timemag.csv'%WWZ_path,dtype=np.str,unpack=True, usecols=[1],delimiter=",")



In [ ]:
data_time,data_flux

In [ ]:
ls_freq_power=pd.read_csv('%s/ls_freq_power.txt'%WWZ_path,header=None,delimiter='\s+')
#ls_freq_power.columns=['mjd','arbitary_flux']

In [ ]:
np.array(ls_freq_power[ls_freq_power.columns[0]]),np.array(ls_freq_power[ls_freq_power.columns[1]])

In [ ]:
np.array(ls_freq_power[ls_freq_power.columns[0]])==ls_freq

In [ ]:
#ls_freq = []
#ls_power = []
ls_freq=np.loadtxt('%s/ls_freq_power.txt'%WWZ_path,dtype=np.float,unpack=True, usecols=[0])
ls_power=np.loadtxt('%s/ls_freq_power.txt'%WWZ_path,dtype=np.float,unpack=True, usecols=[1])

In [ ]:
ls_freq,ls_power

In [ ]:
# Plot the results
fig = plt.figure(figsize=(6, 6))
fig.subplots_adjust(left=0.1, right=0.9, hspace=0.25)

# First panel: the data
ax = fig.add_subplot(111)
ax.scatter(ls_freq,ls_power)
set_ax_tick(ax)


# CLB

## plot_lc

In [ ]:
CLB_list_CLB_neogt20_var=CLB_list_CLB_neogt20[CLB_list_CLB_neogt20['sigma_m_w1_neo']>0.2]
CLB_list_CLB_neogt20_var=drop_index(CLB_list_CLB_neogt20_var)#['V1']

In [ ]:
len(CLB_list_CLB_neogt20),len(CLB_list_CLB_neogt20_var),len(CLB_list_CLB_neogt20_var0)

In [ ]:
CLB_list_CLB_neogt20_var0=CLB_list_CLB_neogt20[CLB_list_CLB_neogt20['sigma_m_w1_neo']<0.1]
CLB_list_CLB_neogt20_var0=drop_index(CLB_list_CLB_neogt20_var0)#['V1']

CLB_list_CLB_neogt20_var['len_neo']

In [ ]:
save_lc_img_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc/CLB_var0'
if not os.path.exists(save_lc_img_dir):
    os.mkdir(save_lc_img_dir)
for index,name in enumerate(CLB_list_CLB_neogt20_var0['V1']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    neo_path='%s/wise_%s_neo.csv'%(CLB_wisedata_dir,save_name)
    #if os.path.exists(neo_path):
        #print(index,neo_path)
    wise_plot_lc(save_name,save_lc_img_dir=save_lc_img_dir,datawise_dir=CLB_wisedata_dir)

## Fermi_bll_lc

In [ ]:
save_lc_img_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc/Fermi_bll'
if not os.path.exists(save_lc_img_dir):
    os.mkdir(save_lc_img_dir)
for index,name in enumerate(Fermi_4LAC_DR3_h_bll_pd_neogt20['Source_Name']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    
    #name_neo= os.path.join(Fermi_bll_wisedata_dir,'wise_%s_neo.csv'%save_name)
    #name_mep= os.path.join(Fermi_bll_wisedata_dir,'wise_%s_mep.csv'%save_name)
    
    #if os.path.exists(name_neo) and os.path.exists(name_mep):     
    wise_plot_lc(save_name,save_lc_img_dir=save_lc_img_dir,datawise_dir=Fermi_bll_wisedata_dir)

In [ ]:
Fermi_bll_wisedata_dir

In [ ]:
!open /Volumes/ExtremeSSD/data_source/WISE_data/WISE_fermi_blazars/BLL

In [ ]:
save_lc_img_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_lc/Fermi_fsrq'
if not os.path.exists(save_lc_img_dir):
    os.mkdir(save_lc_img_dir)
for index,name in enumerate(Fermi_4LAC_DR3_h_fsrq_pd_neogt20['Source_Name']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    wise_plot_lc(save_name,save_lc_img_dir=save_lc_img_dir,datawise_dir=Fermi_fsrq_wisedata_dir)

# CLB_list_CLB

In [122]:
CLB_list=pd.read_csv('/Users/lyubing/Downloads/kangshiju_paper/A02_Table_catalog_plot.csv')

In [123]:
CLB_list_CLB=CLB_list[CLB_list['V5']=='CLB']
CLB_list_CLB=drop_index(CLB_list_CLB)

In [124]:
CLB_list_CLB['V1']

0      4FGL J0006.3-0620
1      4FGL J0014.2+0854
2      4FGL J0022.0+0006
3      4FGL J0058.4+3315
4      4FGL J0058.4+3315
             ...        
132    4FGL J2315.6-5018
133    4FGL J2315.6-5018
134    4FGL J2345.2-1555
135    4FGL J2349.4+0534
136    4FGL J2357.4-0152
Name: V1, Length: 137, dtype: object

for index,name in enumerate(CLB_list_CLB_neogt20['V1'][:1]):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')    
    #if os.path.exists('./wise_%s_mep.csv'%save_name): 
    neo_path='%s/wise_%s_neo.csv'%(CLB_wisedata_dir,save_name)
    if os.path.exists(neo_path):# and os.path.exists('./wise_%s_mep.csv'%save_name):
        print(save_name)

## CLB_list_CLB_neogt20

CLB_list_CLB

In [128]:
for index,name in enumerate(CLB_list_CLB['V1']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    #if os.path.exists('./wise_%s_mep.csv'%save_name):
    neo_path='%s/wise_%s_neo.csv'%(CLB_wisedata_dir,save_name)
    if not os.path.exists(neo_path):# and os.path.exists('./wise_%s_mep.csv'%save_name):
    #if not os.path.exists('./CLB_list_CLB/wise_%s_neo.csv'%save_name):
        CLB_list_CLB.loc[index,'len_neo']=0
        continue
    data=pd.read_csv(neo_path)
    data=data[data['w1mpro'].notnull()]
    data=data[data['w2mpro'].notnull()]
    data=data[data['w1sigmpro'].notnull()]
    data=data[data['w2sigmpro'].notnull()]
    
    #data=data[data['w1mpro']<15]
    #data=data[data['w2mpro']<13]
    #data=data[data['w1rchi2']<5]
    #data=data[data['w2rchi2']<5]
    #data=data[data['nb']<3]
    #data=data[data['na']==0]
    data=data[data['qual_frame']>0]
    data=data.reset_index(drop=True)
    #data=data.fillna(0)
    if len(data)>5:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1
        
        pearson_coeff,p_value=pearsonr(data['w1mpro'],data['w2mpro'])
        CLB_list_CLB.loc[index,'pearson_coeff_neo']=pearson_coeff
        
        sigma_m_w1=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        CLB_list_CLB.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        CLB_list_CLB.loc[index,'mean_m_w1_neo']=np.average(data[mag_label])
        
        
        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        CLB_list_CLB.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        CLB_list_CLB.loc[index,'mean_m_w2_neo']=np.average(data[mag_label])
        CLB_list_CLB.loc[index,'len_neo']=len(data)
        CLB_list_CLB.loc[index,'W1-W2']=np.average(data['w1mpro']-data['w2mpro'])
    
    else:
        CLB_list_CLB.loc[index,'len_neo']=0

In [129]:
CLB_list_CLB_neogt20=CLB_list_CLB[CLB_list_CLB['len_neo']>20]
CLB_list_CLB_neogt20=drop_index(CLB_list_CLB_neogt20)

CLB_list_CLB_neogt20.to_csv('CLB_neogt20.csv',index=False)

In [130]:
CLB_list_CLB_neogt20=pd.read_csv('CLB_neogt20.csv')

for i in CLB_list_CLB_neogt20.columns:
    print(i)

In [131]:
for index,z in enumerate(CLB_list_CLB_neogt20['Redshift.y']):
    d_cm=cosmo.luminosity_distance(z=z).to(u.cm).value
    
    CLB_list_CLB_neogt20.loc[index,'LW1']=W1mag_to_Lum(CLB_list_CLB_neogt20.loc[index,'mean_m_w1_neo'],d_cm)
    CLB_list_CLB_neogt20.loc[index,'LW2']=W1mag_to_Lum(CLB_list_CLB_neogt20.loc[index,'mean_m_w2_neo'],d_cm)
    

CLB_list_CLB_neogt20[CLB_list_CLB_neogt20['sigma_m_w1_neo']==0]['len_neo']

In [ ]:
len(CLB_list_CLB_neogt20)

# CLB_list_CLB_candidates

In [132]:
CLB_list_CLB_candidates=CLB_list[CLB_list['V5']=='Predictions']
CLB_list_CLB_candidates=drop_index(CLB_list_CLB_candidates)

In [135]:
CLBcand_wisedata_dir

'/Volumes/ExtremeSSD/data_source/WISE_data/WISE_CLB_candidates'

In [133]:
for index,name in enumerate(CLB_list_CLB_candidates['V1']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    #if os.path.exists('./wise_%s_mep.csv'%save_name):
    neo_path='%s/wise_%s_neo.csv'%(CLBcand_wisedata_dir,save_name)
    if not os.path.exists(neo_path):# and os.path.exists('./wise_%s_mep.csv'%save_name):
    #if not os.path.exists('./CLB_list_CLB/wise_%s_neo.csv'%save_name):
        CLB_list_CLB.loc[index,'len_neo']=0
        continue
    data=pd.read_csv(neo_path)
    data=data[data['w1mpro'].notnull()]
    data=data[data['w2mpro'].notnull()]
    data=data[data['w1sigmpro'].notnull()]
    data=data[data['w2sigmpro'].notnull()]
    
    #data=data[data['w1mpro']<15]
    #data=data[data['w2mpro']<13]
    #data=data[data['w1rchi2']<5]
    #data=data[data['w2rchi2']<5]
    #data=data[data['nb']<3]
    #data=data[data['na']==0]
    data=data[data['qual_frame']>0]
    data=data.reset_index(drop=True)
    #data=data.fillna(0)
    if len(data)>5:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1
        
        pearson_coeff,p_value=pearsonr(data['w1mpro'],data['w2mpro'])
        CLB_list_CLB.loc[index,'pearson_coeff_neo']=pearson_coeff
        
        sigma_m_w1=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        CLB_list_CLB_candidates.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        CLB_list_CLB_candidates.loc[index,'mean_m_w1_neo']=np.average(data[mag_label])
        
        
        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        CLB_list_CLB_candidates.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        CLB_list_CLB_candidates.loc[index,'mean_m_w2_neo']=np.average(data[mag_label])
        CLB_list_CLB_candidates.loc[index,'len_neo']=len(data)
        CLB_list_CLB_candidates.loc[index,'W1-W2']=np.average(data['w1mpro']-data['w2mpro'])
    
    else:
        CLB_list_CLB_candidates.loc[index,'len_neo']=0

## CLB_list_CLB_candidates_neogt20

In [134]:
CLB_list_CLB_candidates_neogt20=CLB_list_CLB_candidates[CLB_list_CLB_candidates['len_neo']>20]
CLB_list_CLB_candidates_neogt20=drop_index(CLB_list_CLB_candidates_neogt20)

KeyError: 'len_neo'

In [127]:
for index,z in enumerate(CLB_list_CLB_candidates_neogt20['Redshift.y']):
    d_cm=cosmo.luminosity_distance(z=z).to(u.cm).value
    
    CLB_list_CLB_candidates_neogt20.loc[index,'LW1']=W1mag_to_Lum(CLB_list_CLB_candidates_neogt20.loc[index,'mean_m_w1_neo'],d_cm)
    CLB_list_CLB_candidates_neogt20.loc[index,'LW2']=W1mag_to_Lum(CLB_list_CLB_candidates_neogt20.loc[index,'mean_m_w2_neo'],d_cm)
    

NameError: name 'CLB_list_CLB_candidates_neogt20' is not defined

# Fermi_4LAC_DR3_h

In [ ]:
'Source_Name'

In [ ]:
Fermi_4LAC_DR3_h = fits.open('/Users/lyubing/Downloads/kangshiju_paper/table-4LAC-DR3-h.fits')
Fermi_4LAC_DR3_h_data = Fermi_4LAC_DR3_h[1].data

In [ ]:
len(Fermi_4LAC_DR3_h_bll),len(Fermi_4LAC_DR3_h_fsrq)

In [ ]:
Fermi_4LAC_DR3_h_bll = Fermi_4LAC_DR3_h_data[Fermi_4LAC_DR3_h_data['CLASS'].lower()=='bll']

In [ ]:
Fermi_4LAC_DR3_h_fsrq=Fermi_4LAC_DR3_h_data[Fermi_4LAC_DR3_h_data['CLASS'].lower()=='fsrq']

In [ ]:
Fermi_4LAC_DR3_h_bll_pd=Table(Fermi_4LAC_DR3_h_bll).to_pandas()

In [ ]:
Fermi_4LAC_DR3_h_fsrq_pd=Table(Fermi_4LAC_DR3_h_fsrq).to_pandas()

In [ ]:
Fermi_4LAC_DR3_h_bll_pd.columns

## Fermi_4LAC_DR3_h_bll_pd

In [ ]:
for index,name in enumerate(Fermi_4LAC_DR3_h_bll_pd['Source_Name']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    #if os.path.exists('./wise_%s_mep.csv'%save_name):
    neo_path='%s/wise_%s_neo.csv'%(Fermi_bll_wisedata_dir,save_name)
    if not os.path.exists(neo_path):# and os.path.exists('./wise_%s_mep.csv'%save_name):
    #if not os.path.exists('./Fermi_4LAC_DR3_h_bll_pd/wise_%s_neo.csv'%save_name):
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'len_neo']=0
        continue
    data=pd.read_csv(neo_path)
    data=data[data['w1mpro'].notnull()]
    data=data[data['w2mpro'].notnull()]
    data=data[data['w1sigmpro'].notnull()]
    data=data[data['w2sigmpro'].notnull()]
    
    #data=data[data['w1mpro']<15]
    #data=data[data['w2mpro']<13]
    #data=data[data['w1rchi2']<5]
    #data=data[data['w2rchi2']<5]
    #data=data[data['nb']<3]
    #data=data[data['na']==0]
    data=data[data['qual_frame']>0]
    data=data.reset_index(drop=True)
    #data=data.fillna(0)
    if len(data)>5:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1
        
        pearson_coeff,p_value=pearsonr(data['w1mpro'],data['w2mpro'])
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'pearson_coeff_neo']=pearson_coeff
        
        sigma_m_w1=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'mean_m_w1_neo']=np.average(data[mag_label])
        
        
        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'mean_m_w2_neo']=np.average(data[mag_label])
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'len_neo']=len(data)
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'W1-W2']=np.average(data['w1mpro']-data['w2mpro'])
    
    else:
        Fermi_4LAC_DR3_h_bll_pd.loc[index,'len_neo']=0


In [ ]:
Fermi_4LAC_DR3_h_bll_pd_neogt20=Fermi_4LAC_DR3_h_bll_pd[Fermi_4LAC_DR3_h_bll_pd['len_neo']>20]
Fermi_4LAC_DR3_h_bll_pd_neogt20=drop_index(Fermi_4LAC_DR3_h_bll_pd_neogt20)

In [ ]:
Fermi_4LAC_DR3_h_bll_pd_neogt20.to_csv('Fermi_4LAC_DR3_h_bll_pd_neogt20.csv',index=False)

In [ ]:
Fermi_4LAC_DR3_h_bll_pd_neogt20.columns

for i in Fermi_4LAC_DR3_h_bll_pd_neogt20[Fermi_4LAC_DR3_h_bll_pd_neogt20['sigma_m_w1_neo']==0]['len_neo']:
    print(i)

for i in Fermi_4LAC_DR3_h_fsrq_pd_neogt20[Fermi_4LAC_DR3_h_fsrq_pd_neogt20['sigma_m_w1_neo']==0]['len_neo']:
    print(i)

In [ ]:
#D_cm_bll= cosmo.luminosity_distance(z=Fermi_4LAC_DR3_h_bll_pd_neogt20['Redshift']).to(u.cm) 
for index,z in enumerate(Fermi_4LAC_DR3_h_bll_pd_neogt20['Redshift']):
    if z>0:
        d_cm=cosmo.luminosity_distance(z=z).to(u.cm).value    
        Fermi_4LAC_DR3_h_bll_pd_neogt20.loc[index,'LW1']=W1mag_to_Lum(Fermi_4LAC_DR3_h_bll_pd_neogt20.loc[index,'mean_m_w1_neo'],d_cm)
        Fermi_4LAC_DR3_h_bll_pd_neogt20.loc[index,'LW2']=W1mag_to_Lum(Fermi_4LAC_DR3_h_bll_pd_neogt20.loc[index,'mean_m_w2_neo'],d_cm)
    else:
        Fermi_4LAC_DR3_h_bll_pd_neogt20.loc[index,'LW1']=-1
        Fermi_4LAC_DR3_h_bll_pd_neogt20.loc[index,'LW2']=-1
           

### Withredshift

In [ ]:
Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift=Fermi_4LAC_DR3_h_bll_pd_neogt20[Fermi_4LAC_DR3_h_bll_pd_neogt20['LW1']>0]
Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift=drop_index(Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift)                                                                                                                           
    

In [ ]:
plt.hist(Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift['Redshift'])

## Fermi_4LAC_DR3_h_fsrq_pd

In [ ]:
for index,name in enumerate(Fermi_4LAC_DR3_h_fsrq_pd['Source_Name']):
    search_name=name.strip()
    save_name=name.strip().replace(' ','')
    #if os.path.exists('./wise_%s_mep.csv'%save_name):
    neo_path='%s/wise_%s_neo.csv'%(Fermi_fsrq_wisedata_dir,save_name)
    if not os.path.exists(neo_path):# and os.path.exists('./wise_%s_mep.csv'%save_name):
    #if not os.path.exists('./Fermi_4LAC_DR3_h_fsrq_pd/wise_%s_neo.csv'%save_name):
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'len_neo']=0
        continue
    data=pd.read_csv(neo_path)
    data=data[data['w1mpro'].notnull()]
    data=data[data['w2mpro'].notnull()]
    data=data[data['w1sigmpro'].notnull()]
    data=data[data['w2sigmpro'].notnull()]
    
    #data=data[data['w1mpro']<15]
    #data=data[data['w2mpro']<13]
    #data=data[data['w1rchi2']<5]
    #data=data[data['w2rchi2']<5]
    #data=data[data['nb']<3]
    #data=data[data['na']==0]
    data=data[data['qual_frame']>0]
    data=data.reset_index(drop=True)
    #data=data.fillna(0)
    if len(data)>5:
        mag_label='w1mpro'
        magerr_label='w1sigmpro'
        epsilon_s=epsilon_s_w1
        
        pearson_coeff,p_value=pearsonr(data['w1mpro'],data['w2mpro'])
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'pearson_coeff_neo']=pearson_coeff
        
        sigma_m_w1=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'sigma_m_w1_neo']=sigma_m_w1
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'mean_m_w1_neo']=np.average(data[mag_label])
        
        
        mag_label='w2mpro'
        magerr_label='w2sigmpro'
        epsilon_s=epsilon_s_w2
        sigma_m_w2=get_intrinsic_var(data,mag_label,magerr_label,epsilon_s)
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'sigma_m_w2_neo']=sigma_m_w2
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'mean_m_w2_neo']=np.average(data[mag_label])
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'len_neo']=len(data)
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'W1-W2']=np.average(data['w1mpro']-data['w2mpro'])
    
    else:
        Fermi_4LAC_DR3_h_fsrq_pd.loc[index,'len_neo']=0


In [ ]:
Fermi_4LAC_DR3_h_fsrq_pd_neogt20=Fermi_4LAC_DR3_h_fsrq_pd[Fermi_4LAC_DR3_h_fsrq_pd['len_neo']>20]
Fermi_4LAC_DR3_h_fsrq_pd_neogt20=drop_index(Fermi_4LAC_DR3_h_fsrq_pd_neogt20)


In [ ]:
#Fermi_4LAC_DR3_h_fsrq_pd_neogt20.to_csv('Fermi_4LAC_DR3_h_fsrq_pd_neogt20.csv',index=False)

In [ ]:
for index,z in enumerate(Fermi_4LAC_DR3_h_fsrq_pd_neogt20['Redshift']):
    d_cm=cosmo.luminosity_distance(z=z).to(u.cm).value
    
    Fermi_4LAC_DR3_h_fsrq_pd_neogt20.loc[index,'LW1']=W1mag_to_Lum(Fermi_4LAC_DR3_h_fsrq_pd_neogt20.loc[index,'mean_m_w1_neo'],d_cm)
    Fermi_4LAC_DR3_h_fsrq_pd_neogt20.loc[index,'LW2']=W1mag_to_Lum(Fermi_4LAC_DR3_h_fsrq_pd_neogt20.loc[index,'mean_m_w2_neo'],d_cm)
    
   

!open .

In [ ]:
CLB_list_CLB_neogt20,Fermi_4LAC_DR3_h_bll_pd_neogt20,Fermi_4LAC_DR3_h_fsrq_pd_neogt20

Fermi_4LAC_DR3_h_bll_pd_neogt20

In [ ]:
Fermi_4LAC_DR3_h_bll_pd_neogt20.columns


## sigma_W1 W1-W2

In [ ]:
#fig = plt.figure(figsize=(6, 6))
#ax_histx=fig.add_subplot(1,1,1)

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.005

#x = np.random.randn(1000)
#y = np.random.randn(1000)

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.4, height]

# start with a square Figure
fig = plt.figure(figsize=(6, 6))

ax = fig.add_axes(rect_scatter)
#ax_histx = fig.add_axes(rect_histx, sharex=ax)
'''ax_histy = fig.add_axes(rect_histy, sharey=ax)
#ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
'''

'''for index,name in enumerate(Extreme_QSO_select['Name']):
    ax.scatter(Extreme_QSO_select.loc[index,'logL_W1']-Extreme_QSO_select.loc[index,'logBH']-np.log10(1.263e38),
               Extreme_QSO_select.loc[index,'W1-W2'],
               s=size,color='cyan',marker='*',alpha=0.5,label='EVQ',zorder=1)
'''
        
select_plot=Fermi_4LAC_DR3_h_bll_pd_neogt20
xplot_label='W1-W2'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='BLL',
           ms=3., mew=1, capsize=0,mec='r',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=0)


select_plot=Fermi_4LAC_DR3_h_fsrq_pd_neogt20
xplot_label='W1-W2'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='FSRQ',
           ms=3., mew=1, capsize=0,mec='g',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=0)



select_plot=CLB_list_CLB_neogt20
xplot_label='W1-W2'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB',
           ms=6., mew=1, capsize=0,mec='blue',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)



select_plot=CLB_list_CLB_candidates_neogt20
xplot_label='W1-W2'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB_candidates',
           ms=3., mew=0, capsize=0,mec='blue',ecolor='blue',
           mfc='blue',
           elinewidth=2,ls='',fillstyle='full',zorder=0)




set_ax_legend(ax,bbox_to_anchor=(0.51,0.99))
ax.set_xlabel('<W1-W2>')
ax.set_ylabel(r'$\sigma_{W1}$')
ax.set_ylim(0.01,1.8)
set_ax_tick(ax)

fig.savefig('%s/WISE_plot/CLB_WISE_sigmaW1_W1-W2.png'%CLAGN_statistic_plot_dir,
            dpi=400, transparent=False, bbox_inches='tight')
plt.show()
plt.close()

## LW1 sigma W1

In [ ]:
#fig = plt.figure(figsize=(6, 6))
#ax_histx=fig.add_subplot(1,1,1)

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.005

#x = np.random.randn(1000)
#y = np.random.randn(1000)

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.4, height]

# start with a square Figure
fig = plt.figure(figsize=(6, 6))

ax = fig.add_axes(rect_scatter)
#ax_histx = fig.add_axes(rect_histx, sharex=ax)
'''ax_histy = fig.add_axes(rect_histy, sharey=ax)
#ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
'''

'''for index,name in enumerate(Extreme_QSO_select['Name']):
    ax.scatter(Extreme_QSO_select.loc[index,'logL_W1']-Extreme_QSO_select.loc[index,'logBH']-np.log10(1.263e38),
               Extreme_QSO_select.loc[index,'W1-W2'],
               s=size,color='cyan',marker='*',alpha=0.5,label='EVQ',zorder=1)
'''
        
select_plot=Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift
xplot_label='LW1'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='BLL',
           ms=3., mew=1, capsize=0,mec='r',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)


select_plot=Fermi_4LAC_DR3_h_fsrq_pd_neogt20
xplot_label='LW1'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='FSRQ',
           ms=3., mew=1, capsize=0,mec='g',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)



select_plot=CLB_list_CLB_neogt20
xplot_label='LW1'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB',
           ms=6., mew=1, capsize=0,mec='blue',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)


select_plot=CLB_list_CLB_candidates_neogt20
xplot_label='LW1'
yplot_label='sigma_m_w1_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB_candidates',
           ms=3., mew=0, capsize=0,mec='blue',ecolor='blue',
           mfc='blue',
           elinewidth=2,ls='',fillstyle='full',zorder=0)


ax.set_xlabel('LW1')
ax.set_ylabel(r'$\sigma_{W1}$')
ax.set_ylim(0.01,1.8)

set_ax_tick(ax)
set_ax_legend(ax,bbox_to_anchor=(0.51,0.99))

fig.savefig('%s/WISE_plot/CLB_WISE_sigmaW1-LW1.png'%CLAGN_statistic_plot_dir,
            dpi=400, transparent=False, bbox_inches='tight')
plt.show()
plt.close()

## LW2 sigma w2

In [ ]:
#fig = plt.figure(figsize=(6, 6))
#ax_histx=fig.add_subplot(1,1,1)

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.005

#x = np.random.randn(1000)
#y = np.random.randn(1000)

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.4, height]

# start with a square Figure
fig = plt.figure(figsize=(6, 6))

ax = fig.add_axes(rect_scatter)
#ax_histx = fig.add_axes(rect_histx, sharex=ax)
'''ax_histy = fig.add_axes(rect_histy, sharey=ax)
#ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)
'''

'''for index,name in enumerate(Extreme_QSO_select['Name']):
    ax.scatter(Extreme_QSO_select.loc[index,'logL_W1']-Extreme_QSO_select.loc[index,'logBH']-np.log10(1.263e38),
               Extreme_QSO_select.loc[index,'W1-W2'],
               s=size,color='cyan',marker='*',alpha=0.5,label='EVQ',zorder=1)
'''
        
select_plot=Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift
xplot_label='LW2'
yplot_label='sigma_m_w2_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='BLL',
           ms=3., mew=1, capsize=0,mec='r',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)


select_plot=Fermi_4LAC_DR3_h_fsrq_pd_neogt20
xplot_label='LW2'
yplot_label='sigma_m_w2_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='FSRQ',
           ms=3., mew=1, capsize=0,mec='g',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)



select_plot=CLB_list_CLB_neogt20
xplot_label='LW2'
yplot_label='sigma_m_w2_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB',
           ms=6., mew=1, capsize=0,mec='blue',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)


select_plot=CLB_list_CLB_candidates_neogt20
xplot_label='LW2'
yplot_label='sigma_m_w2_neo'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB_candidates',
           ms=3., mew=0, capsize=0,mec='blue',ecolor='blue',
           mfc='blue',
           elinewidth=2,ls='',fillstyle='full',zorder=0)


ax.set_xlabel('LW2')
ax.set_ylabel(r'$\sigma_{W2}$')
ax.set_ylim(0.01,1.8)

set_ax_tick(ax)
set_ax_legend(ax,bbox_to_anchor=(0.51,0.99))

fig.savefig('%s/WISE_plot/CLB_WISE_sigmaW2-LW2.png'%CLAGN_statistic_plot_dir,
            dpi=400, transparent=False, bbox_inches='tight')
plt.show()
plt.close()


## Lw1 vs W1-W2

In [ ]:
#fig = plt.figure(figsize=(6, 6))
#ax_histx=fig.add_subplot(1,1,1)

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.005

#x = np.random.randn(1000)
#y = np.random.randn(1000)

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.4]
rect_histy = [left + width + spacing, bottom, 0.4, height]

# start with a square Figure
fig = plt.figure(figsize=(6, 6))

ax = fig.add_axes(rect_scatter)

ax_histx = fig.add_axes(rect_histx, sharex=ax)
ax_histy = fig.add_axes(rect_histy, sharey=ax)
ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)



        
#select_plot=Fermi_4LAC_DR3_h_bll_pd_neogt20 
select_plot=Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift

xplot_label='LW1'
yplot_label='W1-W2'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='BLL',
           ms=3., mew=1, capsize=0,mec='r',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=0)
ax_histx.hist(select_plot[xplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         #linestyle ='--',
              linewidth=2,
        label='BLL',edgecolor='red',
              #orientation='horizontal'
             )
ax_histy.hist(select_plot[yplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         #linestyle ='--',
              linewidth=2,
        label='BLL',edgecolor='red',
              orientation='horizontal'
             )



select_plot=Fermi_4LAC_DR3_h_fsrq_pd_neogt20
xplot_label='LW1'
yplot_label='W1-W2'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='FSRQ',
           ms=3., mew=1, capsize=0,mec='g',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=0)
ax_histx.hist(select_plot[xplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         #linestyle ='--',
              linewidth=2,
        label='FSRQ',edgecolor='g',
              #orientation='horizontal'
             )
ax_histy.hist(select_plot[yplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         #linestyle ='--',
              linewidth=2,
        label='FSRQ',edgecolor='g',
              orientation='horizontal'
             )


select_plot=CLB_list_CLB_neogt20
xplot_label='LW1'
yplot_label='W1-W2'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB',
           ms=6., mew=1, capsize=0,mec='blue',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=1)

ax_histx.hist(select_plot[xplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         #linestyle ='--',
              linewidth=2,
        label='CLB',edgecolor='blue',
              #orientation='horizontal'
             )
ax_histy.hist(select_plot[yplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
              #linestyle ='--',
              linewidth=2,
              label='CLB',edgecolor='blue',
              orientation='horizontal'
             )

select_plot=CLB_list_CLB_candidates_neogt20
xplot_label='LW1'
yplot_label='W1-W2'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='o',label='CLB_candidates',
           ms=3., mew=0, capsize=0,mec='blue',ecolor='blue',
           mfc='blue',
           elinewidth=2,ls='',fillstyle='full',zorder=0)

ax_histx.hist(select_plot[xplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
              linestyle ='--',
              linewidth=2,
        label='CLB_candidates',edgecolor='blue',
              #orientation='horizontal'
             )
ax_histy.hist(select_plot[yplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
              linestyle ='--',
              linewidth=2,
              label='CLB_candidates',edgecolor='blue',
              orientation='horizontal'
             )





select_plot=QSO_select
xplot_label='logL_W1'
yplot_label='W1-W2'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='*',label='QSO',
           ms=3., mew=1, capsize=0,mec='grey',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=0)

ax_histx.hist(select_plot[xplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         linestyle ='--',
              linewidth=2,
        label='QSO',edgecolor='grey',
              #orientation='horizontal'
             )
ax_histy.hist(select_plot[yplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
              #linestyle ='--',
              linewidth=2,
              label='QSO',edgecolor='grey',
              orientation='horizontal'
             )



select_plot=LLAGN_gu2009
xplot_label='logL_W1'
yplot_label='W1-W2'
ax.errorbar(x= select_plot[xplot_label],
           y=select_plot[yplot_label],
           marker='+',label='LLAGN',
           ms=3., mew=1, capsize=0,mec='black',ecolor='r',
           elinewidth=2,ls='',fillstyle='none',zorder=0)

ax_histx.hist(select_plot[xplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
         linestyle ='-.',
              linewidth=2,
        label='LLAGN',edgecolor='black',
              #orientation='horizontal'
             )
ax_histy.hist(select_plot[yplot_label], bins=8,fill=False,density=True,stacked =True,
              histtype='step',
              linestyle ='-.',
              linewidth=2,
              label='LLAGN',edgecolor='black',
              orientation='horizontal'
             )





ax.set_xlabel(r'$L_{W1}$')
ax.set_ylabel(r'<W1-W2>')
#ax.set_ylim(0.01,1.3)
set_ax_tick(ax)
#set_ax_legend(ax,bbox_to_anchor=(0.69,0.5))
set_ax_legend(ax_histx,bbox_to_anchor=(0.57,0.99))
set_ax_legend(ax,bbox_to_anchor=(0.61,0.99))
ax_histx.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.2))
ax_histx.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.1))
#ax_histx.set_ylim(0.01,0.9)
#set_ax_tick(ax_histx,)

ax.set_xlim(41-0.1,51.1)

fig.savefig('%s/WISE_plot/CLB_WISE_W1-W2_LW1.png'%CLAGN_statistic_plot_dir,
            dpi=400, transparent=False, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
CLAGN_statistic_plot_dir

In [ ]:
!open /Volumes/ExtremeSSD/CLAGN_statistic_plot/WISE_plot

QSO_select['LW1']

Fermi_4LAC_DR3_h_bll_pd_neogt20['LW1']#['Redshift']

In [ ]:
len(Fermi_4LAC_DR3_h_bll_pd_neogt20)

In [ ]:
len(Fermi_4LAC_DR3_h_bll_pd_neogt20_redshift)

In [ ]:
len(Fermi_4LAC_DR3_h_fsrq_pd_neogt20)

# QSO

In [ ]:
#CLAGN_statistic_plot_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/'
#QSO=pd.read_csv('/Users/brettlv/Downloads/CLAGN_statistic_plot/QSO_wise_var.csv')
QSO=pd.read_csv('%s/QSO_wise_var.csv'%CLAGN_statistic_plot_dir)

QSO['log(L5100)']=(QSO['lg(LHalpha)']-np.log10(5.25)-42)/1.157+44
QSO['log(Lbol)']=np.log10(9)+QSO['log(L5100)']
QSO['log(Ledd)']=np.log10(1.263*10**38)+QSO['lg(L_Mbh)']
QSO['log(Lbol/Ledd)']=QSO['log(Lbol)']-QSO['log(Ledd)']
#print(QSO.columns)
#D_cm_QSO=cosmo.luminosity_distance(z=QSO['redshift']).to(u.cm)
#QSO['D_cm']=cosmo.luminosity_distance(z=QSO['redshift']).to(u.cm)

In [ ]:
for index,z in enumerate(QSO['redshift']):
    d_cm=cosmo.luminosity_distance(z=z).to(u.cm).value
    QSO.loc[index,'D_cm']=d_cm

In [ ]:
QSO['logL_W1']= (-48.6-QSO['mean_m_w1_neo']-2.699)/2.5+np.log10(4*np.pi*QSO['D_cm']**2)+np.log10((constants.c/(3.4*u.um)).to(u.Hz).value)
QSO['logL_W2']= (-48.6-QSO['mean_m_w2_neo']-3.339)/2.5+np.log10(4*np.pi*QSO['D_cm']**2)+np.log10((constants.c/(4.6*u.um)).to(u.Hz).value)


QSO['LW1']=10**QSO['logL_W1']
QSO['LW2']=10**QSO['logL_W2']

In [ ]:
QSO_select=QSO[QSO['len_neo']>20]
#QSO_select=QSO_select[QSO_select['pearson_coeff_neo']>0.8]
QSO_select=drop_index(QSO_select)

In [ ]:
len(QSO_select)

# LLAGN

In [ ]:
LLAGN_gu2009=pd.read_csv('%s/LLAGN_gu2009.csv'%CLAGN_statistic_plot_dir)

LLAGN_gu2009=LLAGN_gu2009[LLAGN_gu2009['len_neo']>20]
LLAGN_gu2009=drop_index(LLAGN_gu2009)

# plot LW1/Lbol

In [ ]:
AGN_wisedata_dir='/Volumes/ExtremeSSD/data_source/WISE_data'

CLAGN_statistic_plot_dir='/Volumes/ExtremeSSD/CLAGN_statistic_plot/'

In [ ]:
CLAGN_210919=pd.read_excel('/Users/lyubing/IHEPBox//data/data_clagn/CLAGN_list210919.xlsx',sheet_name='Sheet1')

In [ ]:
CLAGN_210919.columns

In [ ]:
CLAGN_210919_mass=pd.read_csv('%s/CLAGN_210919_mass_wisevar.csv'%CLAGN_statistic_plot_dir)

In [ ]:
CLAGN_210919_mass#.columns

In [ ]:
!open /Users/lyubing/IHEPBox/data/data_lag/

In [ ]:
#timelag_sources=pd.read_excel('/home/brettlv/Documents/timelag.xlsx',sheet_name='工作表2')
timelag_sources=pd.read_csv('/Users/lyubing/IHEPBox/data/data_lag/Timelag_thiswork.csv')
#timelag_sources=timelag_sources[timelag_sources['select']==1.0]
timelag_sources=timelag_sources.reset_index(drop=True)
timelag_sources['Lbol11']=timelag_sources['Lbol']/10**11/(3.828e26*10**7)
timelag_sources['logLbol11']=np.log10(timelag_sources['Lbol11'])
timelag_sources['logLbol']=np.log10(timelag_sources['Lbol'])

In [ ]:
timelag_sources

In [ ]:
timelag_sources['log(Lw1/Lbol)']=0

In [ ]:
for index,Lbol in enumerate(timelag_sources['logLbol']):
    name=timelag_sources.loc[index,'Name']#.strip().relace(' ','')
    select_= CLAGN_210919_mass[CLAGN_210919_mass['Name']==name]
    if len(select_)>0:
        select_=drop_index(select_)
        Lw1=select_.loc[0,'logL_W1']
        print(name, Lw1-Lbol)
        timelag_sources.loc[index,'log(Lw1/Lbol)']=Lw1-Lbol
        

In [ ]:
timelag_sources

In [ ]:
plt.hist(timelag_sources['log(Lw1/Lbol)'],bins=5)
plt.xlabel('log(Lw1/Lbol)')

In [ ]:
np.log10(0.3),np.log10(0.9)

# hotdog

We report the discovery of the hyperluminous, highly obscured AGN WISE J190445.04+485308.9
(W1904+4853 hereafter, Lbol ∼ 1.1 × 10^13 L⊙) at z = 0.415. 

wiseAGN_hdul=fits.open('/Volumes/ExtremeSSD/Survey_fits/Survey_fits/Survey_fits/AGN/The WISE AGN candidates catalogs/J_ApJS_234_23_r90cat.dat.gz.fits')


wiseAGN_hdul.info()

wiseAGN_data=wiseAGN_hdul[1].data

from astropy.table import Table
t = Table([wiseAGN_data['W1mag']-wiseAGN_data['W2mag'], wiseAGN_data['W2mag']-wiseAGN_data['W3mag'], wiseAGN_data['WISEA']], names=('W1-W2', 'W2-W3', 'WISEA'))
t.write('WISEColor_table1.fits', format='fits')


In [ ]:
wiseAGN_data.columns

In [ ]:
wiseAGN_data['WISEA'][0]

In [ ]:
wiseAGN_data['W2mag'][1],wiseAGN_data['W3mag'][1]

In [ ]:
np.sum(wiseAGN_data['W3mag'])#[:10]

In [ ]:
np.sum(wiseAGN_data['W4mag'])#[:10]

In [ ]:
wiseAGN_data['W3mag'][:10]

wiseAGN_data['W1mag']-wiseAGN_data['W2mag']

wiseAGN_data['W2mag']-wiseAGN_data['W3mag']

In [ ]:
len(WISEColor_table)

In [ ]:
len(WISEColor_table_select)

In [ ]:
WISEColor_table_select_=WISEColor_table_select[WISEColor_table_select['WISEA']=='J190445.04+485308.9']

In [ ]:
WISEColor_table_select_

In [ ]:
wiseAGN_table1_hdul=fits.open('/Volumes/ExtremeSSD/Survey_fits/Survey_fits/Survey_fits/AGN/The WISE AGN candidates catalogs/table1.bin.fits.gz')


In [ ]:
wiseAGN_table1_hdul.info()

In [ ]:
wiseAGN_table1_hdul[0].header

In [ ]:
wiseAGN_data=wiseAGN_table1_hdul[1].data#.columns

In [ ]:
len(wiseAGN_table1_hdul[1].data)

In [ ]:
wiseAGN_data.columns

In [ ]:
#wiseAGN_table1_hdul[1].data['w3mpro'][0]

In [ ]:
from astropy.table import Table
t = Table([wiseAGN_data['w1mpro']-wiseAGN_data['w2mpro'], wiseAGN_data['w2mpro']-wiseAGN_data['w3mpro'], wiseAGN_data['WISE_Designation']], names=('W1-W2', 'W2-W3', 'WISENAME'))
t.write('WISEColor_table1.fits', format='fits')


rm WISEColor_table1.fits

In [ ]:
WISEColor_table=Table.read('WISEColor_table1.fits')

WISEColor_table_select= WISEColor_table[WISEColor_table['W1-W2']>1.9]
WISEColor_table_select=WISEColor_table_select[WISEColor_table_select['W1-W2']<3.0]

WISEColor_table_select=WISEColor_table_select[WISEColor_table_select['W2-W3']<5.3]
WISEColor_table_select=WISEColor_table_select[WISEColor_table_select['W2-W3']>2.9]

In [ ]:
WISEColor_table_select

# 5OXCLAGN

In [ ]:
OX_CLAGN5_nustar=['ESO 362-G18', 'NGC 2992', 'NGC 4151','NGC 7582','NGC 1365']
OX_CLAGN5_redshift=[0.0124,0.0077,0.0033,0.0053,0.0055]
OX_CLAGN5_galNH=[1.8*0.01,5.5*0.01,2.3*0.01,1.3*0.01,1.3*0.01]
OX_CLAGN5_logM=[7.65,7.48,7.56,7.74,6.65]
OX_CLAGN5_inclination=[53,70,58,68,57]

In [ ]:
os.mkdir('/Users/lyubing/Downloads/OXCLAGN/WISEplot')

In [ ]:
!open /Users/lyubing/Downloads/OXCLAGN/WISEplot

In [ ]:
wise_rebin_lc?

In [ ]:
for i in OX_CLAGN5_nustar[:4]:
    download_wise(i,'/Users/lyubing/Downloads/OXCLAGN/WISEplot')
    wise_rebin_lc(i,'/Users/lyubing/Downloads/OXCLAGN/WISEplot','/Users/lyubing/Downloads/OXCLAGN/WISEplot')
    

In [ ]:
name='NGC 1365'
#download_wise(name,'/Users/lyubing/Downloads/OXCLAGN/WISEplot')
wise_rebin_lcandcolor(name,'/Users/lyubing/Downloads/OXCLAGN/WISEplot','/Users/lyubing/Downloads/OXCLAGN/WISEplot')

In [ ]:
for i in OX_CLAGN5_nustar[:5]:
    #download_wise(i,'/Users/lyubing/Downloads/OXCLAGN/WISEplot')
    wise_rebin_lcandcolor(i,'/Users/lyubing/Downloads/OXCLAGN/WISEplot','/Users/lyubing/Downloads/OXCLAGN/WISEplot')
    

In [ ]:
!open /Users/lyubing/Downloads/OXCLAGN/WISEplot

## scaled_fits_dir

In [54]:
scaled_fits_dir='/Users/lyubing/Downloads/LAMOST/PyCali-master_v0/code/lamostz_scaled_fits_dir'

lamostdr10_12_matchSDSS_ztfdata='/Volumes/ExtremeSSD/data_source/ZTFdata/LAMOSTdr10_12'
lamostdr6_9_matchSDSS_ztfdata='/Volumes/ExtremeSSD/data_source/ZTFdata/LAMOST_dr6_9/'
ztf_dir_dr10v1matchSDSSdr16Q='/Volumes/ExtremeSSD/data_source/ZTFdata/dr10v1_match_sdssdr16Q/'

wisedata_path='/Users/lyubing/Downloads/LAMOST/CLAGN_candidates/WISE_lc_CLC/'

dr6to9_dir_lamost_scaled='/Users/lyubing/Downloads/LAMOST/CLAGN_candidates/dr6to9CLAGN_candidates/jin_scaled/LvBin/'
dr10v1galaxy_dir_lamost_unscaled='/Users/lyubing/Downloads/LAMOST/CLAGN_candidates/dr10v1galaxy_matchsdssdr16Q/'


plotdir='/Users/lyubing/Downloads/LAMOST/CLAGN_candidates/spec_ztf_wise_plot/'

## def plot_spec_ztf_wise_lc

## def plot_spec_ztf_wise_lc_dr6and9

In [51]:
def plot_spec_ztf_wise_lc_dr6and9(obsid,ra,dec,z_i,specdir,ztfdir,wisedir,plotdir,marker=".", mec="0.7", ms=6, ecolor="0.7", ls="", zorder=4,**kwargs):
    #filename_lamost='%d.fits'%obsid
    filename_lamost='%d_cali_Jin.fits'%obsid
    file_lamost=os.path.join(specdir,filename_lamost)    
    #scaled_sp= LAMOST_Spec(file_lamost,redshift=[z_i])
    scaled_sp= LAMOST_Spec_Jinscaled(file_lamost,redshift=[z_i])
    
    mjd=scaled_sp.mjd
    
    prop = dict(marker=marker, mec=mec, ms=ms, ecolor=ecolor, ls=ls, zorder=zorder)
    
    
    
    
    ztfsave_name='id%dra%.5f_dec%.5f'%(obsid,ra,dec)
    ztf_file_path='%s/%s.csv'%(ztfdir,ztfsave_name)
    data=pd.read_csv(ztf_file_path) 
    
    data_test_neo,data_test_mep=get_wise_radec_data(ra,dec,datawise_dir=wisedir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100) 

    
    
    fig=plt.figure()
    fig = plt.figure(figsize=(20,8))
    fig.subplots_adjust(hspace=0.0, wspace = 0.2)
    ax = fig.add_subplot(121) 

    ax.plot(scaled_sp.wave/(1+scaled_sp.redshift), scaled_sp.flux)
    ax.set_xlabel(r'Wavelength/(1+z) [$\mathrm{\AA}$]')

    ax.set_ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
    ax.set_title('ra%.5fdec%.5f z=%.3f mjd%d'%(scaled_sp.ra,scaled_sp.dec,scaled_sp.redshift,scaled_sp.mjd))
    
    
    ax1 = fig.add_subplot(222)
    
    
    lc_dataframe = data[data['filtercode']=='zg']
    lc_dataframe=drop_index(lc_dataframe)
    
    lc_dataframe_zr = data[data['filtercode']=='zr']
    lc_dataframe_zr=drop_index(lc_dataframe_zr)
    
    
    
    maxmjd= max(lc_dataframe['mjd'])
    minmjd= min(lc_dataframe['mjd'])
    delta_mjd=maxmjd-minmjd

    if delta_mjd>750: 
        ma_intervals= 365
        min_intervals= 90

    if delta_mjd<750:     
        ma_intervals= delta_mjd//3
        mi_intervals= delta_mjd//6
    
    
    ax1.errorbar(lc_dataframe['mjd'], lc_dataframe["mag"], yerr=lc_dataframe["magerr"],label='zg',
                                **{**prop,**kwargs})
    
    ax1.errorbar(lc_dataframe_zr['mjd'], lc_dataframe_zr["mag"], yerr=lc_dataframe_zr["magerr"],label='zr',
                               **{**prop1,**kwargs})

        
        
    ax1.set_xlabel("MJD")
    #plot_secax(ax1,mi_interval=ma_intervals,ma_interval=min_intervals,rotation=30,)  
    plot_secax(ax1,rotation=30,)  
    ax1.axvline(mjd,color='black',ls=':',linewidth=1)

    handles, labels = ax1.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax1.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    #ax1.set_xlim(58100,59999)
    
    
    ax2 = fig.add_subplot(224,sharex=ax1)
    
    plot_rebinmag_w1(ax2,data_rebin_neo,data_rebin_mep)
    plot_rebinmag_w2(ax2,data_rebin_neo,data_rebin_mep)

    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

    #plot_secax(ax2,mi_interval=365,ma_interval=365*2)
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

    handles, labels = ax2.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    ax1.invert_yaxis()
    ax2.invert_yaxis()

    ax2.set_xlabel('mjd')
    ax1.set_ylabel(r'$mag$')    
    ax2.set_ylabel(r'$mag$')  
    
    save_lc_img_path=os.path.join(plotdir,'%d_z=%.4f.png'%(obsid,z_i))
    if not os.path.exists(save_lc_img_path):
        plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()    
        
    #return fig,ax,ax1,ax2


## def plot_spec_ztf_wise_lc_dr10and12

In [ ]:
def plot_spec_ztf_wise_lc_dr10and12(obsid,ra,dec,z_i,specdir,ztfdir,wisedir,plotdir,marker=".", mec="0.7", ms=6, ls="", zorder=4,**kwargs):
    filename_lamost='%d.fits'%obsid
    file_lamost=os.path.join(specdir,filename_lamost)    
    scaled_sp= LAMOST_Spec(file_lamost,redshift=[z_i])
    mjd=scaled_sp.mjd
    
    #ecolor="0.7",
    
    prop = dict(marker=marker, mec=mec, ms=ms, ecolor='green', ls=ls, zorder=zorder)
    
    prop1 = dict(marker='*', mec=mec, ms=ms, ecolor='red', ls=ls, zorder=zorder)
    
    
    ztfsave_name='id%dra%.5f_dec%.5f'%(obsid,ra,dec)
    ztf_file_path='%s/%s.csv'%(ztfdir,ztfsave_name)
    data=pd.read_csv(ztf_file_path) 
    
    data_test_neo,data_test_mep=get_wise_radec_data(ra,dec,datawise_dir=wisedir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100) 

    
    
    fig=plt.figure()
    fig = plt.figure(figsize=(20,8))
    fig.subplots_adjust(hspace=0.0, wspace = 0.2)
    ax = fig.add_subplot(121) 

    ax.plot(scaled_sp.wave/(1+scaled_sp.redshift), scaled_sp.flux)
    ax.set_xlabel(r'Wavelength/(1+z) [$\mathrm{\AA}$]')

    ax.set_ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
    ax.set_title('ra%.5fdec%.5f z=%.3f mjd%d'%(scaled_sp.ra,scaled_sp.dec,scaled_sp.redshift,scaled_sp.mjd))
    
    
    ax1 = fig.add_subplot(222)
    
    
    lc_dataframe = data[data['filtercode']=='zg']
    lc_dataframe=drop_index(lc_dataframe)
    
    lc_dataframe_zr = data[data['filtercode']=='zr']
    lc_dataframe_zr=drop_index(lc_dataframe_zr)
    
    
    maxmjd= max(lc_dataframe['mjd'])
    minmjd= min(lc_dataframe['mjd'])
    delta_mjd=maxmjd-minmjd

    if delta_mjd>750: 
        ma_intervals= 365
        min_intervals= 90

    if delta_mjd<750:     
        ma_intervals= delta_mjd//3
        mi_intervals= delta_mjd//6
    
    
    ax1.errorbar(lc_dataframe['mjd'], lc_dataframe["mag"], yerr=lc_dataframe["magerr"],label='zg',
                                **{**prop,**kwargs})
    
    ax1.errorbar(lc_dataframe_zr['mjd'], lc_dataframe_zr["mag"], yerr=lc_dataframe_zr["magerr"],label='zr',
                               **{**prop1,**kwargs})
    
    
    
    ax1.set_xlabel("MJD")
    #plot_secax(ax1,mi_interval=ma_intervals,ma_interval=min_intervals,rotation=30,)  
    plot_secax(ax1,rotation=30,)  
    ax1.axvline(mjd,color='black',ls=':',linewidth=1)

    handles, labels = ax1.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax1.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    #ax1.set_xlim(58100,59999)
    
    
    ax2 = fig.add_subplot(224,sharex=ax1)
    
    plot_rebinmag_w1(ax2,data_rebin_neo,data_rebin_mep)
    plot_rebinmag_w2(ax2,data_rebin_neo,data_rebin_mep)

    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

    #plot_secax(ax2,mi_interval=365,ma_interval=365*2)
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

    handles, labels = ax2.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    ax1.invert_yaxis()
    ax2.invert_yaxis()

    ax2.set_xlabel('mjd')
    ax1.set_ylabel(r'$mag$')    
    ax2.set_ylabel(r'$mag$')  
    
    save_lc_img_path=os.path.join(plotdir,'%d_z=%.4f.png'%(obsid,z_i))
    if not os.path.exists(save_lc_img_path):
        plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()    
        
    #return fig,ax,ax1,ax2


## def plot_spec_ztf_wise_lc_dr10galaxy

In [72]:
def plot_spec_ztf_wise_lc_dr10galaxy(obsid,mjdsdss,ra,dec,z_i,specdir,ztfdir,wisedir,plotdir,marker=".", mec="0.7", ms=6, ecolor="0.7", ls="", zorder=4,**kwargs):
    #filename_lamost='%d.fits'%obsid
    filename_lamost='%d_spec.fits'%obsid
    file_lamost=os.path.join(specdir,filename_lamost)    
    #scaled_sp= LAMOST_Spec(file_lamost,redshift=[z_i])
    scaled_sp= LAMOST_Spec(file_lamost,redshift=[z_i])
    
    mjd=scaled_sp.mjd
    
    prop = dict(marker=marker, mec=mec, ms=ms, ecolor=ecolor, ls=ls, zorder=zorder)
    
    prop1 = dict(marker='*', mec=mec, ms=ms, ecolor='red', ls=ls, zorder=zorder)    
    
    
    ztfsave_name='id%dra%.5f_dec%.5f'%(obsid,ra,dec)
    ztf_file_path='%s/%s.csv'%(ztfdir,ztfsave_name)
    data=pd.read_csv(ztf_file_path) 
    
    data_test_neo,data_test_mep=get_wise_radec_data(ra,dec,datawise_dir=wisedir)
    data_rebin_neo,data_rebin_mep=get_rebinwise_data(data_test_neo,data_test_mep,sigma_outlier=100) 

    
    
    fig=plt.figure()
    fig = plt.figure(figsize=(20,8))
    fig.subplots_adjust(hspace=0.0, wspace = 0.2)
    ax = fig.add_subplot(121) 

    ax.plot(scaled_sp.wave/(1+scaled_sp.redshift), scaled_sp.flux)
    ax.set_xlabel(r'Wavelength/(1+z) [$\mathrm{\AA}$]')

    ax.set_ylabel(r'Flux [$\mathrm{10^{-17} erg\;s^{-1}\;cm^{-2}\;\AA^{-1}}$]')
    ax.set_title('ra%.5fdec%.5f z=%.3f mjd%d'%(scaled_sp.ra,scaled_sp.dec,scaled_sp.redshift,scaled_sp.mjd))
    
    
    ax1 = fig.add_subplot(222)
    
    
    lc_dataframe = data[data['filtercode']=='zg']
    lc_dataframe=drop_index(lc_dataframe)
    
    lc_dataframe_zr = data[data['filtercode']=='zr']
    lc_dataframe_zr=drop_index(lc_dataframe_zr)
    
    
    
    maxmjd= max(lc_dataframe['mjd'])
    minmjd= min(lc_dataframe['mjd'])
    delta_mjd=maxmjd-minmjd

    if delta_mjd>750: 
        ma_intervals= 365
        min_intervals= 90

    if delta_mjd<750:     
        ma_intervals= delta_mjd//3
        mi_intervals= delta_mjd//6
    
    
    ax1.errorbar(lc_dataframe['mjd'], lc_dataframe["mag"], yerr=lc_dataframe["magerr"],label='zg',
                                **{**prop,**kwargs})
    
    ax1.errorbar(lc_dataframe_zr['mjd'], lc_dataframe_zr["mag"], yerr=lc_dataframe_zr["magerr"],label='zr',
                               **{**prop1,**kwargs})

        
        
    ax1.set_xlabel("MJD")
    #plot_secax(ax1,mi_interval=ma_intervals,ma_interval=min_intervals,rotation=30,)  
    plot_secax(ax1,rotation=30,)  
    ax1.axvline(mjd,color='black',ls=':',linewidth=1)
    
    ax1.axvline(mjdsdss,color='black',ls='-',linewidth=2)
    

    handles, labels = ax1.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax1.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    #ax1.set_xlim(58100,59999)
    
    
    ax2 = fig.add_subplot(224,sharex=ax1)
    
    plot_rebinmag_w1(ax2,data_rebin_neo,data_rebin_mep)
    plot_rebinmag_w2(ax2,data_rebin_neo,data_rebin_mep)

    #ax.scatter(data_test_neo['mjd'],data_test_neo['w1mpro'],color='red',label='W1')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w1mpro_ep'],color='red',label='W1')

    #plot_secax(ax2,mi_interval=365,ma_interval=365*2)
    #ax.scatter(data_test_neo['mjd'],data_test_neo['w2mpro'],color='blue',label='W2')
    #ax.scatter(data_test_mep['mjd'],data_test_mep['w2mpro_ep'],color='blue',label='W2')

    handles, labels = ax2.get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(0.2, 0.99),ncol=1,
          loc=2,fontsize=10)

    ax1.invert_yaxis()
    ax2.invert_yaxis()

    ax2.set_xlabel('mjd')
    ax1.set_ylabel(r'$mag$')    
    ax2.set_ylabel(r'$mag$')  
    
    save_lc_img_path=os.path.join(plotdir,'%d_z=%.4f.png'%(obsid,z_i))
    if not os.path.exists(save_lc_img_path):
        plt.savefig(save_lc_img_path,dpi=400, transparent=False, bbox_inches='tight')
    plt.close()    
        
    #return fig,ax,ax1,ax2


## Pd_dr10and11_allQSO_CLC

In [ ]:
Pd_dr10and11_allQSO_CLC=pd.read_csv('/Users/lyubing/Downloads/Pd_dr10and11_allQSO_CLC.csv')

In [ ]:
Pd_dr10and11_allQSO_CLC_zr=pd.read_csv('/Users/lyubing/Downloads/LAMOST/Pd_dr10and11_allQSO_CLC1.csv')

In [ ]:
Pd_dr10and11_allQSO_CLC_zr

In [ ]:
Pd_dr10and11_allQSO_CLC#.head

In [ ]:
for index in range(len(Pd_dr10and11_allQSO_CLC)):
    obsid=Pd_dr10and11_allQSO_CLC.loc[index,'OBSID']
    ra=Pd_dr10and11_allQSO_CLC.loc[index,'objra']
    dec=Pd_dr10and11_allQSO_CLC.loc[index,'objdec']
    z_i=Pd_dr10and11_allQSO_CLC.loc[index,'z']
    #fig,ax,ax1,ax2= 
    plot_spec_ztf_wise_lc_dr10and12(obsid,ra,dec,z_i,scaled_fits_dir,lamostdr10_12_matchSDSS_ztfdata,wisedata_path,plotdir)



In [ ]:
for index in range(len(Pd_dr10and11_allQSO_CLC_zr)):
    obsid=Pd_dr10and11_allQSO_CLC_zr.loc[index,'OBSID']
    ra=Pd_dr10and11_allQSO_CLC_zr.loc[index,'objra']
    dec=Pd_dr10and11_allQSO_CLC_zr.loc[index,'objdec']
    z_i=Pd_dr10and11_allQSO_CLC_zr.loc[index,'z']
    #fig,ax,ax1,ax2= 
    
    if len(Pd_dr10and11_allQSO_CLC[Pd_dr10and11_allQSO_CLC['OBSID']==obsid])==0:        
        plot_spec_ztf_wise_lc_dr10and12(obsid,ra,dec,z_i,scaled_fits_dir,lamostdr10_12_matchSDSS_ztfdata,wisedata_path,plotdir)



In [ ]:
obsid=843701018
ra=4.062822
dec=11.098900
z_i=0.393232
specdir=dr6to9_dir_lamost_scaled
ztfdir=lamostdr6_9_matchSDSS_ztfdata
download_wise_withradec(ra,dec,wisedata_path)

plot_spec_ztf_wise_lc_dr6and9(obsid,ra,dec,z_i,specdir,ztfdir,wisedata_path,plotdir)

In [ ]:
239.22698 25.542666
357.82726 6.4480648
163.53826 41.474345

## dr10galaxyclc

In [44]:
dr10galaxyclc=pd.read_csv('/Users/lyubing/Downloads/dr10galaxy_CLc.py',delimiter='\s',header=None)

dr10galaxyclc.columns=['index','obsid','plate','mjd','fiberid','ra','dec','z','L_fwhm','S_fwhm']

dr10galaxyclc_s=dr10galaxyclc[dr10galaxyclc['L_fwhm']==0]
dr10galaxyclc_s=drop_index(dr10galaxyclc_s)

/var/folders/jp/4mwsn7_x6j9gnv1hl1dztqth0000gn/T/ipykernel_6730/3442369713.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dr10galaxyclc=pd.read_csv('/Users/lyubing/Downloads/dr10galaxy_CLc.py',delimiter='\s',header=None)


In [50]:
dr10galaxyclc_s

,index,obsid,plate,mjd,fiberid,ra,dec,z,L_fwhm,S_fwhm
0,2,343710171,1655,53523,204,239.22698,25.542666,0.164468,0.0,7006.490610
1,4,505911087,4405,55854,585,357.82726,6.448065,0.092763,0.0,6571.016275
2,7,632304141,1435,52996,397,163.53826,41.474345,0.230396,0.0,7064.249480


In [65]:
for index in range(len(dr10galaxyclc_s)):
    
    obsid=dr10galaxyclc_s.loc[index,'obsid']
    
    ra=dr10galaxyclc_s.loc[index,'ra']
    dec=dr10galaxyclc_s.loc[index,'dec']
    z_i=dr10galaxyclc_s.loc[index,'z']
    #fig,ax,ax1,ax2= 
    download_wise_withradec(ra,dec,wisedata_path)

In [63]:
wisedata_path

'/Users/lyubing/Downloads/LAMOST/CLAGN_candidates/WISE_lc_CLC/'

In [64]:
!open /Users/lyubing/Downloads/LAMOST/CLAGN_candidates/WISE_lc_CLC/

In [68]:
plotdir_dr10galaxy='/Users/lyubing/Downloads/LAMOST/CLAGN_candidates/spec_ztf_wise_plot_dr10galaxy'

In [74]:
for index in range(len(dr10galaxyclc_s)):
    
    obsid=dr10galaxyclc_s.loc[index,'obsid']
    
    ra=dr10galaxyclc_s.loc[index,'ra']
    dec=dr10galaxyclc_s.loc[index,'dec']
    z_i=dr10galaxyclc_s.loc[index,'z']
    #fig,ax,ax1,ax2= 
    download_wise_withradec(ra,dec,wisedata_path)
    mjdsdss=  dr10galaxyclc_s.loc[index,'mjd']  
    plot_spec_ztf_wise_lc_dr10galaxy(obsid,mjdsdss,ra,dec,z_i,dr10v1galaxy_dir_lamost_unscaled,lamostdr10_12_matchSDSS_ztfdata,wisedata_path,plotdir_dr10galaxy)



/var/folders/jp/4mwsn7_x6j9gnv1hl1dztqth0000gn/T/ipykernel_6730/1592002924.py:60: RuntimeWarning: divide by zero encountered in true_divide
  err = 1./np.sqrt(ivar)


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>